<a href="https://colab.research.google.com/github/NadiaSouza1304/Hack_DS_01/blob/previous_application/Previous_application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Criação de variáveis a partir da tabela Previous_application

## Preparação do Ambiente

### Habilitar o Spark para o colab.


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

###Instalando o pyspark

In [ ]:
!pip install pyspark

In [ ]:
import pandas as pd
import numpy as np

### Mapeando o Google Drive, para possibilitar o acesso aos arquivos de leitura

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### Ler os arquivos e disponibilizar view para manipular com SQL.

O arquivo previous_application.csv: Contém informações sobre aplicações de empréstimo anteriores de um cliente na PoD Bank.

In [39]:
df_previous_application = spark.read.csv('/content/gdrive/MyDrive/HD/CURSOS/Ciência - Análise de Dados/Pod Academy/Formação em Big Data e Analytics/Hackathon - Ciência de Dados/11 - Hackaton/dados/previous_application.csv', header=True,inferSchema=True)
df_previous_application.createOrReplaceTempView('df_previous_application')

#Criação das variáveis - Tabela previous_application

Verificar schema da tabela (metadados).

In [42]:
df_previous_application.printSchema()

root
 |-- SK_ID_PREV: integer (nullable = true)
 |-- SK_ID_CURR: integer (nullable = true)
 |-- NAME_CONTRACT_TYPE: string (nullable = true)
 |-- AMT_ANNUITY: double (nullable = true)
 |-- AMT_APPLICATION: double (nullable = true)
 |-- AMT_CREDIT: double (nullable = true)
 |-- AMT_DOWN_PAYMENT: double (nullable = true)
 |-- AMT_GOODS_PRICE: double (nullable = true)
 |-- WEEKDAY_APPR_PROCESS_START: string (nullable = true)
 |-- HOUR_APPR_PROCESS_START: integer (nullable = true)
 |-- FLAG_LAST_APPL_PER_CONTRACT: string (nullable = true)
 |-- NFLAG_LAST_APPL_IN_DAY: integer (nullable = true)
 |-- RATE_DOWN_PAYMENT: double (nullable = true)
 |-- RATE_INTEREST_PRIMARY: double (nullable = true)
 |-- RATE_INTEREST_PRIVILEGED: double (nullable = true)
 |-- NAME_CASH_LOAN_PURPOSE: string (nullable = true)
 |-- NAME_CONTRACT_STATUS: string (nullable = true)
 |-- DAYS_DECISION: integer (nullable = true)
 |-- NAME_PAYMENT_TYPE: string (nullable = true)
 |-- CODE_REJECT_REASON: string (nullable = t

Mostrar tabela

In [43]:
df_previous_application.show(10,truncate=False)

+----------+----------+------------------+-----------+---------------+----------+----------------+---------------+--------------------------+-----------------------+---------------------------+----------------------+-----------------+---------------------+------------------------+----------------------+--------------------+-------------+---------------------+------------------+---------------+----------------+-------------------+--------------+-----------------+-----------------------+----------------+--------------------+-----------+----------------+------------------------+------------------+--------------+-------------------------+-------------+----------------+-------------------------+
|SK_ID_PREV|SK_ID_CURR|NAME_CONTRACT_TYPE|AMT_ANNUITY|AMT_APPLICATION|AMT_CREDIT|AMT_DOWN_PAYMENT|AMT_GOODS_PRICE|WEEKDAY_APPR_PROCESS_START|HOUR_APPR_PROCESS_START|FLAG_LAST_APPL_PER_CONTRACT|NFLAG_LAST_APPL_IN_DAY|RATE_DOWN_PAYMENT|RATE_INTEREST_PRIMARY|RATE_INTEREST_PRIVILEGED|NAME_CASH_LOAN_PURP

Contagem de linhas totais (Considerando as repetidas)

In [44]:
df_previous_application.count()

1670214

Contagem de linhas distintas

In [45]:
spark.sql("""
SELECT
COUNT(DISTINCT(SK_ID_CURR)) AS Quantidade_SK_ID
FROM
df_previous_application
""").show()

+----------------+
|Quantidade_SK_ID|
+----------------+
|          338857|
+----------------+



### Analisando os dados categóricos da tabela previous_application:

- NAME_CONTRACT_TYPE
- WEEKDAY_APPR_PROCESS_START
- FLAG_LAST_APPL_PER_CONTRACT
- NAME_CASH_LOAN_PURPOSE
- NAME_CONTRACT_STATUS
- NAME_PAYMENT_TYPE
- CODE_REJECT_REASON
- NAME_TYPE_SUITE
- NAME_CLIENT_TYPE
- NAME_GOODS_CATEGORY
- NAME_PORTFOLIO
- NAME_PRODUCT_TYPE
- CHANNEL_TYPE
- NAME_SELLER_INDUSTRY
- NAME_YIELD_GROUP
- PRODUCT_COMBINATION

In [46]:
spark.sql("""
SELECT
SK_ID_CURR,
NAME_CONTRACT_TYPE,
WEEKDAY_APPR_PROCESS_START,
FLAG_LAST_APPL_PER_CONTRACT,
NAME_CASH_LOAN_PURPOSE,
NAME_CONTRACT_STATUS,
NAME_PAYMENT_TYPE,
CODE_REJECT_REASON,
NAME_TYPE_SUITE,
NAME_CLIENT_TYPE,
NAME_GOODS_CATEGORY,
NAME_PORTFOLIO,
NAME_PRODUCT_TYPE,
CHANNEL_TYPE,
NAME_SELLER_INDUSTRY,
NAME_YIELD_GROUP,
PRODUCT_COMBINATION
FROM
df_previous_application
WHERE SK_ID_CURR = '100003'
""").show()

+----------+------------------+--------------------------+---------------------------+----------------------+--------------------+--------------------+------------------+---------------+----------------+--------------------+--------------+-----------------+--------------------+--------------------+----------------+--------------------+
|SK_ID_CURR|NAME_CONTRACT_TYPE|WEEKDAY_APPR_PROCESS_START|FLAG_LAST_APPL_PER_CONTRACT|NAME_CASH_LOAN_PURPOSE|NAME_CONTRACT_STATUS|   NAME_PAYMENT_TYPE|CODE_REJECT_REASON|NAME_TYPE_SUITE|NAME_CLIENT_TYPE| NAME_GOODS_CATEGORY|NAME_PORTFOLIO|NAME_PRODUCT_TYPE|        CHANNEL_TYPE|NAME_SELLER_INDUSTRY|NAME_YIELD_GROUP| PRODUCT_COMBINATION|
+----------+------------------+--------------------------+---------------------------+----------------------+--------------------+--------------------+------------------+---------------+----------------+--------------------+--------------+-----------------+--------------------+--------------------+----------------+--------

Analisando a tabela acima vemos que cada coluna categórica possui domínios variados.

###Identificando todos domínios das variáveis categóricas da tabela previous_application:

- NAME_CONTRACT_TYPE
- WEEKDAY_APPR_PROCESS_START
- FLAG_LAST_APPL_PER_CONTRACT
- NAME_CASH_LOAN_PURPOSE
- NAME_CONTRACT_STATUS
- NAME_PAYMENT_TYPE
- CODE_REJECT_REASON
- NAME_TYPE_SUITE
- NAME_CLIENT_TYPE
- NAME_GOODS_CATEGORY
- NAME_PORTFOLIO
- NAME_PRODUCT_TYPE
- CHANNEL_TYPE
- NAME_SELLER_INDUSTRY
- NAME_YIELD_GROUP
- PRODUCT_COMBINATION

In [47]:
spark.sql("""
SELECT
NAME_CONTRACT_TYPE,
Count(*)
FROM
df_previous_application
GROUP BY NAME_CONTRACT_TYPE
ORDER BY Count(*)
""").show()

+------------------+--------+
|NAME_CONTRACT_TYPE|count(1)|
+------------------+--------+
|               XNA|     346|
|   Revolving loans|  193164|
|    Consumer loans|  729151|
|        Cash loans|  747553|
+------------------+--------+



In [48]:
spark.sql("""
SELECT
WEEKDAY_APPR_PROCESS_START,
Count(*)
FROM
df_previous_application
GROUP BY WEEKDAY_APPR_PROCESS_START
ORDER BY Count(*)
""").show()

+--------------------------+--------+
|WEEKDAY_APPR_PROCESS_START|count(1)|
+--------------------------+--------+
|                    SUNDAY|  164751|
|                  SATURDAY|  240631|
|                  THURSDAY|  249099|
|                    FRIDAY|  252048|
|                    MONDAY|  253557|
|                 WEDNESDAY|  255010|
|                   TUESDAY|  255118|
+--------------------------+--------+



In [49]:
spark.sql("""
SELECT
FLAG_LAST_APPL_PER_CONTRACT,
Count(*)
FROM
df_previous_application
GROUP BY FLAG_LAST_APPL_PER_CONTRACT
ORDER BY Count(*)
""").show()

+---------------------------+--------+
|FLAG_LAST_APPL_PER_CONTRACT|count(1)|
+---------------------------+--------+
|                          N|    8475|
|                          Y| 1661739|
+---------------------------+--------+



In [50]:
spark.sql("""
SELECT
NAME_CASH_LOAN_PURPOSE,
Count(*)
FROM
df_previous_application
GROUP BY NAME_CASH_LOAN_PURPOSE
ORDER BY Count(*)
""").show(truncate=False)

+--------------------------------+--------+
|NAME_CASH_LOAN_PURPOSE          |count(1)|
+--------------------------------+--------+
|Refusal to name the goal        |15      |
|Money for a third person        |25      |
|Hobby                           |55      |
|Buying a garage                 |136     |
|Gasification / water supply     |300     |
|Business development            |426     |
|Buying a holiday home / land    |533     |
|Furniture                       |749     |
|Car repairs                     |797     |
|Buying a home                   |865     |
|Wedding / gift / holiday        |962     |
|Buying a new car                |1012    |
|Purchase of electronic equipment|1061    |
|Journey                         |1239    |
|Education                       |1573    |
|Payments on other loans         |1931    |
|Medicine                        |2174    |
|Everyday expenses               |2416    |
|Building a house or an annex    |2693    |
|Buying a used car              

In [51]:
spark.sql("""
SELECT
NAME_CONTRACT_STATUS,
Count(*)
FROM
df_previous_application
GROUP BY NAME_CONTRACT_STATUS
ORDER BY Count(*)
""").show()

+--------------------+--------+
|NAME_CONTRACT_STATUS|count(1)|
+--------------------+--------+
|        Unused offer|   26436|
|             Refused|  290678|
|            Canceled|  316319|
|            Approved| 1036781|
+--------------------+--------+



In [52]:
spark.sql("""
SELECT
NAME_PAYMENT_TYPE,
Count(*)
FROM
df_previous_application
GROUP BY NAME_PAYMENT_TYPE
ORDER BY Count(*)
""").show(truncate=False)

+-----------------------------------------+--------+
|NAME_PAYMENT_TYPE                        |count(1)|
+-----------------------------------------+--------+
|Cashless from the account of the employer|1085    |
|Non-cash from your account               |8193    |
|XNA                                      |627384  |
|Cash through the bank                    |1033552 |
+-----------------------------------------+--------+



In [53]:
spark.sql("""
SELECT
CODE_REJECT_REASON,
Count(*)
FROM
df_previous_application
GROUP BY CODE_REJECT_REASON
ORDER BY Count(*)
""").show()

+------------------+--------+
|CODE_REJECT_REASON|count(1)|
+------------------+--------+
|            SYSTEM|     717|
|             VERIF|    3535|
|               XNA|    5244|
|             SCOFR|   12811|
|            CLIENT|   26436|
|               SCO|   37467|
|             LIMIT|   55680|
|                HC|  175231|
|               XAP| 1353093|
+------------------+--------+



In [54]:
spark.sql("""
SELECT
NAME_TYPE_SUITE,
Count(*)
FROM
df_previous_application
GROUP BY NAME_TYPE_SUITE
ORDER BY Count(*)
""").show()

+---------------+--------+
|NAME_TYPE_SUITE|count(1)|
+---------------+--------+
|Group of people|    2240|
|        Other_A|    9077|
|        Other_B|   17624|
|       Children|   31566|
|Spouse, partner|   67069|
|         Family|  213263|
|  Unaccompanied|  508970|
|           null|  820405|
+---------------+--------+



In [55]:
spark.sql("""
SELECT
NAME_CLIENT_TYPE,
Count(*)
FROM
df_previous_application
GROUP BY NAME_CLIENT_TYPE
ORDER BY Count(*)
""").show()

+----------------+--------+
|NAME_CLIENT_TYPE|count(1)|
+----------------+--------+
|             XNA|    1941|
|       Refreshed|  135649|
|             New|  301363|
|        Repeater| 1231261|
+----------------+--------+



In [56]:
spark.sql("""
SELECT
NAME_GOODS_CATEGORY,
Count(*)
FROM
df_previous_application
GROUP BY NAME_GOODS_CATEGORY
ORDER BY Count(*)
""").show(truncate=False)

+------------------------+--------+
|NAME_GOODS_CATEGORY     |count(1)|
+------------------------+--------+
|House Construction      |1       |
|Animals                 |1       |
|Insurance               |64      |
|Weapon                  |77      |
|Education               |107     |
|Additional Service      |128     |
|Fitness                 |209     |
|Direct Sales            |446     |
|Medicine                |1550    |
|Tourism                 |1659    |
|Office Appliances       |2333    |
|Other                   |2554    |
|Gardening               |2668    |
|Sport and Leisure       |2981    |
|Vehicles                |3370    |
|Medical Supplies        |3843    |
|Homewares               |5023    |
|Jewelry                 |6290    |
|Auto Accessories        |7381    |
|Clothing and Accessories|23554   |
+------------------------+--------+
only showing top 20 rows



In [57]:
spark.sql("""
SELECT
NAME_PORTFOLIO,
Count(*)
FROM
df_previous_application
GROUP BY NAME_PORTFOLIO
ORDER BY Count(*)
""").show()

+--------------+--------+
|NAME_PORTFOLIO|count(1)|
+--------------+--------+
|          Cars|     425|
|         Cards|  144985|
|           XNA|  372230|
|          Cash|  461563|
|           POS|  691011|
+--------------+--------+



In [58]:
spark.sql("""
SELECT
NAME_PRODUCT_TYPE,
Count(*)
FROM
df_previous_application
GROUP BY NAME_PRODUCT_TYPE
ORDER BY Count(*)
""").show()

+-----------------+--------+
|NAME_PRODUCT_TYPE|count(1)|
+-----------------+--------+
|          walk-in|  150261|
|           x-sell|  456287|
|              XNA| 1063666|
+-----------------+--------+



In [59]:
spark.sql("""
SELECT
CHANNEL_TYPE,
Count(*)
FROM
df_previous_application
GROUP BY CHANNEL_TYPE
ORDER BY Count(*)
""").show(truncate=False)

+--------------------------+--------+
|CHANNEL_TYPE              |count(1)|
+--------------------------+--------+
|Car dealer                |452     |
|Channel of corporate sales|6150    |
|AP+ (Cash loan)           |57046   |
|Contact center            |71297   |
|Regional / Local          |108528  |
|Stone                     |212083  |
|Country-wide              |494690  |
|Credit and cash offices   |719968  |
+--------------------------+--------+



In [60]:
spark.sql("""
SELECT
NAME_SELLER_INDUSTRY,
Count(*)
FROM
df_previous_application
GROUP BY NAME_SELLER_INDUSTRY
ORDER BY Count(*)
""").show()

+--------------------+--------+
|NAME_SELLER_INDUSTRY|count(1)|
+--------------------+--------+
|             Tourism|     513|
|        MLM partners|    1215|
|             Jewelry|    2709|
|     Auto technology|    4990|
|            Industry|   19194|
|            Clothing|   23949|
|        Construction|   29781|
|           Furniture|   57849|
|        Connectivity|  276029|
|Consumer electronics|  398265|
|                 XNA|  855720|
+--------------------+--------+



In [61]:
spark.sql("""
SELECT
NAME_YIELD_GROUP,
Count(*)
FROM
df_previous_application
GROUP BY NAME_YIELD_GROUP
ORDER BY Count(*)
""").show()

+----------------+--------+
|NAME_YIELD_GROUP|count(1)|
+----------------+--------+
|      low_action|   92041|
|      low_normal|  322095|
|            high|  353331|
|          middle|  385532|
|             XNA|  517215|
+----------------+--------+



In [62]:
spark.sql("""
SELECT
PRODUCT_COMBINATION,
Count(*)
FROM
df_previous_application
GROUP BY PRODUCT_COMBINATION
ORDER BY Count(*)
""").show(truncate=False)

+------------------------------+--------+
|PRODUCT_COMBINATION           |count(1)|
+------------------------------+--------+
|null                          |346     |
|POS others without interest   |2555    |
|POS industry without interest |12602   |
|POS other with interest       |23879   |
|POS mobile without interest   |24082   |
|Cash Street: low              |33834   |
|Cash Street: middle           |34658   |
|Cash X-Sell: high             |59301   |
|Cash Street: high             |59639   |
|Card X-Sell                   |80582   |
|POS household without interest|82908   |
|POS industry with interest    |98833   |
|Card Street                   |112582  |
|Cash X-Sell: low              |130248  |
|Cash X-Sell: middle           |143883  |
|POS mobile with interest      |220670  |
|POS household with interest   |263622  |
|Cash                          |285990  |
+------------------------------+--------+



## Agregando apenas as variáveis categóricas e seus domínios


**OBS:** Para automatizar a criação de cada linha de agregação vamos desenvolver uma rotina que gera o texto de agregação. Após, basta susbstiruir na requesição SQL.

###1- Considerando a variável categórica NAME_CONTRACT_TYPE

In [63]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['NAME_CONTRACT_TYPE']

# Criando os domínios

dominios_cat = ['XNA', 'Revolving loans', 'Consumer loans', 'Cash loans']

In [64]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN 1 ELSE 0 END) AS PA_" + ag + "_" + cate + "_" + dominio.replace(' ', '_').replace('/', '').replace('-', '').replace('__', '') +',')

SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_XNA,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Revolving_loans,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Consumer_loans,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Cash_loans,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CONTRACT_TYPE_XNA,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CONTRACT_TYPE_Revolving_loans,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CONTRACT_TYPE_Consumer_loans,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CONTRACT_TYPE_Cash_loans,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS PA_MAX_NAME_CONTRACT_TYPE_XNA,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Revolvin

In [65]:
df_previous_application_NAME_CONTRACT_TYPE = spark.sql("""
SELECT
SK_ID_CURR,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_XNA,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Revolving_loans,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Consumer_loans,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Cash_loans,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CONTRACT_TYPE_XNA,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CONTRACT_TYPE_Revolving_loans,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CONTRACT_TYPE_Consumer_loans,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CONTRACT_TYPE_Cash_loans,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS PA_MAX_NAME_CONTRACT_TYPE_XNA,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN 1 ELSE 0 END) AS PA_MAX_NAME_CONTRACT_TYPE_Revolving_loans,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN 1 ELSE 0 END) AS PA_MAX_NAME_CONTRACT_TYPE_Consumer_loans,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN 1 ELSE 0 END) AS PA_MAX_NAME_CONTRACT_TYPE_Cash_loans,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_XNA,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN 1 ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Revolving_loans,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN 1 ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Consumer_loans,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN 1 ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Cash_loans
FROM
df_previous_application
GROUP BY
  SK_ID_CURR
""")

###2- Considerando a variável categórica WEEKDAY_APPR_PROCESS_START

In [66]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['WEEKDAY_APPR_PROCESS_START']

# Criando os domínios

dominios_cat = ['SUNDAY', 'SATURDAY', 'THURSDAY', 'FRIDAY','MONDAY','WEDNESDAY','TUESDAY']


In [67]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN 1 ELSE 0 END) AS PA_" + ag + "_" + cate + "_" + dominio.replace(' ', '_').replace('/', '').replace('-', '').replace('__', '') +',')

SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN 1 ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN 1 ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SATURDAY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN 1 ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_THURSDAY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN 1 ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_FRIDAY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN 1 ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_MONDAY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN 1 ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_WEDNESDAY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN 1 ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_TUESDAY,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN 1 ELSE 0 END) AS PA_MIN_WEEKDAY_APPR_PROCESS_START_SUNDAY,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN 1 ELSE

In [68]:
df_previous_application_WEEKDAY_APPR_PROCESS_START = spark.sql("""
SELECT
SK_ID_CURR,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN 1 ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN 1 ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SATURDAY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN 1 ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_THURSDAY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN 1 ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_FRIDAY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN 1 ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_MONDAY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN 1 ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_WEDNESDAY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN 1 ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_TUESDAY,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN 1 ELSE 0 END) AS PA_MIN_WEEKDAY_APPR_PROCESS_START_SUNDAY,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN 1 ELSE 0 END) AS PA_MIN_WEEKDAY_APPR_PROCESS_START_SATURDAY,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN 1 ELSE 0 END) AS PA_MIN_WEEKDAY_APPR_PROCESS_START_THURSDAY,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN 1 ELSE 0 END) AS PA_MIN_WEEKDAY_APPR_PROCESS_START_FRIDAY,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN 1 ELSE 0 END) AS PA_MIN_WEEKDAY_APPR_PROCESS_START_MONDAY,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN 1 ELSE 0 END) AS PA_MIN_WEEKDAY_APPR_PROCESS_START_WEDNESDAY,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN 1 ELSE 0 END) AS PA_MIN_WEEKDAY_APPR_PROCESS_START_TUESDAY,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN 1 ELSE 0 END) AS PA_MAX_WEEKDAY_APPR_PROCESS_START_SUNDAY,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN 1 ELSE 0 END) AS PA_MAX_WEEKDAY_APPR_PROCESS_START_SATURDAY,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN 1 ELSE 0 END) AS PA_MAX_WEEKDAY_APPR_PROCESS_START_THURSDAY,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN 1 ELSE 0 END) AS PA_MAX_WEEKDAY_APPR_PROCESS_START_FRIDAY,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN 1 ELSE 0 END) AS PA_MAX_WEEKDAY_APPR_PROCESS_START_MONDAY,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN 1 ELSE 0 END) AS PA_MAX_WEEKDAY_APPR_PROCESS_START_WEDNESDAY,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN 1 ELSE 0 END) AS PA_MAX_WEEKDAY_APPR_PROCESS_START_TUESDAY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN 1 ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_SUNDAY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN 1 ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_SATURDAY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN 1 ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_THURSDAY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN 1 ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_FRIDAY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN 1 ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_MONDAY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN 1 ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_WEDNESDAY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN 1 ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_TUESDAY
FROM
df_previous_application
GROUP BY
  SK_ID_CURR
""")

###3 - Considerando a variável categórica FLAG_LAST_APPL_PER_CONTRACT

In [69]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['FLAG_LAST_APPL_PER_CONTRACT']

# Criando os domínios

dominios_cat = ['N', 'Y']

In [70]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN 1 ELSE 0 END) AS PA_" + ag + "_" + cate + "_" + dominio.replace(' ', '_').replace('/', '').replace('-', '').replace('__', '') +',')

SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN 1 ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_N,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN 1 ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_Y,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN 1 ELSE 0 END) AS PA_MIN_FLAG_LAST_APPL_PER_CONTRACT_N,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN 1 ELSE 0 END) AS PA_MIN_FLAG_LAST_APPL_PER_CONTRACT_Y,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN 1 ELSE 0 END) AS PA_MAX_FLAG_LAST_APPL_PER_CONTRACT_N,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN 1 ELSE 0 END) AS PA_MAX_FLAG_LAST_APPL_PER_CONTRACT_Y,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN 1 ELSE 0 END) AS PA_AVG_FLAG_LAST_APPL_PER_CONTRACT_N,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN 1 ELSE 0 END) AS PA_AVG_FLAG_LAST_APPL_PER_CONTRACT_Y,


In [71]:
df_previous_application_FLAG_LAST_APPL_PER_CONTRACT = spark.sql("""
SELECT
SK_ID_CURR,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN 1 ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_N,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN 1 ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_Y,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN 1 ELSE 0 END) AS PA_MIN_FLAG_LAST_APPL_PER_CONTRACT_N,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN 1 ELSE 0 END) AS PA_MIN_FLAG_LAST_APPL_PER_CONTRACT_Y,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN 1 ELSE 0 END) AS PA_MAX_FLAG_LAST_APPL_PER_CONTRACT_N,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN 1 ELSE 0 END) AS PA_MAX_FLAG_LAST_APPL_PER_CONTRACT_Y,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN 1 ELSE 0 END) AS PA_AVG_FLAG_LAST_APPL_PER_CONTRACT_N,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN 1 ELSE 0 END) AS PA_AVG_FLAG_LAST_APPL_PER_CONTRACT_Y
FROM
df_previous_application
GROUP BY
  SK_ID_CURR
""")

###4 - Considerando a variável categórica NAME_CASH_LOAN_PURPOSE

In [72]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['NAME_CASH_LOAN_PURPOSE']

# Criando os domínios

dominios_cat = ['Refusal to name the goal', 'Money for a third person', 'Hobby', 'Buying a garage', 'Gasification / water supply', 'Business development', 'Buying a holiday home / land', 'Furniture', 'Car repairs', 'Buying a home', 'Wedding / gift / holiday', 'Buying a new car', 'Purchase of electronic equipment', 'Journey', 'Education', 'Payments on other loans', 'Medicine', 'Everyday expenses', 'Building a house or an annex', 'Buying a used car']


In [73]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN 1 ELSE 0 END) AS PA_" + ag + "_" + cate + "_" + dominio.replace(' ', '_').replace('/', '').replace('-', '').replace('__', '') +',')

SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Hobby,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_garage,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Business_development,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN 

In [74]:
df_previous_application_NAME_CASH_LOAN_PURPOSE = spark.sql("""
SELECT
SK_ID_CURR,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Hobby,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_garage,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Business_development,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Furniture,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car repairs' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Car_repairs,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a home' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_home,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding / gift / holiday' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Weddinggiftholiday,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a new car' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase of electronic equipment' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Journey,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Education,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments on other loans' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Medicine,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday expenses' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Everyday_expenses,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building a house or an annex' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a used car' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car,
MIN(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal,
MIN(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person,
MIN(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CASH_LOAN_PURPOSE_Hobby,
MIN(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CASH_LOAN_PURPOSE_Buying_a_garage,
MIN(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply,
MIN(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CASH_LOAN_PURPOSE_Business_development,
MIN(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland,
MIN(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CASH_LOAN_PURPOSE_Furniture,
MIN(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car repairs' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CASH_LOAN_PURPOSE_Car_repairs,
MIN(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a home' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CASH_LOAN_PURPOSE_Buying_a_home,
MIN(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding / gift / holiday' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CASH_LOAN_PURPOSE_Weddinggiftholiday,
MIN(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a new car' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car,
MIN(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase of electronic equipment' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment,
MIN(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CASH_LOAN_PURPOSE_Journey,
MIN(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CASH_LOAN_PURPOSE_Education,
MIN(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments on other loans' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans,
MIN(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CASH_LOAN_PURPOSE_Medicine,
MIN(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday expenses' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CASH_LOAN_PURPOSE_Everyday_expenses,
MIN(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building a house or an annex' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex,
MIN(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a used car' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car,
MAX(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN 1 ELSE 0 END) AS PA_MAX_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal,
MAX(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN 1 ELSE 0 END) AS PA_MAX_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person,
MAX(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN 1 ELSE 0 END) AS PA_MAX_NAME_CASH_LOAN_PURPOSE_Hobby,
MAX(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN 1 ELSE 0 END) AS PA_MAX_NAME_CASH_LOAN_PURPOSE_Buying_a_garage,
MAX(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN 1 ELSE 0 END) AS PA_MAX_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply,
MAX(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN 1 ELSE 0 END) AS PA_MAX_NAME_CASH_LOAN_PURPOSE_Business_development,
MAX(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN 1 ELSE 0 END) AS PA_MAX_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland,
MAX(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN 1 ELSE 0 END) AS PA_MAX_NAME_CASH_LOAN_PURPOSE_Furniture,
MAX(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car repairs' THEN 1 ELSE 0 END) AS PA_MAX_NAME_CASH_LOAN_PURPOSE_Car_repairs,
MAX(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a home' THEN 1 ELSE 0 END) AS PA_MAX_NAME_CASH_LOAN_PURPOSE_Buying_a_home,
MAX(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding / gift / holiday' THEN 1 ELSE 0 END) AS PA_MAX_NAME_CASH_LOAN_PURPOSE_Weddinggiftholiday,
MAX(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a new car' THEN 1 ELSE 0 END) AS PA_MAX_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car,
MAX(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase of electronic equipment' THEN 1 ELSE 0 END) AS PA_MAX_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment,
MAX(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN 1 ELSE 0 END) AS PA_MAX_NAME_CASH_LOAN_PURPOSE_Journey,
MAX(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN 1 ELSE 0 END) AS PA_MAX_NAME_CASH_LOAN_PURPOSE_Education,
MAX(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments on other loans' THEN 1 ELSE 0 END) AS PA_MAX_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans,
MAX(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN 1 ELSE 0 END) AS PA_MAX_NAME_CASH_LOAN_PURPOSE_Medicine,
MAX(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday expenses' THEN 1 ELSE 0 END) AS PA_MAX_NAME_CASH_LOAN_PURPOSE_Everyday_expenses,
MAX(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building a house or an annex' THEN 1 ELSE 0 END) AS PA_MAX_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex,
MAX(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a used car' THEN 1 ELSE 0 END) AS PA_MAX_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN 1 ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN 1 ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN 1 ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Hobby,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN 1 ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_garage,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN 1 ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN 1 ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Business_development,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN 1 ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN 1 ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Furniture,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car repairs' THEN 1 ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Car_repairs,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a home' THEN 1 ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_home,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding / gift / holiday' THEN 1 ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Weddinggiftholiday,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a new car' THEN 1 ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase of electronic equipment' THEN 1 ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN 1 ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Journey,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN 1 ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Education,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments on other loans' THEN 1 ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN 1 ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Medicine,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday expenses' THEN 1 ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Everyday_expenses,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building a house or an annex' THEN 1 ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a used car' THEN 1 ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car
FROM
df_previous_application
GROUP BY
  SK_ID_CURR
""")

###5 - Considerando a variável categórica NAME_CONTRACT_STATUS

In [75]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['NAME_CONTRACT_STATUS']

# Criando os domínios

dominios_cat = ['Unused offer', 'Refused', 'Canceled', 'Approved']

In [76]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN 1 ELSE 0 END) AS PA_" + ag + "_" + cate + "_" + dominio.replace(' ', '_').replace('/', '').replace('-', '').replace('__', '') +',')

SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Unused_offer,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Refused,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Canceled,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Approved,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CONTRACT_STATUS_Unused_offer,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CONTRACT_STATUS_Refused,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CONTRACT_STATUS_Canceled,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CONTRACT_STATUS_Approved,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN 1 ELSE 0 END) AS PA_MAX_NAME_CONTRACT_STATUS_Unused_offer,
MAX(CASE WHEN NAM

In [77]:
df_previous_application_NAME_CONTRACT_STATUS = spark.sql("""
SELECT
SK_ID_CURR,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Unused_offer,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Refused,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Canceled,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Approved,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CONTRACT_STATUS_Unused_offer,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CONTRACT_STATUS_Refused,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CONTRACT_STATUS_Canceled,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CONTRACT_STATUS_Approved,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN 1 ELSE 0 END) AS PA_MAX_NAME_CONTRACT_STATUS_Unused_offer,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN 1 ELSE 0 END) AS PA_MAX_NAME_CONTRACT_STATUS_Refused,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN 1 ELSE 0 END) AS PA_MAX_NAME_CONTRACT_STATUS_Canceled,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN 1 ELSE 0 END) AS PA_MAX_NAME_CONTRACT_STATUS_Approved,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN 1 ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Unused_offer,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN 1 ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Refused,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN 1 ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Canceled,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN 1 ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Approved
FROM
df_previous_application
GROUP BY
  SK_ID_CURR
""")

###6 - Considerando a variável categórica NAME_PAYMENT_TYPE

In [78]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['NAME_PAYMENT_TYPE']

# Criando os domínios

dominios_cat = ['Cashless from the account of the employer', 'Non cash from your account', 'XNA', 'Cash through the bank']


In [79]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN 1 ELSE 0 END) AS PA_" + ag + "_" + cate + "_" + dominio.replace(' ', '_').replace('/', '').replace('-', '').replace('__', '') +',')

SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN 1 ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN 1 ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Non_cash_from_your_account,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_XNA,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN 1 ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cash_through_the_bank,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN 1 ELSE 0 END) AS PA_MIN_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN 1 ELSE 0 END) AS PA_MIN_NAME_PAYMENT_TYPE_Non_cash_from_your_account,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS PA_MIN_NAME_PAYMENT_TYPE_XNA,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN 1 ELSE 0 END) AS PA_M

In [80]:
df_previous_application_NAME_PAYMENT_TYPE = spark.sql("""
SELECT
SK_ID_CURR,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN 1 ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN 1 ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Non_cash_from_your_account,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_XNA,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN 1 ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cash_through_the_bank,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN 1 ELSE 0 END) AS PA_MIN_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN 1 ELSE 0 END) AS PA_MIN_NAME_PAYMENT_TYPE_Non_cash_from_your_account,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS PA_MIN_NAME_PAYMENT_TYPE_XNA,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN 1 ELSE 0 END) AS PA_MIN_NAME_PAYMENT_TYPE_Cash_through_the_bank,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN 1 ELSE 0 END) AS PA_MAX_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN 1 ELSE 0 END) AS PA_MAX_NAME_PAYMENT_TYPE_Non_cash_from_your_account,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS PA_MAX_NAME_PAYMENT_TYPE_XNA,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN 1 ELSE 0 END) AS PA_MAX_NAME_PAYMENT_TYPE_Cash_through_the_bank,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN 1 ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN 1 ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Non_cash_from_your_account,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_XNA,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN 1 ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Cash_through_the_bank
FROM
df_previous_application
GROUP BY
  SK_ID_CURR
""")

###7 - Considerando a variável categórica CODE_REJECT_REASON

In [81]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['CODE_REJECT_REASON']

# Criando os domínios

dominios_cat = ['SYSTEM', 'VERIF', 'XNA', 'SCOFR','CLIENT','SCO','LIMIT', 'HC','XAP']

In [82]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN 1 ELSE 0 END) AS PA_" + ag + "_" + cate + "_" + dominio.replace(' ', '_').replace('/', '').replace('-', '').replace('__', '') +',')

SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN 1 ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SYSTEM,
SUM(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN 1 ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_VERIF,
SUM(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN 1 ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_XNA,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN 1 ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SCOFR,
SUM(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN 1 ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_CLIENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN 1 ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SCO,
SUM(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN 1 ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_LIMIT,
SUM(CASE WHEN CODE_REJECT_REASON= 'HC' THEN 1 ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_HC,
SUM(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN 1 ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_XAP,
MIN(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN 1 ELSE 0 END) AS PA_MIN_CODE_REJECT_REASON_SYSTEM,
MIN(CASE WHEN CODE_REJECT_REASON= 'VERIF' 

In [83]:
df_previous_application_CODE_REJECT_REASON = spark.sql("""
SELECT
SK_ID_CURR,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN 1 ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SYSTEM,
SUM(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN 1 ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_VERIF,
SUM(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN 1 ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_XNA,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN 1 ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SCOFR,
SUM(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN 1 ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_CLIENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN 1 ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SCO,
SUM(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN 1 ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_LIMIT,
SUM(CASE WHEN CODE_REJECT_REASON= 'HC' THEN 1 ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_HC,
SUM(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN 1 ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_XAP,
MIN(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN 1 ELSE 0 END) AS PA_MIN_CODE_REJECT_REASON_SYSTEM,
MIN(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN 1 ELSE 0 END) AS PA_MIN_CODE_REJECT_REASON_VERIF,
MIN(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN 1 ELSE 0 END) AS PA_MIN_CODE_REJECT_REASON_XNA,
MIN(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN 1 ELSE 0 END) AS PA_MIN_CODE_REJECT_REASON_SCOFR,
MIN(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN 1 ELSE 0 END) AS PA_MIN_CODE_REJECT_REASON_CLIENT,
MIN(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN 1 ELSE 0 END) AS PA_MIN_CODE_REJECT_REASON_SCO,
MIN(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN 1 ELSE 0 END) AS PA_MIN_CODE_REJECT_REASON_LIMIT,
MIN(CASE WHEN CODE_REJECT_REASON= 'HC' THEN 1 ELSE 0 END) AS PA_MIN_CODE_REJECT_REASON_HC,
MIN(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN 1 ELSE 0 END) AS PA_MIN_CODE_REJECT_REASON_XAP,
MAX(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN 1 ELSE 0 END) AS PA_MAX_CODE_REJECT_REASON_SYSTEM,
MAX(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN 1 ELSE 0 END) AS PA_MAX_CODE_REJECT_REASON_VERIF,
MAX(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN 1 ELSE 0 END) AS PA_MAX_CODE_REJECT_REASON_XNA,
MAX(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN 1 ELSE 0 END) AS PA_MAX_CODE_REJECT_REASON_SCOFR,
MAX(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN 1 ELSE 0 END) AS PA_MAX_CODE_REJECT_REASON_CLIENT,
MAX(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN 1 ELSE 0 END) AS PA_MAX_CODE_REJECT_REASON_SCO,
MAX(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN 1 ELSE 0 END) AS PA_MAX_CODE_REJECT_REASON_LIMIT,
MAX(CASE WHEN CODE_REJECT_REASON= 'HC' THEN 1 ELSE 0 END) AS PA_MAX_CODE_REJECT_REASON_HC,
MAX(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN 1 ELSE 0 END) AS PA_MAX_CODE_REJECT_REASON_XAP,
AVG(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN 1 ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SYSTEM,
AVG(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN 1 ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_VERIF,
AVG(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN 1 ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_XNA,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN 1 ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SCOFR,
AVG(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN 1 ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_CLIENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN 1 ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SCO,
AVG(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN 1 ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_LIMIT,
AVG(CASE WHEN CODE_REJECT_REASON= 'HC' THEN 1 ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_HC,
AVG(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN 1 ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_XAP
FROM
df_previous_application
GROUP BY
  SK_ID_CURR
""")

###8 - Considerando a variável categórica NAME_TYPE_SUITE

In [84]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['NAME_TYPE_SUITE']

# Criando os domínios

dominios_cat = ['Group of people', 'Other_A', 'Other_B', 'Children','Spouse, partner','Family','Unaccompanied', 'null']


In [85]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN 1 ELSE 0 END) AS PA_" + ag + "_" + cate + "_" + dominio.replace(' ', '_').replace('/', '').replace('-', '').replace('__', '') +',')

SUM(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN 1 ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Group_of_people,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN 1 ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Other_A,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN 1 ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Other_B,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN 1 ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Children,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN 1 ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Spouse,_partner,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN 1 ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Family,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN 1 ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Unaccompanied,
SUM(CASE WHEN NAME_TYPE_SUITE= 'null' THEN 1 ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_null,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN 1 ELSE 0 END) AS PA_MIN_NAME_TYPE_SUITE_Group_of_people,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN 1 ELSE 0 END) AS PA_MIN_NAME_TYPE_SUITE_Other

In [86]:
df_previous_application_NAME_TYPE_SUITE = spark.sql("""
SELECT
SK_ID_CURR,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN 1 ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Group_of_people,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN 1 ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Other_A,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN 1 ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Other_B,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN 1 ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Children,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN 1 ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Spouse_partner,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN 1 ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Family,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN 1 ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Unaccompanied,
SUM(CASE WHEN NAME_TYPE_SUITE= 'null' THEN 1 ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_null,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN 1 ELSE 0 END) AS PA_MIN_NAME_TYPE_SUITE_Group_of_people,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN 1 ELSE 0 END) AS PA_MIN_NAME_TYPE_SUITE_Other_A,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN 1 ELSE 0 END) AS PA_MIN_NAME_TYPE_SUITE_Other_B,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN 1 ELSE 0 END) AS PA_MIN_NAME_TYPE_SUITE_Children,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN 1 ELSE 0 END) AS PA_MIN_NAME_TYPE_SUITE_Spouse_partner,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN 1 ELSE 0 END) AS PA_MIN_NAME_TYPE_SUITE_Family,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN 1 ELSE 0 END) AS PA_MIN_NAME_TYPE_SUITE_Unaccompanied,
MIN(CASE WHEN NAME_TYPE_SUITE= 'null' THEN 1 ELSE 0 END) AS PA_MIN_NAME_TYPE_SUITE_null,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN 1 ELSE 0 END) AS PA_MAX_NAME_TYPE_SUITE_Group_of_people,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN 1 ELSE 0 END) AS PA_MAX_NAME_TYPE_SUITE_Other_A,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN 1 ELSE 0 END) AS PA_MAX_NAME_TYPE_SUITE_Other_B,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN 1 ELSE 0 END) AS PA_MAX_NAME_TYPE_SUITE_Children,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN 1 ELSE 0 END) AS PA_MAX_NAME_TYPE_SUITE_Spouse_partner,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN 1 ELSE 0 END) AS PA_MAX_NAME_TYPE_SUITE_Family,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN 1 ELSE 0 END) AS PA_MAX_NAME_TYPE_SUITE_Unaccompanied,
MAX(CASE WHEN NAME_TYPE_SUITE= 'null' THEN 1 ELSE 0 END) AS PA_MAX_NAME_TYPE_SUITE_null,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN 1 ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Group_of_people,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN 1 ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Other_A,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN 1 ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Other_B,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN 1 ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Children,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN 1 ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Spouse_partner,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN 1 ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Family,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN 1 ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Unaccompanied,
AVG(CASE WHEN NAME_TYPE_SUITE= 'null' THEN 1 ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_null
FROM
df_previous_application
GROUP BY
  SK_ID_CURR
""")

###9 - Considerando a variável categórica NAME_CLIENT_TYPE

In [87]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['NAME_CLIENT_TYPE']

# Criando os domínios

dominios_cat = ['XNA', 'Refreshed', 'New', 'Repeater']

In [88]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN 1 ELSE 0 END) AS PA_" + ag + "_" + cate + "_" + dominio.replace(' ', '_').replace('/', '').replace('-', '').replace('__', '') +',')

SUM(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_XNA,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_Refreshed,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_New,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_Repeater,
MIN(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CLIENT_TYPE_XNA,
MIN(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CLIENT_TYPE_Refreshed,
MIN(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CLIENT_TYPE_New,
MIN(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CLIENT_TYPE_Repeater,
MAX(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS PA_MAX_NAME_CLIENT_TYPE_XNA,
MAX(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN 1 ELSE 0 END) AS PA_MAX_NAME_CLIENT_TYPE_Refreshed,
MAX(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN 1 ELSE 0 EN

In [89]:
df_previous_application_NAME_CLIENT_TYPE = spark.sql("""
SELECT
SK_ID_CURR,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_XNA,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_Refreshed,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_New,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN 1 ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_Repeater,
MIN(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CLIENT_TYPE_XNA,
MIN(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CLIENT_TYPE_Refreshed,
MIN(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CLIENT_TYPE_New,
MIN(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN 1 ELSE 0 END) AS PA_MIN_NAME_CLIENT_TYPE_Repeater,
MAX(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS PA_MAX_NAME_CLIENT_TYPE_XNA,
MAX(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN 1 ELSE 0 END) AS PA_MAX_NAME_CLIENT_TYPE_Refreshed,
MAX(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN 1 ELSE 0 END) AS PA_MAX_NAME_CLIENT_TYPE_New,
MAX(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN 1 ELSE 0 END) AS PA_MAX_NAME_CLIENT_TYPE_Repeater,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_XNA,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN 1 ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_Refreshed,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN 1 ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_New,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN 1 ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_Repeater
FROM
df_previous_application
GROUP BY
  SK_ID_CURR
""")

###10 - Considerando a variável categórica NAME_GOODS_CATEGORY

In [90]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['NAME_GOODS_CATEGORY']

# Criando os domínios

dominios_cat = ['House Construction', 'Animals', 'Insurance', 'Education','Additional Service','Fitness','Direct Sales', 'Medicine', 'Tourism','Office Appliances','Other','Gardening','Sport and Leisure','Vehicles','Medical Supplies','Homewares','Jewelry','Auto Accessories','Clothing and Accessories']


In [91]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN 1 ELSE 0 END) AS PA_" + ag + "_" + cate + "_" + dominio.replace(' ', '_').replace('/', '').replace('-', '').replace('__', '') +',')

SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN 1 ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_House_Construction,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN 1 ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Animals,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN 1 ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Insurance,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN 1 ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Education,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN 1 ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Additional_Service,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN 1 ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Fitness,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN 1 ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Direct_Sales,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN 1 ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Medicine,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN 1 ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Tourism,
SUM(CASE 

In [92]:
df_previous_application_NAME_GOODS_CATEGORY = spark.sql("""
SELECT
SK_ID_CURR,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN 1 ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_House_Construction,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN 1 ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Animals,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN 1 ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Insurance,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN 1 ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Education,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN 1 ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Additional_Service,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN 1 ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Fitness,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN 1 ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Direct_Sales,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN 1 ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Medicine,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN 1 ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Tourism,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Office Appliances' THEN 1 ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Office_Appliances,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN 1 ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Other,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN 1 ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Gardening,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Sport and Leisure' THEN 1 ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Sport_and_Leisure,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN 1 ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Vehicles,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medical Supplies' THEN 1 ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Medical_Supplies,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN 1 ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Homewares,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN 1 ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Jewelry,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Auto Accessories' THEN 1 ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Auto_Accessories,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing and Accessories' THEN 1 ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Clothing_and_Accessories,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN 1 ELSE 0 END) AS PA_MIN_NAME_GOODS_CATEGORY_House_Construction,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN 1 ELSE 0 END) AS PA_MIN_NAME_GOODS_CATEGORY_Animals,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN 1 ELSE 0 END) AS PA_MIN_NAME_GOODS_CATEGORY_Insurance,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN 1 ELSE 0 END) AS PA_MIN_NAME_GOODS_CATEGORY_Education,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN 1 ELSE 0 END) AS PA_MIN_NAME_GOODS_CATEGORY_Additional_Service,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN 1 ELSE 0 END) AS PA_MIN_NAME_GOODS_CATEGORY_Fitness,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN 1 ELSE 0 END) AS PA_MIN_NAME_GOODS_CATEGORY_Direct_Sales,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN 1 ELSE 0 END) AS PA_MIN_NAME_GOODS_CATEGORY_Medicine,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN 1 ELSE 0 END) AS PA_MIN_NAME_GOODS_CATEGORY_Tourism,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Office Appliances' THEN 1 ELSE 0 END) AS PA_MIN_NAME_GOODS_CATEGORY_Office_Appliances,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN 1 ELSE 0 END) AS PA_MIN_NAME_GOODS_CATEGORY_Other,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN 1 ELSE 0 END) AS PA_MIN_NAME_GOODS_CATEGORY_Gardening,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Sport and Leisure' THEN 1 ELSE 0 END) AS PA_MIN_NAME_GOODS_CATEGORY_Sport_and_Leisure,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN 1 ELSE 0 END) AS PA_MIN_NAME_GOODS_CATEGORY_Vehicles,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Medical Supplies' THEN 1 ELSE 0 END) AS PA_MIN_NAME_GOODS_CATEGORY_Medical_Supplies,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN 1 ELSE 0 END) AS PA_MIN_NAME_GOODS_CATEGORY_Homewares,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN 1 ELSE 0 END) AS PA_MIN_NAME_GOODS_CATEGORY_Jewelry,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Auto Accessories' THEN 1 ELSE 0 END) AS PA_MIN_NAME_GOODS_CATEGORY_Auto_Accessories,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing and Accessories' THEN 1 ELSE 0 END) AS PA_MIN_NAME_GOODS_CATEGORY_Clothing_and_Accessories,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN 1 ELSE 0 END) AS PA_MAX_NAME_GOODS_CATEGORY_House_Construction,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN 1 ELSE 0 END) AS PA_MAX_NAME_GOODS_CATEGORY_Animals,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN 1 ELSE 0 END) AS PA_MAX_NAME_GOODS_CATEGORY_Insurance,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN 1 ELSE 0 END) AS PA_MAX_NAME_GOODS_CATEGORY_Education,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN 1 ELSE 0 END) AS PA_MAX_NAME_GOODS_CATEGORY_Additional_Service,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN 1 ELSE 0 END) AS PA_MAX_NAME_GOODS_CATEGORY_Fitness,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN 1 ELSE 0 END) AS PA_MAX_NAME_GOODS_CATEGORY_Direct_Sales,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN 1 ELSE 0 END) AS PA_MAX_NAME_GOODS_CATEGORY_Medicine,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN 1 ELSE 0 END) AS PA_MAX_NAME_GOODS_CATEGORY_Tourism,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Office Appliances' THEN 1 ELSE 0 END) AS PA_MAX_NAME_GOODS_CATEGORY_Office_Appliances,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN 1 ELSE 0 END) AS PA_MAX_NAME_GOODS_CATEGORY_Other,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN 1 ELSE 0 END) AS PA_MAX_NAME_GOODS_CATEGORY_Gardening,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Sport and Leisure' THEN 1 ELSE 0 END) AS PA_MAX_NAME_GOODS_CATEGORY_Sport_and_Leisure,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN 1 ELSE 0 END) AS PA_MAX_NAME_GOODS_CATEGORY_Vehicles,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Medical Supplies' THEN 1 ELSE 0 END) AS PA_MAX_NAME_GOODS_CATEGORY_Medical_Supplies,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN 1 ELSE 0 END) AS PA_MAX_NAME_GOODS_CATEGORY_Homewares,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN 1 ELSE 0 END) AS PA_MAX_NAME_GOODS_CATEGORY_Jewelry,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Auto Accessories' THEN 1 ELSE 0 END) AS PA_MAX_NAME_GOODS_CATEGORY_Auto_Accessories,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing and Accessories' THEN 1 ELSE 0 END) AS PA_MAX_NAME_GOODS_CATEGORY_Clothing_and_Accessories,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN 1 ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_House_Construction,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN 1 ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Animals,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN 1 ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Insurance,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN 1 ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Education,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN 1 ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Additional_Service,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN 1 ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Fitness,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN 1 ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Direct_Sales,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN 1 ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Medicine,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN 1 ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Tourism,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Office Appliances' THEN 1 ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Office_Appliances,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN 1 ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Other,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN 1 ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Gardening,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Sport and Leisure' THEN 1 ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Sport_and_Leisure,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN 1 ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Vehicles,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medical Supplies' THEN 1 ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Medical_Supplies,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN 1 ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Homewares,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN 1 ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Jewelry,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Auto Accessories' THEN 1 ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Auto_Accessories,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing and Accessories' THEN 1 ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Clothing_and_Accessories
FROM
df_previous_application
GROUP BY
  SK_ID_CURR
""")

###11 - Considerando a variável categórica NAME_PORTFOLIO

In [93]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['NAME_PORTFOLIO']

# Criando os domínios

dominios_cat = ['Cars', 'Cards', 'XNA', 'Cash','POS']

In [94]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN 1 ELSE 0 END) AS PA_" + ag + "_" + cate + "_" + dominio.replace(' ', '_').replace('/', '').replace('-', '').replace('__', '') +',')

SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN 1 ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cars,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN 1 ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cards,
SUM(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN 1 ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_XNA,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN 1 ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cash,
SUM(CASE WHEN NAME_PORTFOLIO= 'POS' THEN 1 ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_POS,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN 1 ELSE 0 END) AS PA_MIN_NAME_PORTFOLIO_Cars,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN 1 ELSE 0 END) AS PA_MIN_NAME_PORTFOLIO_Cards,
MIN(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN 1 ELSE 0 END) AS PA_MIN_NAME_PORTFOLIO_XNA,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN 1 ELSE 0 END) AS PA_MIN_NAME_PORTFOLIO_Cash,
MIN(CASE WHEN NAME_PORTFOLIO= 'POS' THEN 1 ELSE 0 END) AS PA_MIN_NAME_PORTFOLIO_POS,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN 1 ELSE 0 END) AS PA_MAX_NAME_PORTFOLIO_Cars,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN 1 EL

In [95]:
df_previous_application_NAME_PORTFOLIO = spark.sql("""
SELECT
SK_ID_CURR,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN 1 ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cars,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN 1 ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cards,
SUM(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN 1 ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_XNA,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN 1 ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cash,
SUM(CASE WHEN NAME_PORTFOLIO= 'POS' THEN 1 ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_POS,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN 1 ELSE 0 END) AS PA_MIN_NAME_PORTFOLIO_Cars,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN 1 ELSE 0 END) AS PA_MIN_NAME_PORTFOLIO_Cards,
MIN(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN 1 ELSE 0 END) AS PA_MIN_NAME_PORTFOLIO_XNA,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN 1 ELSE 0 END) AS PA_MIN_NAME_PORTFOLIO_Cash,
MIN(CASE WHEN NAME_PORTFOLIO= 'POS' THEN 1 ELSE 0 END) AS PA_MIN_NAME_PORTFOLIO_POS,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN 1 ELSE 0 END) AS PA_MAX_NAME_PORTFOLIO_Cars,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN 1 ELSE 0 END) AS PA_MAX_NAME_PORTFOLIO_Cards,
MAX(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN 1 ELSE 0 END) AS PA_MAX_NAME_PORTFOLIO_XNA,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN 1 ELSE 0 END) AS PA_MAX_NAME_PORTFOLIO_Cash,
MAX(CASE WHEN NAME_PORTFOLIO= 'POS' THEN 1 ELSE 0 END) AS PA_MAX_NAME_PORTFOLIO_POS,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN 1 ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cars,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN 1 ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cards,
AVG(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN 1 ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_XNA,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN 1 ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cash,
AVG(CASE WHEN NAME_PORTFOLIO= 'POS' THEN 1 ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_POS
FROM
df_previous_application
GROUP BY
  SK_ID_CURR
""")

###12 - Considerando a variável categórica NAME_PRODUCT_TYPE

In [96]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['NAME_PRODUCT_TYPE']

# Criando os domínios

dominios_cat = ['walk-in', 'x-sell', 'XNA']

In [97]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN 1 ELSE 0 END) AS PA_" + ag + "_" + cate + "_" + dominio.replace(' ', '_').replace('/', '').replace('-', '').replace('__', '') +',')

SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN 1 ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_walkin,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN 1 ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_xsell,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_XNA,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN 1 ELSE 0 END) AS PA_MIN_NAME_PRODUCT_TYPE_walkin,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN 1 ELSE 0 END) AS PA_MIN_NAME_PRODUCT_TYPE_xsell,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS PA_MIN_NAME_PRODUCT_TYPE_XNA,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN 1 ELSE 0 END) AS PA_MAX_NAME_PRODUCT_TYPE_walkin,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN 1 ELSE 0 END) AS PA_MAX_NAME_PRODUCT_TYPE_xsell,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS PA_MAX_NAME_PRODUCT_TYPE_XNA,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN 1 ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_walkin,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN 

In [98]:
df_previous_application_NAME_PRODUCT_TYPE = spark.sql("""
SELECT
SK_ID_CURR,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN 1 ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_walkin,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN 1 ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_xsell,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_XNA,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN 1 ELSE 0 END) AS PA_MIN_NAME_PRODUCT_TYPE_walkin,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN 1 ELSE 0 END) AS PA_MIN_NAME_PRODUCT_TYPE_xsell,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS PA_MIN_NAME_PRODUCT_TYPE_XNA,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN 1 ELSE 0 END) AS PA_MAX_NAME_PRODUCT_TYPE_walkin,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN 1 ELSE 0 END) AS PA_MAX_NAME_PRODUCT_TYPE_xsell,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS PA_MAX_NAME_PRODUCT_TYPE_XNA,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN 1 ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_walkin,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN 1 ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_xsell,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_XNA
FROM
df_previous_application
GROUP BY
  SK_ID_CURR
""")

###13 - Considerando a variável categórica CHANNEL_TYPE

In [99]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['CHANNEL_TYPE']

# Criando os domínios

dominios_cat = ['Car dealer', 'Channel of corporate sales', 'AP+ (Cash loan)','Contact center','Regional / Local','Stone','Country-wide','Credit and cash offices']


In [100]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN 1 ELSE 0 END) AS PA_" + ag + "_" + cate + "_" + dominio.replace(' ', '_').replace('/', '').replace('+', '').replace('(', '').replace(')', '').replace('-', '').replace('__', '') +',')

SUM(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN 1 ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Car_dealer,
SUM(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN 1 ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Channel_of_corporate_sales,
SUM(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN 1 ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_AP_Cash_loan,
SUM(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN 1 ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Contact_center,
SUM(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN 1 ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_RegionalLocal,
SUM(CASE WHEN CHANNEL_TYPE= 'Stone' THEN 1 ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Stone,
SUM(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN 1 ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Countrywide,
SUM(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN 1 ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Credit_and_cash_offices,
MIN(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN 1 ELSE 0 END) AS PA_MIN_CHANNEL_TYPE_Car_dealer,
MIN(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN 1 ELSE 0

In [101]:
df_previous_application_CHANNEL_TYPE = spark.sql("""
SELECT
SK_ID_CURR,
SUM(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN 1 ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Car_dealer,
SUM(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN 1 ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Channel_of_corporate_sales,
SUM(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN 1 ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_AP_Cash_loan,
SUM(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN 1 ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Contact_center,
SUM(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN 1 ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_RegionalLocal,
SUM(CASE WHEN CHANNEL_TYPE= 'Stone' THEN 1 ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Stone,
SUM(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN 1 ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Countrywide,
SUM(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN 1 ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Credit_and_cash_offices,
MIN(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN 1 ELSE 0 END) AS PA_MIN_CHANNEL_TYPE_Car_dealer,
MIN(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN 1 ELSE 0 END) AS PA_MIN_CHANNEL_TYPE_Channel_of_corporate_sales,
MIN(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN 1 ELSE 0 END) AS PA_MIN_CHANNEL_TYPE_AP_Cash_loan,
MIN(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN 1 ELSE 0 END) AS PA_MIN_CHANNEL_TYPE_Contact_center,
MIN(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN 1 ELSE 0 END) AS PA_MIN_CHANNEL_TYPE_RegionalLocal,
MIN(CASE WHEN CHANNEL_TYPE= 'Stone' THEN 1 ELSE 0 END) AS PA_MIN_CHANNEL_TYPE_Stone,
MIN(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN 1 ELSE 0 END) AS PA_MIN_CHANNEL_TYPE_Countrywide,
MIN(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN 1 ELSE 0 END) AS PA_MIN_CHANNEL_TYPE_Credit_and_cash_offices,
MAX(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN 1 ELSE 0 END) AS PA_MAX_CHANNEL_TYPE_Car_dealer,
MAX(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN 1 ELSE 0 END) AS PA_MAX_CHANNEL_TYPE_Channel_of_corporate_sales,
MAX(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN 1 ELSE 0 END) AS PA_MAX_CHANNEL_TYPE_AP_Cash_loan,
MAX(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN 1 ELSE 0 END) AS PA_MAX_CHANNEL_TYPE_Contact_center,
MAX(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN 1 ELSE 0 END) AS PA_MAX_CHANNEL_TYPE_RegionalLocal,
MAX(CASE WHEN CHANNEL_TYPE= 'Stone' THEN 1 ELSE 0 END) AS PA_MAX_CHANNEL_TYPE_Stone,
MAX(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN 1 ELSE 0 END) AS PA_MAX_CHANNEL_TYPE_Countrywide,
MAX(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN 1 ELSE 0 END) AS PA_MAX_CHANNEL_TYPE_Credit_and_cash_offices,
AVG(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN 1 ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Car_dealer,
AVG(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN 1 ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Channel_of_corporate_sales,
AVG(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN 1 ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_AP_Cash_loan,
AVG(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN 1 ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Contact_center,
AVG(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN 1 ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_RegionalLocal,
AVG(CASE WHEN CHANNEL_TYPE= 'Stone' THEN 1 ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Stone,
AVG(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN 1 ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Countrywide,
AVG(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN 1 ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Credit_and_cash_offices
FROM
df_previous_application
GROUP BY
  SK_ID_CURR
""")

###14 - Considerando a variável categórica NAME_SELLER_INDUSTRY

In [102]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['NAME_SELLER_INDUSTRY']

# Criando os domínios

dominios_cat = ['MLM partners', 'Jewelry', 'Auto technology','Industry','Clothing','Construction','Furniture','Connectivity','Consumer electronics','XNA']


In [103]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN 1 ELSE 0 END) AS PA_" + ag + "_" + cate + "_" + dominio.replace(' ', '_').replace('/', '').replace('-', '').replace('__', '') +',')

SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN 1 ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_MLM_partners,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN 1 ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Jewelry,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN 1 ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Auto_technology,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN 1 ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Industry,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN 1 ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Clothing,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN 1 ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Construction,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN 1 ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Furniture,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN 1 ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Connectivity,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN 1 ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUST

In [104]:
df_previous_application_NAME_SELLER_INDUSTRY= spark.sql("""
SELECT
SK_ID_CURR,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN 1 ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_MLM_partners,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN 1 ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Jewelry,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN 1 ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Auto_technology,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN 1 ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Industry,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN 1 ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Clothing,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN 1 ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Construction,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN 1 ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Furniture,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN 1 ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Connectivity,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN 1 ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Consumer_electronics,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN 1 ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_XNA,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN 1 ELSE 0 END) AS PA_MIN_NAME_SELLER_INDUSTRY_MLM_partners,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN 1 ELSE 0 END) AS PA_MIN_NAME_SELLER_INDUSTRY_Jewelry,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN 1 ELSE 0 END) AS PA_MIN_NAME_SELLER_INDUSTRY_Auto_technology,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN 1 ELSE 0 END) AS PA_MIN_NAME_SELLER_INDUSTRY_Industry,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN 1 ELSE 0 END) AS PA_MIN_NAME_SELLER_INDUSTRY_Clothing,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN 1 ELSE 0 END) AS PA_MIN_NAME_SELLER_INDUSTRY_Construction,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN 1 ELSE 0 END) AS PA_MIN_NAME_SELLER_INDUSTRY_Furniture,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN 1 ELSE 0 END) AS PA_MIN_NAME_SELLER_INDUSTRY_Connectivity,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN 1 ELSE 0 END) AS PA_MIN_NAME_SELLER_INDUSTRY_Consumer_electronics,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN 1 ELSE 0 END) AS PA_MIN_NAME_SELLER_INDUSTRY_XNA,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN 1 ELSE 0 END) AS PA_MAX_NAME_SELLER_INDUSTRY_MLM_partners,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN 1 ELSE 0 END) AS PA_MAX_NAME_SELLER_INDUSTRY_Jewelry,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN 1 ELSE 0 END) AS PA_MAX_NAME_SELLER_INDUSTRY_Auto_technology,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN 1 ELSE 0 END) AS PA_MAX_NAME_SELLER_INDUSTRY_Industry,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN 1 ELSE 0 END) AS PA_MAX_NAME_SELLER_INDUSTRY_Clothing,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN 1 ELSE 0 END) AS PA_MAX_NAME_SELLER_INDUSTRY_Construction,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN 1 ELSE 0 END) AS PA_MAX_NAME_SELLER_INDUSTRY_Furniture,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN 1 ELSE 0 END) AS PA_MAX_NAME_SELLER_INDUSTRY_Connectivity,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN 1 ELSE 0 END) AS PA_MAX_NAME_SELLER_INDUSTRY_Consumer_electronics,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN 1 ELSE 0 END) AS PA_MAX_NAME_SELLER_INDUSTRY_XNA,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN 1 ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_MLM_partners,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN 1 ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Jewelry,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN 1 ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Auto_technology,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN 1 ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Industry,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN 1 ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Clothing,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN 1 ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Construction,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN 1 ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Furniture,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN 1 ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Connectivity,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN 1 ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Consumer_electronics,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN 1 ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_XNA
FROM
df_previous_application
GROUP BY
  SK_ID_CURR
""")

###15 - Considerando a variável categórica NAME_YIELD_GROUP

In [105]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['NAME_YIELD_GROUP']

# Criando os domínios

dominios_cat = ['low_action', 'low_normal', 'high','middle','XNA']

In [106]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN 1 ELSE 0 END) AS PA_" + ag + "_" + cate + "_" + dominio.replace(' ', '_').replace('/', '').replace('-', '').replace('__', '') +',')

SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN 1 ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_action,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN 1 ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_normal,
SUM(CASE WHEN NAME_YIELD_GROUP= 'high' THEN 1 ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_high,
SUM(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN 1 ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_middle,
SUM(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN 1 ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_XNA,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN 1 ELSE 0 END) AS PA_MIN_NAME_YIELD_GROUP_low_action,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN 1 ELSE 0 END) AS PA_MIN_NAME_YIELD_GROUP_low_normal,
MIN(CASE WHEN NAME_YIELD_GROUP= 'high' THEN 1 ELSE 0 END) AS PA_MIN_NAME_YIELD_GROUP_high,
MIN(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN 1 ELSE 0 END) AS PA_MIN_NAME_YIELD_GROUP_middle,
MIN(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN 1 ELSE 0 END) AS PA_MIN_NAME_YIELD_GROUP_XNA,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_a

In [107]:
df_previous_application_NAME_YIELD_GROUP = spark.sql("""
SELECT
SK_ID_CURR,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN 1 ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_action,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN 1 ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_normal,
SUM(CASE WHEN NAME_YIELD_GROUP= 'high' THEN 1 ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_high,
SUM(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN 1 ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_middle,
SUM(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN 1 ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_XNA,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN 1 ELSE 0 END) AS PA_MIN_NAME_YIELD_GROUP_low_action,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN 1 ELSE 0 END) AS PA_MIN_NAME_YIELD_GROUP_low_normal,
MIN(CASE WHEN NAME_YIELD_GROUP= 'high' THEN 1 ELSE 0 END) AS PA_MIN_NAME_YIELD_GROUP_high,
MIN(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN 1 ELSE 0 END) AS PA_MIN_NAME_YIELD_GROUP_middle,
MIN(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN 1 ELSE 0 END) AS PA_MIN_NAME_YIELD_GROUP_XNA,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN 1 ELSE 0 END) AS PA_MAX_NAME_YIELD_GROUP_low_action,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN 1 ELSE 0 END) AS PA_MAX_NAME_YIELD_GROUP_low_normal,
MAX(CASE WHEN NAME_YIELD_GROUP= 'high' THEN 1 ELSE 0 END) AS PA_MAX_NAME_YIELD_GROUP_high,
MAX(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN 1 ELSE 0 END) AS PA_MAX_NAME_YIELD_GROUP_middle,
MAX(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN 1 ELSE 0 END) AS PA_MAX_NAME_YIELD_GROUP_XNA,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN 1 ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_low_action,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN 1 ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_low_normal,
AVG(CASE WHEN NAME_YIELD_GROUP= 'high' THEN 1 ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_high,
AVG(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN 1 ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_middle,
AVG(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN 1 ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_XNA
FROM
df_previous_application
GROUP BY
  SK_ID_CURR
""")

###16 - Considerando a variável categórica PRODUCT_COMBINATION

In [108]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['PRODUCT_COMBINATION']

# Criando os domínios

dominios_cat = ['null', 'POS others without interest', 'POS industry without interest','POS other with interest','POS mobile without interest','Cash Street: low','Cash Street: middle','Cash X-Sell: high','Cash Street: high', 'Card X-Sell','POS household without interest','POS industry with interest','Card Street','Cash X-Sell: low','Cash X-Sell: middle','POS mobile with interest','POS household with interest','Cash']


In [109]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN 1 ELSE 0 END) AS PA_" + ag + "_" + cate + "_" + dominio.replace(' ', '_').replace('/', '').replace('+', '').replace('(', '').replace(')', '').replace(':', '').replace('-', '').replace('__', '') +',')

SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN 1 ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_null,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN 1 ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_others_without_interest,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN 1 ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_industry_without_interest,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN 1 ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_other_with_interest,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN 1 ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_mobile_without_interest,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN 1 ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_low,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN 1 ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_middle,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN 1 ELSE 0 END) AS PA_SUM_PRODU

In [110]:
df_previous_application_PRODUCT_COMBINATION = spark.sql("""
SELECT
SK_ID_CURR,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN 1 ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_null,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN 1 ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_others_without_interest,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN 1 ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_industry_without_interest,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN 1 ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_other_with_interest,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN 1 ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_mobile_without_interest,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN 1 ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_low,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN 1 ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_middle,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN 1 ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_high,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: high' THEN 1 ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_high,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card X-Sell' THEN 1 ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Card_XSell,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS household without interest' THEN 1 ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_household_without_interest,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS industry with interest' THEN 1 ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_industry_with_interest,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card Street' THEN 1 ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Card_Street,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: low' THEN 1 ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_low,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: middle' THEN 1 ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_middle,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS mobile with interest' THEN 1 ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_mobile_with_interest,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS household with interest' THEN 1 ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_household_with_interest,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN 1 ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash,
MIN(CASE WHEN PRODUCT_COMBINATION= 'null' THEN 1 ELSE 0 END) AS PA_MIN_PRODUCT_COMBINATION_null,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN 1 ELSE 0 END) AS PA_MIN_PRODUCT_COMBINATION_POS_others_without_interest,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN 1 ELSE 0 END) AS PA_MIN_PRODUCT_COMBINATION_POS_industry_without_interest,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN 1 ELSE 0 END) AS PA_MIN_PRODUCT_COMBINATION_POS_other_with_interest,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN 1 ELSE 0 END) AS PA_MIN_PRODUCT_COMBINATION_POS_mobile_without_interest,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN 1 ELSE 0 END) AS PA_MIN_PRODUCT_COMBINATION_Cash_Street_low,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN 1 ELSE 0 END) AS PA_MIN_PRODUCT_COMBINATION_Cash_Street_middle,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN 1 ELSE 0 END) AS PA_MIN_PRODUCT_COMBINATION_Cash_XSell_high,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: high' THEN 1 ELSE 0 END) AS PA_MIN_PRODUCT_COMBINATION_Cash_Street_high,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Card X-Sell' THEN 1 ELSE 0 END) AS PA_MIN_PRODUCT_COMBINATION_Card_XSell,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS household without interest' THEN 1 ELSE 0 END) AS PA_MIN_PRODUCT_COMBINATION_POS_household_without_interest,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS industry with interest' THEN 1 ELSE 0 END) AS PA_MIN_PRODUCT_COMBINATION_POS_industry_with_interest,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Card Street' THEN 1 ELSE 0 END) AS PA_MIN_PRODUCT_COMBINATION_Card_Street,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: low' THEN 1 ELSE 0 END) AS PA_MIN_PRODUCT_COMBINATION_Cash_XSell_low,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: middle' THEN 1 ELSE 0 END) AS PA_MIN_PRODUCT_COMBINATION_Cash_XSell_middle,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS mobile with interest' THEN 1 ELSE 0 END) AS PA_MIN_PRODUCT_COMBINATION_POS_mobile_with_interest,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS household with interest' THEN 1 ELSE 0 END) AS PA_MIN_PRODUCT_COMBINATION_POS_household_with_interest,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN 1 ELSE 0 END) AS PA_MIN_PRODUCT_COMBINATION_Cash,
MAX(CASE WHEN PRODUCT_COMBINATION= 'null' THEN 1 ELSE 0 END) AS PA_MAX_PRODUCT_COMBINATION_null,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN 1 ELSE 0 END) AS PA_MAX_PRODUCT_COMBINATION_POS_others_without_interest,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN 1 ELSE 0 END) AS PA_MAX_PRODUCT_COMBINATION_POS_industry_without_interest,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN 1 ELSE 0 END) AS PA_MAX_PRODUCT_COMBINATION_POS_other_with_interest,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN 1 ELSE 0 END) AS PA_MAX_PRODUCT_COMBINATION_POS_mobile_without_interest,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN 1 ELSE 0 END) AS PA_MAX_PRODUCT_COMBINATION_Cash_Street_low,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN 1 ELSE 0 END) AS PA_MAX_PRODUCT_COMBINATION_Cash_Street_middle,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN 1 ELSE 0 END) AS PA_MAX_PRODUCT_COMBINATION_Cash_XSell_high,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: high' THEN 1 ELSE 0 END) AS PA_MAX_PRODUCT_COMBINATION_Cash_Street_high,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Card X-Sell' THEN 1 ELSE 0 END) AS PA_MAX_PRODUCT_COMBINATION_Card_XSell,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS household without interest' THEN 1 ELSE 0 END) AS PA_MAX_PRODUCT_COMBINATION_POS_household_without_interest,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS industry with interest' THEN 1 ELSE 0 END) AS PA_MAX_PRODUCT_COMBINATION_POS_industry_with_interest,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Card Street' THEN 1 ELSE 0 END) AS PA_MAX_PRODUCT_COMBINATION_Card_Street,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: low' THEN 1 ELSE 0 END) AS PA_MAX_PRODUCT_COMBINATION_Cash_XSell_low,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: middle' THEN 1 ELSE 0 END) AS PA_MAX_PRODUCT_COMBINATION_Cash_XSell_middle,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS mobile with interest' THEN 1 ELSE 0 END) AS PA_MAX_PRODUCT_COMBINATION_POS_mobile_with_interest,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS household with interest' THEN 1 ELSE 0 END) AS PA_MAX_PRODUCT_COMBINATION_POS_household_with_interest,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN 1 ELSE 0 END) AS PA_MAX_PRODUCT_COMBINATION_Cash,
AVG(CASE WHEN PRODUCT_COMBINATION= 'null' THEN 1 ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_null,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN 1 ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_others_without_interest,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN 1 ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_industry_without_interest,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN 1 ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_other_with_interest,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN 1 ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_mobile_without_interest,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN 1 ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_low,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN 1 ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_middle,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN 1 ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_high,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: high' THEN 1 ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_high,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card X-Sell' THEN 1 ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Card_XSell,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS household without interest' THEN 1 ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_household_without_interest,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS industry with interest' THEN 1 ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_industry_with_interest,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card Street' THEN 1 ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Card_Street,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: low' THEN 1 ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_low,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: middle' THEN 1 ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_middle,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS mobile with interest' THEN 1 ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_mobile_with_interest,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS household with interest' THEN 1 ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_household_with_interest,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN 1 ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash
FROM
df_previous_application
GROUP BY
  SK_ID_CURR
""")

##Criação do dataframe df_previous_application_cat somente com agregações de variáveis categóricas

In [111]:
df_previous_application_cat = df_previous_application_NAME_CONTRACT_TYPE.join(df_previous_application_WEEKDAY_APPR_PROCESS_START, "SK_ID_CURR",  how = 'left')\
               .join(df_previous_application_FLAG_LAST_APPL_PER_CONTRACT, "SK_ID_CURR",  how = 'left')\
               .join(df_previous_application_NAME_CASH_LOAN_PURPOSE, "SK_ID_CURR",  how = 'left')\
               .join(df_previous_application_NAME_CONTRACT_STATUS, "SK_ID_CURR",  how = 'left')\
               .join(df_previous_application_NAME_PAYMENT_TYPE, "SK_ID_CURR",  how = 'left')\
               .join(df_previous_application_CODE_REJECT_REASON, "SK_ID_CURR",  how = 'left')\
               .join(df_previous_application_NAME_TYPE_SUITE, "SK_ID_CURR",  how = 'left')\
               .join(df_previous_application_NAME_CLIENT_TYPE, "SK_ID_CURR",  how = 'left')\
               .join(df_previous_application_NAME_GOODS_CATEGORY, "SK_ID_CURR",  how = 'left')\
               .join(df_previous_application_NAME_PORTFOLIO, "SK_ID_CURR",  how = 'left')\
               .join(df_previous_application_NAME_PRODUCT_TYPE, "SK_ID_CURR",  how = 'left')\
               .join(df_previous_application_CHANNEL_TYPE, "SK_ID_CURR",  how = 'left')\
               .join(df_previous_application_NAME_SELLER_INDUSTRY, "SK_ID_CURR",  how = 'left')\
               .join(df_previous_application_NAME_YIELD_GROUP, "SK_ID_CURR",  how = 'left')\
               .join(df_previous_application_PRODUCT_COMBINATION, "SK_ID_CURR",  how = 'left')


In [112]:
df_previous_application_cat.show()

+----------+-----------------------------+-----------------------------------------+----------------------------------------+------------------------------------+-----------------------------+-----------------------------------------+----------------------------------------+------------------------------------+-----------------------------+-----------------------------------------+----------------------------------------+------------------------------------+-----------------------------+-----------------------------------------+----------------------------------------+------------------------------------+----------------------------------------+------------------------------------------+------------------------------------------+----------------------------------------+----------------------------------------+-------------------------------------------+-----------------------------------------+----------------------------------------+------------------------------------------+---------

In [113]:
# Obtenha a lista de todas as colunas
column_list = df_previous_application_cat.columns

# Crie um dicionário para contar as ocorrências
column_dict = {i:column_list.count(i) for i in column_list}

# Encontre as colunas que aparecem mais de uma vez
duplicated_columns = [k for k, v in column_dict.items() if v > 1]

print("Colunas duplicadas: ", duplicated_columns)

Colunas duplicadas:  []


In [114]:
df_previous_application_cat.createOrReplaceTempView("df_previous_application_cat")

In [115]:
#df_previous_application_cat.write.csv('/content/gdrive/MyDrive/HD/CURSOS/Ciência - Análise de Dados/Pod Academy/Formação em Big Data e Analytics/Hackathon - Ciência de Dados/11 - Hackaton/dados_etl/df_previous_application_cat.csv')

In [116]:
#df_previous_application_cat.coalesce(1).write.csv('/content/gdrive/MyDrive/11 - Hackaton/dados_etl/df_previous_application_cat.csv')

##Transformando as variáveis numéricas em agregações

Após criar o dataframe df_previous_application_cat somente com agregações de variáveis categóricas, podemos criar outros dataframes com agregações entre as variáveis categóricas e numéricas. As variáveis numéricas da tabela df_previous_application são:

OBS: Para automatizar a criação de cada linha de agregação vamos desenvolver uma rotina que gera o texto de agregação. Após, basta susbstiruir na requesição SQL.


- AMT_ANNUITY: double
- AMT_APPLICATION: double
- AMT_CREDIT: double
- AMT_DOWN_PAYMENT: double
- AMT_GOODS_PRICE: double
- HOUR_APPR_PROCESS_START: integer
- NFLAG_LAST_APPL_IN_DAY: integer
- RATE_DOWN_PAYMENT: double
- RATE_INTEREST_PRIMARY: double
- RATE_INTEREST_PRIVILEGED: double
- DAYS_DECISION: integer
- SELLERPLACE_AREA: integer
- CNT_PAYMENT: double
- DAYS_FIRST_DRAWING: double
- DAYS_FIRST_DUE: double
- DAYS_LAST_DUE_1ST_VERSION: double
- DAYS_LAST_DUE: double
- DAYS_TERMINATION: double
- NFLAG_INSURED_ON_APPROVAL: double


In [117]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável numérica e de data

var_num = ['AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE', 'HOUR_APPR_PROCESS_START', 'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'DAYS_DECISION', 'SELLERPLACE_AREA', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL']

In [118]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for var_n in var_num:
        print(ag + "(" + var_n + ") AS PA_" + ag + "_" + var_n + ',')

SUM(AMT_ANNUITY) AS PA_SUM_AMT_ANNUITY,
SUM(AMT_APPLICATION) AS PA_SUM_AMT_APPLICATION,
SUM(AMT_CREDIT) AS PA_SUM_AMT_CREDIT,
SUM(AMT_DOWN_PAYMENT) AS PA_SUM_AMT_DOWN_PAYMENT,
SUM(AMT_GOODS_PRICE) AS PA_SUM_AMT_GOODS_PRICE,
SUM(HOUR_APPR_PROCESS_START) AS PA_SUM_HOUR_APPR_PROCESS_START,
SUM(NFLAG_LAST_APPL_IN_DAY) AS PA_SUM_NFLAG_LAST_APPL_IN_DAY,
SUM(RATE_DOWN_PAYMENT) AS PA_SUM_RATE_DOWN_PAYMENT,
SUM(RATE_INTEREST_PRIMARY) AS PA_SUM_RATE_INTEREST_PRIMARY,
SUM(RATE_INTEREST_PRIVILEGED) AS PA_SUM_RATE_INTEREST_PRIVILEGED,
SUM(DAYS_DECISION) AS PA_SUM_DAYS_DECISION,
SUM(SELLERPLACE_AREA) AS PA_SUM_SELLERPLACE_AREA,
SUM(CNT_PAYMENT) AS PA_SUM_CNT_PAYMENT,
SUM(DAYS_FIRST_DRAWING) AS PA_SUM_DAYS_FIRST_DRAWING,
SUM(DAYS_FIRST_DUE) AS PA_SUM_DAYS_FIRST_DUE,
SUM(DAYS_LAST_DUE_1ST_VERSION) AS PA_SUM_DAYS_LAST_DUE_1ST_VERSION,
SUM(DAYS_LAST_DUE) AS PA_SUM_DAYS_LAST_DUE,
SUM(DAYS_TERMINATION) AS PA_SUM_DAYS_TERMINATION,
SUM(NFLAG_INSURED_ON_APPROVAL) AS PA_SUM_NFLAG_INSURED_ON_APPROVAL,
MIN(AMT_

##Criação do dataframe df_previous_application_num somente com agregações de variáveis numericas

In [119]:
df_previous_application_num = spark.sql("""
SELECT
SK_ID_CURR,
SUM(AMT_ANNUITY) AS PA_SUM_AMT_ANNUITY,
SUM(AMT_APPLICATION) AS PA_SUM_AMT_APPLICATION,
SUM(AMT_CREDIT) AS PA_SUM_AMT_CREDIT,
SUM(AMT_DOWN_PAYMENT) AS PA_SUM_AMT_DOWN_PAYMENT,
SUM(AMT_GOODS_PRICE) AS PA_SUM_AMT_GOODS_PRICE,
SUM(HOUR_APPR_PROCESS_START) AS PA_SUM_HOUR_APPR_PROCESS_START,
SUM(NFLAG_LAST_APPL_IN_DAY) AS PA_SUM_NFLAG_LAST_APPL_IN_DAY,
SUM(RATE_DOWN_PAYMENT) AS PA_SUM_RATE_DOWN_PAYMENT,
SUM(RATE_INTEREST_PRIMARY) AS PA_SUM_RATE_INTEREST_PRIMARY,
SUM(RATE_INTEREST_PRIVILEGED) AS PA_SUM_RATE_INTEREST_PRIVILEGED,
SUM(DAYS_DECISION) AS PA_SUM_DAYS_DECISION,
SUM(SELLERPLACE_AREA) AS PA_SUM_SELLERPLACE_AREA,
SUM(CNT_PAYMENT) AS PA_SUM_CNT_PAYMENT,
SUM(DAYS_FIRST_DRAWING) AS PA_SUM_DAYS_FIRST_DRAWING,
SUM(DAYS_FIRST_DUE) AS PA_SUM_DAYS_FIRST_DUE,
SUM(DAYS_LAST_DUE_1ST_VERSION) AS PA_SUM_DAYS_LAST_DUE_1ST_VERSION,
SUM(DAYS_LAST_DUE) AS PA_SUM_DAYS_LAST_DUE,
SUM(DAYS_TERMINATION) AS PA_SUM_DAYS_TERMINATION,
SUM(NFLAG_INSURED_ON_APPROVAL) AS PA_SUM_NFLAG_INSURED_ON_APPROVAL,
MIN(AMT_ANNUITY) AS PA_MIN_AMT_ANNUITY,
MIN(AMT_APPLICATION) AS PA_MIN_AMT_APPLICATION,
MIN(AMT_CREDIT) AS PA_MIN_AMT_CREDIT,
MIN(AMT_DOWN_PAYMENT) AS PA_MIN_AMT_DOWN_PAYMENT,
MIN(AMT_GOODS_PRICE) AS PA_MIN_AMT_GOODS_PRICE,
MIN(HOUR_APPR_PROCESS_START) AS PA_MIN_HOUR_APPR_PROCESS_START,
MIN(NFLAG_LAST_APPL_IN_DAY) AS PA_MIN_NFLAG_LAST_APPL_IN_DAY,
MIN(RATE_DOWN_PAYMENT) AS PA_MIN_RATE_DOWN_PAYMENT,
MIN(RATE_INTEREST_PRIMARY) AS PA_MIN_RATE_INTEREST_PRIMARY,
MIN(RATE_INTEREST_PRIVILEGED) AS PA_MIN_RATE_INTEREST_PRIVILEGED,
MIN(DAYS_DECISION) AS PA_MIN_DAYS_DECISION,
MIN(SELLERPLACE_AREA) AS PA_MIN_SELLERPLACE_AREA,
MIN(CNT_PAYMENT) AS PA_MIN_CNT_PAYMENT,
MIN(DAYS_FIRST_DRAWING) AS PA_MIN_DAYS_FIRST_DRAWING,
MIN(DAYS_FIRST_DUE) AS PA_MIN_DAYS_FIRST_DUE,
MIN(DAYS_LAST_DUE_1ST_VERSION) AS PA_MIN_DAYS_LAST_DUE_1ST_VERSION,
MIN(DAYS_LAST_DUE) AS PA_MIN_DAYS_LAST_DUE,
MIN(DAYS_TERMINATION) AS PA_MIN_DAYS_TERMINATION,
MIN(NFLAG_INSURED_ON_APPROVAL) AS PA_MIN_NFLAG_INSURED_ON_APPROVAL,
MAX(AMT_ANNUITY) AS PA_MAX_AMT_ANNUITY,
MAX(AMT_APPLICATION) AS PA_MAX_AMT_APPLICATION,
MAX(AMT_CREDIT) AS PA_MAX_AMT_CREDIT,
MAX(AMT_DOWN_PAYMENT) AS PA_MAX_AMT_DOWN_PAYMENT,
MAX(AMT_GOODS_PRICE) AS PA_MAX_AMT_GOODS_PRICE,
MAX(HOUR_APPR_PROCESS_START) AS PA_MAX_HOUR_APPR_PROCESS_START,
MAX(NFLAG_LAST_APPL_IN_DAY) AS PA_MAX_NFLAG_LAST_APPL_IN_DAY,
MAX(RATE_DOWN_PAYMENT) AS PA_MAX_RATE_DOWN_PAYMENT,
MAX(RATE_INTEREST_PRIMARY) AS PA_MAX_RATE_INTEREST_PRIMARY,
MAX(RATE_INTEREST_PRIVILEGED) AS PA_MAX_RATE_INTEREST_PRIVILEGED,
MAX(DAYS_DECISION) AS PA_MAX_DAYS_DECISION,
MAX(SELLERPLACE_AREA) AS PA_MAX_SELLERPLACE_AREA,
MAX(CNT_PAYMENT) AS PA_MAX_CNT_PAYMENT,
MAX(DAYS_FIRST_DRAWING) AS PA_MAX_DAYS_FIRST_DRAWING,
MAX(DAYS_FIRST_DUE) AS PA_MAX_DAYS_FIRST_DUE,
MAX(DAYS_LAST_DUE_1ST_VERSION) AS PA_MAX_DAYS_LAST_DUE_1ST_VERSION,
MAX(DAYS_LAST_DUE) AS PA_MAX_DAYS_LAST_DUE,
MAX(DAYS_TERMINATION) AS PA_MAX_DAYS_TERMINATION,
MAX(NFLAG_INSURED_ON_APPROVAL) AS PA_MAX_NFLAG_INSURED_ON_APPROVAL,
AVG(AMT_ANNUITY) AS PA_AVG_AMT_ANNUITY,
AVG(AMT_APPLICATION) AS PA_AVG_AMT_APPLICATION,
AVG(AMT_CREDIT) AS PA_AVG_AMT_CREDIT,
AVG(AMT_DOWN_PAYMENT) AS PA_AVG_AMT_DOWN_PAYMENT,
AVG(AMT_GOODS_PRICE) AS PA_AVG_AMT_GOODS_PRICE,
AVG(HOUR_APPR_PROCESS_START) AS PA_AVG_HOUR_APPR_PROCESS_START,
AVG(NFLAG_LAST_APPL_IN_DAY) AS PA_AVG_NFLAG_LAST_APPL_IN_DAY,
AVG(RATE_DOWN_PAYMENT) AS PA_AVG_RATE_DOWN_PAYMENT,
AVG(RATE_INTEREST_PRIMARY) AS PA_AVG_RATE_INTEREST_PRIMARY,
AVG(RATE_INTEREST_PRIVILEGED) AS PA_AVG_RATE_INTEREST_PRIVILEGED,
AVG(DAYS_DECISION) AS PA_AVG_DAYS_DECISION,
AVG(SELLERPLACE_AREA) AS PA_AVG_SELLERPLACE_AREA,
AVG(CNT_PAYMENT) AS PA_AVG_CNT_PAYMENT,
AVG(DAYS_FIRST_DRAWING) AS PA_AVG_DAYS_FIRST_DRAWING,
AVG(DAYS_FIRST_DUE) AS PA_AVG_DAYS_FIRST_DUE,
AVG(DAYS_LAST_DUE_1ST_VERSION) AS PA_AVG_DAYS_LAST_DUE_1ST_VERSION,
AVG(DAYS_LAST_DUE) AS PA_AVG_DAYS_LAST_DUE,
AVG(DAYS_TERMINATION) AS PA_AVG_DAYS_TERMINATION,
AVG(NFLAG_INSURED_ON_APPROVAL) AS PA_AVG_NFLAG_INSURED_ON_APPROVAL
FROM
df_previous_application
GROUP BY
  SK_ID_CURR
""")

In [120]:
df_previous_application_num.show()

+----------+------------------+----------------------+-----------------+-----------------------+----------------------+------------------------------+-----------------------------+------------------------+----------------------------+-------------------------------+--------------------+-----------------------+------------------+-------------------------+---------------------+--------------------------------+--------------------+-----------------------+--------------------------------+------------------+----------------------+-----------------+-----------------------+----------------------+------------------------------+-----------------------------+------------------------+----------------------------+-------------------------------+--------------------+-----------------------+------------------+-------------------------+---------------------+--------------------------------+--------------------+-----------------------+--------------------------------+------------------+---------------

In [121]:
df_previous_application_num.createOrReplaceTempView("df_previous_application_num")

In [122]:
#df_previous_application_num.write.csv('/content/gdrive/MyDrive/HD/CURSOS/Ciência - Análise de Dados/Pod Academy/Formação em Big Data e Analytics/Hackathon - Ciência de Dados/11 - Hackaton/dados_etl/df_previous_application_num.csv')

In [123]:
#df_previous_application_num.coalesce(1).write.csv('/content/gdrive/MyDrive/11 - Hackaton/dados_etl/df_previous_application_num.csv')

##Criando as variáveis numéricas e atrelando a diferentes status das variáveis categóricas

###1- Considerando a variável categórica NAME_CONTRACT_TYPE

In [124]:
# Criando as agregações

agg = ['SUM', 'AVG']

# Criando a variável categórica

cat = ['NAME_CONTRACT_TYPE']

# Criando os domínios

dominios_cat = ['XNA', 'Revolving loans', 'Consumer loans', 'Cash loans']

# Criando variável numérica

var_num = ['AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE','HOUR_APPR_PROCESS_START', 'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'DAYS_DECISION', 'SELLERPLACE_AREA', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL']

In [125]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN " + var_n + " ELSE 0 END) AS PA_" + ag + "_" + cate + "_" + dominio.replace(" ", "_").replace('/', '').replace('+', '').replace('(', '').replace(')', '').replace(':', '').replace('-', '').replace('__', '') + "_" + var_n + ',')

SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_XNA_AMT_ANNUITY,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_XNA_AMT_APPLICATION,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_XNA_AMT_CREDIT,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_XNA_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_XNA_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_XNA_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_XNA_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_XNA_RATE_DOWN_PAYMENT

In [126]:
df_previous_application_contract_NAME_CONTRACT_TYPE = spark.sql("""
SELECT
SK_ID_CURR,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_XNA_AMT_ANNUITY,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_XNA_AMT_APPLICATION,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_XNA_AMT_CREDIT,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_XNA_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_XNA_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_XNA_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_XNA_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_XNA_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_XNA_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_XNA_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_XNA_DAYS_DECISION,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_XNA_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_XNA_CNT_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_XNA_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_XNA_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_XNA_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_XNA_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_XNA_DAYS_TERMINATION,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_XNA_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Revolving_loans_AMT_ANNUITY,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Revolving_loans_AMT_APPLICATION,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Revolving_loans_AMT_CREDIT,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Revolving_loans_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Revolving_loans_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Revolving_loans_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Revolving_loans_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Revolving_loans_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Revolving_loans_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Revolving_loans_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Revolving_loans_DAYS_DECISION,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Revolving_loans_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Revolving_loans_CNT_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Revolving_loans_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Revolving_loans_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Revolving_loans_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Revolving_loans_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Revolving_loans_DAYS_TERMINATION,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Revolving_loans_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Consumer_loans_AMT_ANNUITY,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Consumer_loans_AMT_APPLICATION,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Consumer_loans_AMT_CREDIT,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Consumer_loans_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Consumer_loans_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Consumer_loans_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Consumer_loans_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Consumer_loans_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Consumer_loans_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Consumer_loans_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Consumer_loans_DAYS_DECISION,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Consumer_loans_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Consumer_loans_CNT_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Consumer_loans_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Consumer_loans_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Consumer_loans_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Consumer_loans_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Consumer_loans_DAYS_TERMINATION,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Consumer_loans_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Cash_loans_AMT_ANNUITY,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Cash_loans_AMT_APPLICATION,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Cash_loans_AMT_CREDIT,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Cash_loans_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Cash_loans_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Cash_loans_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Cash_loans_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Cash_loans_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Cash_loans_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Cash_loans_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Cash_loans_DAYS_DECISION,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Cash_loans_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Cash_loans_CNT_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Cash_loans_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Cash_loans_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Cash_loans_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Cash_loans_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Cash_loans_DAYS_TERMINATION,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_CONTRACT_TYPE_Cash_loans_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_XNA_AMT_ANNUITY,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_XNA_AMT_APPLICATION,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_XNA_AMT_CREDIT,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_XNA_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_XNA_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_XNA_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_XNA_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_XNA_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_XNA_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_XNA_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_XNA_DAYS_DECISION,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_XNA_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_XNA_CNT_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_XNA_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_XNA_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_XNA_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_XNA_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_XNA_DAYS_TERMINATION,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_XNA_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Revolving_loans_AMT_ANNUITY,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Revolving_loans_AMT_APPLICATION,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Revolving_loans_AMT_CREDIT,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Revolving_loans_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Revolving_loans_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Revolving_loans_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Revolving_loans_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Revolving_loans_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Revolving_loans_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Revolving_loans_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Revolving_loans_DAYS_DECISION,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Revolving_loans_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Revolving_loans_CNT_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Revolving_loans_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Revolving_loans_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Revolving_loans_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Revolving_loans_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Revolving_loans_DAYS_TERMINATION,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving loans' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Revolving_loans_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Consumer_loans_AMT_ANNUITY,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Consumer_loans_AMT_APPLICATION,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Consumer_loans_AMT_CREDIT,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Consumer_loans_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Consumer_loans_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Consumer_loans_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Consumer_loans_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Consumer_loans_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Consumer_loans_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Consumer_loans_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Consumer_loans_DAYS_DECISION,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Consumer_loans_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Consumer_loans_CNT_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Consumer_loans_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Consumer_loans_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Consumer_loans_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Consumer_loans_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Consumer_loans_DAYS_TERMINATION,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer loans' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Consumer_loans_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Cash_loans_AMT_ANNUITY,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Cash_loans_AMT_APPLICATION,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Cash_loans_AMT_CREDIT,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Cash_loans_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Cash_loans_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Cash_loans_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Cash_loans_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Cash_loans_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Cash_loans_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Cash_loans_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Cash_loans_DAYS_DECISION,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Cash_loans_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Cash_loans_CNT_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Cash_loans_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Cash_loans_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Cash_loans_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Cash_loans_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Cash_loans_DAYS_TERMINATION,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Cash loans' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_CONTRACT_TYPE_Cash_loans_NFLAG_INSURED_ON_APPROVAL
FROM
  df_previous_application
GROUP BY
  SK_ID_CURR
""")

###2- Considerando a variável categórica WEEKDAY_APPR_PROCESS_START

In [127]:
# Criando as agregações

agg = ['SUM', 'AVG']

# Criando a variável categórica

cat = ['WEEKDAY_APPR_PROCESS_START']

# Criando os domínios

dominios_cat = ['SUNDAY', 'SATURDAY', 'THURSDAY', 'FRIDAY','MONDAY','WEDNESDAY','TUESDAY']

# Criando variável numérica

var_num = ['AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE','HOUR_APPR_PROCESS_START', 'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'DAYS_DECISION', 'SELLERPLACE_AREA', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL']

In [128]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN " + var_n + " ELSE 0 END) AS PA_" + ag + "_" + cate + "_" + dominio.replace(" ", "_").replace('/', '').replace('+', '').replace('(', '').replace(')', '').replace(':', '').replace('-', '').replace('__', '') + "_" + var_n + ',')

SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_AMT_ANNUITY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_AMT_APPLICATION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_AMT_CREDIT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_AMT_DOWN_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_AMT_GOODS_PRICE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SU

In [129]:
df_previous_application_contract_WEEKDAY_APPR_PROCESS_START = spark.sql("""
SELECT
SK_ID_CURR,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_AMT_ANNUITY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_AMT_APPLICATION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_AMT_CREDIT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_AMT_DOWN_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_AMT_GOODS_PRICE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_RATE_DOWN_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_DAYS_DECISION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_SELLERPLACE_AREA,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_CNT_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_DAYS_FIRST_DRAWING,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_DAYS_FIRST_DUE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_DAYS_LAST_DUE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_DAYS_TERMINATION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SATURDAY_AMT_ANNUITY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SATURDAY_AMT_APPLICATION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SATURDAY_AMT_CREDIT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SATURDAY_AMT_DOWN_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SATURDAY_AMT_GOODS_PRICE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SATURDAY_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SATURDAY_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SATURDAY_RATE_DOWN_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SATURDAY_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SATURDAY_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SATURDAY_DAYS_DECISION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SATURDAY_SELLERPLACE_AREA,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SATURDAY_CNT_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SATURDAY_DAYS_FIRST_DRAWING,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SATURDAY_DAYS_FIRST_DUE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SATURDAY_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SATURDAY_DAYS_LAST_DUE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SATURDAY_DAYS_TERMINATION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_SATURDAY_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_THURSDAY_AMT_ANNUITY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_THURSDAY_AMT_APPLICATION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_THURSDAY_AMT_CREDIT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_THURSDAY_AMT_DOWN_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_THURSDAY_AMT_GOODS_PRICE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_THURSDAY_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_THURSDAY_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_THURSDAY_RATE_DOWN_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_THURSDAY_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_THURSDAY_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_THURSDAY_DAYS_DECISION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_THURSDAY_SELLERPLACE_AREA,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_THURSDAY_CNT_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_THURSDAY_DAYS_FIRST_DRAWING,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_THURSDAY_DAYS_FIRST_DUE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_THURSDAY_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_THURSDAY_DAYS_LAST_DUE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_THURSDAY_DAYS_TERMINATION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_THURSDAY_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_FRIDAY_AMT_ANNUITY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_FRIDAY_AMT_APPLICATION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_FRIDAY_AMT_CREDIT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_FRIDAY_AMT_DOWN_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_FRIDAY_AMT_GOODS_PRICE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_FRIDAY_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_FRIDAY_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_FRIDAY_RATE_DOWN_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_FRIDAY_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_FRIDAY_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_FRIDAY_DAYS_DECISION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_FRIDAY_SELLERPLACE_AREA,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_FRIDAY_CNT_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_FRIDAY_DAYS_FIRST_DRAWING,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_FRIDAY_DAYS_FIRST_DUE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_FRIDAY_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_FRIDAY_DAYS_LAST_DUE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_FRIDAY_DAYS_TERMINATION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_FRIDAY_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_MONDAY_AMT_ANNUITY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_MONDAY_AMT_APPLICATION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_MONDAY_AMT_CREDIT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_MONDAY_AMT_DOWN_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_MONDAY_AMT_GOODS_PRICE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_MONDAY_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_MONDAY_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_MONDAY_RATE_DOWN_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_MONDAY_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_MONDAY_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_MONDAY_DAYS_DECISION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_MONDAY_SELLERPLACE_AREA,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_MONDAY_CNT_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_MONDAY_DAYS_FIRST_DRAWING,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_MONDAY_DAYS_FIRST_DUE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_MONDAY_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_MONDAY_DAYS_LAST_DUE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_MONDAY_DAYS_TERMINATION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_MONDAY_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_AMT_ANNUITY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_AMT_APPLICATION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_AMT_CREDIT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_AMT_DOWN_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_AMT_GOODS_PRICE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_RATE_DOWN_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_DAYS_DECISION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_SELLERPLACE_AREA,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_CNT_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_DAYS_FIRST_DRAWING,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_DAYS_FIRST_DUE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_DAYS_LAST_DUE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_DAYS_TERMINATION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_TUESDAY_AMT_ANNUITY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_TUESDAY_AMT_APPLICATION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_TUESDAY_AMT_CREDIT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_TUESDAY_AMT_DOWN_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_TUESDAY_AMT_GOODS_PRICE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_TUESDAY_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_TUESDAY_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_TUESDAY_RATE_DOWN_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_TUESDAY_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_TUESDAY_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_TUESDAY_DAYS_DECISION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_TUESDAY_SELLERPLACE_AREA,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_TUESDAY_CNT_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_TUESDAY_DAYS_FIRST_DRAWING,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_TUESDAY_DAYS_FIRST_DUE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_TUESDAY_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_TUESDAY_DAYS_LAST_DUE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_TUESDAY_DAYS_TERMINATION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_WEEKDAY_APPR_PROCESS_START_TUESDAY_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_SUNDAY_AMT_ANNUITY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_SUNDAY_AMT_APPLICATION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_SUNDAY_AMT_CREDIT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_SUNDAY_AMT_DOWN_PAYMENT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_SUNDAY_AMT_GOODS_PRICE,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_SUNDAY_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_SUNDAY_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_SUNDAY_RATE_DOWN_PAYMENT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_SUNDAY_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_SUNDAY_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_SUNDAY_DAYS_DECISION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_SUNDAY_SELLERPLACE_AREA,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_SUNDAY_CNT_PAYMENT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_SUNDAY_DAYS_FIRST_DRAWING,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_SUNDAY_DAYS_FIRST_DUE,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_SUNDAY_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_SUNDAY_DAYS_LAST_DUE,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_SUNDAY_DAYS_TERMINATION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_SUNDAY_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_SATURDAY_AMT_ANNUITY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_SATURDAY_AMT_APPLICATION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_SATURDAY_AMT_CREDIT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_SATURDAY_AMT_DOWN_PAYMENT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_SATURDAY_AMT_GOODS_PRICE,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_SATURDAY_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_SATURDAY_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_SATURDAY_RATE_DOWN_PAYMENT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_SATURDAY_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_SATURDAY_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_SATURDAY_DAYS_DECISION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_SATURDAY_SELLERPLACE_AREA,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_SATURDAY_CNT_PAYMENT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_SATURDAY_DAYS_FIRST_DRAWING,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_SATURDAY_DAYS_FIRST_DUE,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_SATURDAY_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_SATURDAY_DAYS_LAST_DUE,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_SATURDAY_DAYS_TERMINATION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_SATURDAY_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_THURSDAY_AMT_ANNUITY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_THURSDAY_AMT_APPLICATION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_THURSDAY_AMT_CREDIT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_THURSDAY_AMT_DOWN_PAYMENT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_THURSDAY_AMT_GOODS_PRICE,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_THURSDAY_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_THURSDAY_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_THURSDAY_RATE_DOWN_PAYMENT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_THURSDAY_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_THURSDAY_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_THURSDAY_DAYS_DECISION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_THURSDAY_SELLERPLACE_AREA,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_THURSDAY_CNT_PAYMENT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_THURSDAY_DAYS_FIRST_DRAWING,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_THURSDAY_DAYS_FIRST_DUE,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_THURSDAY_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_THURSDAY_DAYS_LAST_DUE,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_THURSDAY_DAYS_TERMINATION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_THURSDAY_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_FRIDAY_AMT_ANNUITY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_FRIDAY_AMT_APPLICATION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_FRIDAY_AMT_CREDIT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_FRIDAY_AMT_DOWN_PAYMENT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_FRIDAY_AMT_GOODS_PRICE,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_FRIDAY_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_FRIDAY_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_FRIDAY_RATE_DOWN_PAYMENT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_FRIDAY_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_FRIDAY_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_FRIDAY_DAYS_DECISION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_FRIDAY_SELLERPLACE_AREA,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_FRIDAY_CNT_PAYMENT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_FRIDAY_DAYS_FIRST_DRAWING,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_FRIDAY_DAYS_FIRST_DUE,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_FRIDAY_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_FRIDAY_DAYS_LAST_DUE,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_FRIDAY_DAYS_TERMINATION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_FRIDAY_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_MONDAY_AMT_ANNUITY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_MONDAY_AMT_APPLICATION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_MONDAY_AMT_CREDIT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_MONDAY_AMT_DOWN_PAYMENT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_MONDAY_AMT_GOODS_PRICE,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_MONDAY_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_MONDAY_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_MONDAY_RATE_DOWN_PAYMENT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_MONDAY_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_MONDAY_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_MONDAY_DAYS_DECISION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_MONDAY_SELLERPLACE_AREA,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_MONDAY_CNT_PAYMENT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_MONDAY_DAYS_FIRST_DRAWING,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_MONDAY_DAYS_FIRST_DUE,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_MONDAY_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_MONDAY_DAYS_LAST_DUE,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_MONDAY_DAYS_TERMINATION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_MONDAY_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_AMT_ANNUITY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_AMT_APPLICATION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_AMT_CREDIT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_AMT_DOWN_PAYMENT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_AMT_GOODS_PRICE,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_RATE_DOWN_PAYMENT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_DAYS_DECISION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_SELLERPLACE_AREA,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_CNT_PAYMENT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_DAYS_FIRST_DRAWING,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_DAYS_FIRST_DUE,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_DAYS_LAST_DUE,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_DAYS_TERMINATION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_TUESDAY_AMT_ANNUITY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_TUESDAY_AMT_APPLICATION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_TUESDAY_AMT_CREDIT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_TUESDAY_AMT_DOWN_PAYMENT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_TUESDAY_AMT_GOODS_PRICE,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_TUESDAY_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_TUESDAY_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_TUESDAY_RATE_DOWN_PAYMENT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_TUESDAY_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_TUESDAY_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_TUESDAY_DAYS_DECISION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_TUESDAY_SELLERPLACE_AREA,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_TUESDAY_CNT_PAYMENT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_TUESDAY_DAYS_FIRST_DRAWING,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_TUESDAY_DAYS_FIRST_DUE,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_TUESDAY_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_TUESDAY_DAYS_LAST_DUE,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_TUESDAY_DAYS_TERMINATION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_WEEKDAY_APPR_PROCESS_START_TUESDAY_NFLAG_INSURED_ON_APPROVAL
FROM
  df_previous_application
GROUP BY
  SK_ID_CURR
""")

###3 - Considerando a variável categórica FLAG_LAST_APPL_PER_CONTRACT

In [130]:
# Criando as agregações

agg = ['SUM', 'AVG']

# Criando a variável categórica

cat = ['FLAG_LAST_APPL_PER_CONTRACT']

# Criando os domínios

dominios_cat = ['N', 'Y']

# Criando variável numérica

var_num = ['AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE','HOUR_APPR_PROCESS_START', 'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'DAYS_DECISION', 'SELLERPLACE_AREA', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL']

In [131]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN " + var_n + " ELSE 0 END) AS PA_" + ag + "_" + cate + "_" + dominio.replace(" ", "_").replace('/', '').replace('+', '').replace('(', '').replace(')', '').replace(':', '').replace('-', '').replace('__', '') + "_" + var_n + ',')

SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_N_AMT_ANNUITY,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_N_AMT_APPLICATION,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_N_AMT_CREDIT,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_N_AMT_DOWN_PAYMENT,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_N_AMT_GOODS_PRICE,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_N_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_N_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN FLAG_LAST_APP

In [132]:
df_previous_application_contract_FLAG_LAST_APPL_PER_CONTRACT = spark.sql("""
SELECT
SK_ID_CURR,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_N_AMT_ANNUITY,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_N_AMT_APPLICATION,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_N_AMT_CREDIT,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_N_AMT_DOWN_PAYMENT,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_N_AMT_GOODS_PRICE,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_N_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_N_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_N_RATE_DOWN_PAYMENT,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_N_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_N_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_N_DAYS_DECISION,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_N_SELLERPLACE_AREA,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_N_CNT_PAYMENT,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_N_DAYS_FIRST_DRAWING,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_N_DAYS_FIRST_DUE,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_N_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_N_DAYS_LAST_DUE,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_N_DAYS_TERMINATION,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_N_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_Y_AMT_ANNUITY,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_Y_AMT_APPLICATION,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_Y_AMT_CREDIT,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_Y_AMT_DOWN_PAYMENT,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_Y_AMT_GOODS_PRICE,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_Y_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_Y_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_Y_RATE_DOWN_PAYMENT,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_Y_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_Y_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_Y_DAYS_DECISION,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_Y_SELLERPLACE_AREA,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_Y_CNT_PAYMENT,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_Y_DAYS_FIRST_DRAWING,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_Y_DAYS_FIRST_DUE,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_Y_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_Y_DAYS_LAST_DUE,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_Y_DAYS_TERMINATION,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_FLAG_LAST_APPL_PER_CONTRACT_Y_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_FLAG_LAST_APPL_PER_CONTRACT_N_AMT_ANNUITY,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_FLAG_LAST_APPL_PER_CONTRACT_N_AMT_APPLICATION,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_FLAG_LAST_APPL_PER_CONTRACT_N_AMT_CREDIT,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_FLAG_LAST_APPL_PER_CONTRACT_N_AMT_DOWN_PAYMENT,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_FLAG_LAST_APPL_PER_CONTRACT_N_AMT_GOODS_PRICE,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_FLAG_LAST_APPL_PER_CONTRACT_N_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_FLAG_LAST_APPL_PER_CONTRACT_N_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_FLAG_LAST_APPL_PER_CONTRACT_N_RATE_DOWN_PAYMENT,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_FLAG_LAST_APPL_PER_CONTRACT_N_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_FLAG_LAST_APPL_PER_CONTRACT_N_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_FLAG_LAST_APPL_PER_CONTRACT_N_DAYS_DECISION,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_FLAG_LAST_APPL_PER_CONTRACT_N_SELLERPLACE_AREA,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_FLAG_LAST_APPL_PER_CONTRACT_N_CNT_PAYMENT,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_FLAG_LAST_APPL_PER_CONTRACT_N_DAYS_FIRST_DRAWING,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_FLAG_LAST_APPL_PER_CONTRACT_N_DAYS_FIRST_DUE,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_FLAG_LAST_APPL_PER_CONTRACT_N_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_FLAG_LAST_APPL_PER_CONTRACT_N_DAYS_LAST_DUE,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_FLAG_LAST_APPL_PER_CONTRACT_N_DAYS_TERMINATION,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_FLAG_LAST_APPL_PER_CONTRACT_N_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_FLAG_LAST_APPL_PER_CONTRACT_Y_AMT_ANNUITY,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_FLAG_LAST_APPL_PER_CONTRACT_Y_AMT_APPLICATION,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_FLAG_LAST_APPL_PER_CONTRACT_Y_AMT_CREDIT,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_FLAG_LAST_APPL_PER_CONTRACT_Y_AMT_DOWN_PAYMENT,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_FLAG_LAST_APPL_PER_CONTRACT_Y_AMT_GOODS_PRICE,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_FLAG_LAST_APPL_PER_CONTRACT_Y_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_FLAG_LAST_APPL_PER_CONTRACT_Y_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_FLAG_LAST_APPL_PER_CONTRACT_Y_RATE_DOWN_PAYMENT,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_FLAG_LAST_APPL_PER_CONTRACT_Y_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_FLAG_LAST_APPL_PER_CONTRACT_Y_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_FLAG_LAST_APPL_PER_CONTRACT_Y_DAYS_DECISION,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_FLAG_LAST_APPL_PER_CONTRACT_Y_SELLERPLACE_AREA,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_FLAG_LAST_APPL_PER_CONTRACT_Y_CNT_PAYMENT,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_FLAG_LAST_APPL_PER_CONTRACT_Y_DAYS_FIRST_DRAWING,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_FLAG_LAST_APPL_PER_CONTRACT_Y_DAYS_FIRST_DUE,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_FLAG_LAST_APPL_PER_CONTRACT_Y_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_FLAG_LAST_APPL_PER_CONTRACT_Y_DAYS_LAST_DUE,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_FLAG_LAST_APPL_PER_CONTRACT_Y_DAYS_TERMINATION,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_FLAG_LAST_APPL_PER_CONTRACT_Y_NFLAG_INSURED_ON_APPROVAL
FROM
  df_previous_application
GROUP BY
  SK_ID_CURR
""")

###4 - Considerando a variável categórica NAME_CASH_LOAN_PURPOSE

In [133]:
# Criando as agregações

agg = ['SUM', 'AVG']

# Criando a variável categórica

cat = ['NAME_CASH_LOAN_PURPOSE']

# Criando os domínios

dominios_cat = ['Refusal to name the goal', 'Money for a third person', 'Hobby', 'Buying a garage', 'Gasification / water supply', 'Business development', 'Buying a holiday home / land', 'Furniture', 'Car repairs', 'Buying a home', 'Wedding / gift / holiday', 'Buying a new car', 'Purchase of electronic equipment', 'Journey', 'Education', 'Payments on other loans', 'Medicine', 'Everyday expenses', 'Building a house or an annex', 'Buying a used car']

# Criando variável numérica

var_num = ['AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE','HOUR_APPR_PROCESS_START', 'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'DAYS_DECISION', 'SELLERPLACE_AREA', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL']

In [134]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN " + var_n + " ELSE 0 END) AS PA_" + ag + "_" + cate + "_" + dominio.replace(" ", "_").replace('/', '').replace('+', '').replace('(', '').replace(')', '').replace(':', '').replace('-', '').replace('__', '') + "_" + var_n + ',')

SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_AMT_ANNUITY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_AMT_APPLICATION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_AMT_CREDIT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Refusal_to_n

In [135]:
df_previous_application_contract_NAME_CASH_LOAN_PURPOSE = spark.sql("""
SELECT
SK_ID_CURR,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_AMT_ANNUITY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_AMT_APPLICATION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_AMT_CREDIT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_DAYS_DECISION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_CNT_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_DAYS_TERMINATION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person_AMT_ANNUITY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person_AMT_APPLICATION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person_AMT_CREDIT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person_DAYS_DECISION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person_CNT_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person_DAYS_TERMINATION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Hobby_AMT_ANNUITY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Hobby_AMT_APPLICATION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Hobby_AMT_CREDIT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Hobby_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Hobby_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Hobby_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Hobby_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Hobby_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Hobby_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Hobby_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Hobby_DAYS_DECISION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Hobby_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Hobby_CNT_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Hobby_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Hobby_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Hobby_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Hobby_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Hobby_DAYS_TERMINATION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Hobby_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_garage_AMT_ANNUITY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_garage_AMT_APPLICATION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_garage_AMT_CREDIT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_garage_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_garage_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_garage_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_garage_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_garage_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_garage_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_garage_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_garage_DAYS_DECISION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_garage_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_garage_CNT_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_garage_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_garage_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_garage_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_garage_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_garage_DAYS_TERMINATION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_garage_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply_AMT_ANNUITY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply_AMT_APPLICATION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply_AMT_CREDIT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply_DAYS_DECISION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply_CNT_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply_DAYS_TERMINATION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Business_development_AMT_ANNUITY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Business_development_AMT_APPLICATION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Business_development_AMT_CREDIT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Business_development_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Business_development_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Business_development_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Business_development_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Business_development_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Business_development_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Business_development_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Business_development_DAYS_DECISION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Business_development_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Business_development_CNT_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Business_development_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Business_development_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Business_development_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Business_development_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Business_development_DAYS_TERMINATION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Business_development_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland_AMT_ANNUITY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland_AMT_APPLICATION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland_AMT_CREDIT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland_DAYS_DECISION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland_CNT_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland_DAYS_TERMINATION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Furniture_AMT_ANNUITY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Furniture_AMT_APPLICATION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Furniture_AMT_CREDIT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Furniture_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Furniture_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Furniture_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Furniture_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Furniture_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Furniture_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Furniture_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Furniture_DAYS_DECISION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Furniture_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Furniture_CNT_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Furniture_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Furniture_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Furniture_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Furniture_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Furniture_DAYS_TERMINATION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Furniture_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car repairs' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Car_repairs_AMT_ANNUITY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car repairs' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Car_repairs_AMT_APPLICATION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car repairs' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Car_repairs_AMT_CREDIT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car repairs' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Car_repairs_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car repairs' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Car_repairs_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car repairs' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Car_repairs_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car repairs' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Car_repairs_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car repairs' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Car_repairs_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car repairs' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Car_repairs_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car repairs' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Car_repairs_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car repairs' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Car_repairs_DAYS_DECISION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car repairs' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Car_repairs_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car repairs' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Car_repairs_CNT_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car repairs' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Car_repairs_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car repairs' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Car_repairs_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car repairs' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Car_repairs_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car repairs' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Car_repairs_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car repairs' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Car_repairs_DAYS_TERMINATION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car repairs' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Car_repairs_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a home' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_home_AMT_ANNUITY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a home' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_home_AMT_APPLICATION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a home' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_home_AMT_CREDIT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a home' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_home_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a home' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_home_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a home' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_home_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a home' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_home_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a home' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_home_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a home' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_home_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a home' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_home_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a home' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_home_DAYS_DECISION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a home' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_home_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a home' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_home_CNT_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a home' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_home_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a home' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_home_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a home' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_home_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a home' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_home_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a home' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_home_DAYS_TERMINATION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a home' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_home_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding / gift / holiday' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Weddinggiftholiday_AMT_ANNUITY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding / gift / holiday' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Weddinggiftholiday_AMT_APPLICATION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding / gift / holiday' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Weddinggiftholiday_AMT_CREDIT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding / gift / holiday' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Weddinggiftholiday_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding / gift / holiday' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Weddinggiftholiday_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding / gift / holiday' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Weddinggiftholiday_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding / gift / holiday' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Weddinggiftholiday_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding / gift / holiday' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Weddinggiftholiday_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding / gift / holiday' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Weddinggiftholiday_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding / gift / holiday' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Weddinggiftholiday_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding / gift / holiday' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Weddinggiftholiday_DAYS_DECISION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding / gift / holiday' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Weddinggiftholiday_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding / gift / holiday' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Weddinggiftholiday_CNT_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding / gift / holiday' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Weddinggiftholiday_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding / gift / holiday' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Weddinggiftholiday_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding / gift / holiday' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Weddinggiftholiday_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding / gift / holiday' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Weddinggiftholiday_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding / gift / holiday' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Weddinggiftholiday_DAYS_TERMINATION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding / gift / holiday' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Weddinggiftholiday_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a new car' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_AMT_ANNUITY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a new car' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_AMT_APPLICATION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a new car' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_AMT_CREDIT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a new car' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a new car' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a new car' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a new car' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a new car' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a new car' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a new car' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a new car' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_DAYS_DECISION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a new car' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a new car' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_CNT_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a new car' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a new car' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a new car' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a new car' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a new car' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_DAYS_TERMINATION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a new car' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase of electronic equipment' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment_AMT_ANNUITY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase of electronic equipment' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment_AMT_APPLICATION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase of electronic equipment' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment_AMT_CREDIT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase of electronic equipment' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase of electronic equipment' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase of electronic equipment' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase of electronic equipment' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase of electronic equipment' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase of electronic equipment' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase of electronic equipment' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase of electronic equipment' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment_DAYS_DECISION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase of electronic equipment' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase of electronic equipment' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment_CNT_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase of electronic equipment' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase of electronic equipment' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase of electronic equipment' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase of electronic equipment' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase of electronic equipment' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment_DAYS_TERMINATION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase of electronic equipment' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Journey_AMT_ANNUITY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Journey_AMT_APPLICATION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Journey_AMT_CREDIT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Journey_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Journey_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Journey_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Journey_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Journey_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Journey_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Journey_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Journey_DAYS_DECISION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Journey_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Journey_CNT_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Journey_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Journey_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Journey_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Journey_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Journey_DAYS_TERMINATION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Journey_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Education_AMT_ANNUITY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Education_AMT_APPLICATION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Education_AMT_CREDIT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Education_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Education_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Education_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Education_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Education_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Education_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Education_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Education_DAYS_DECISION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Education_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Education_CNT_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Education_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Education_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Education_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Education_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Education_DAYS_TERMINATION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Education_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments on other loans' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans_AMT_ANNUITY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments on other loans' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans_AMT_APPLICATION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments on other loans' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans_AMT_CREDIT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments on other loans' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments on other loans' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments on other loans' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments on other loans' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments on other loans' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments on other loans' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments on other loans' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments on other loans' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans_DAYS_DECISION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments on other loans' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments on other loans' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans_CNT_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments on other loans' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments on other loans' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments on other loans' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments on other loans' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments on other loans' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans_DAYS_TERMINATION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments on other loans' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Medicine_AMT_ANNUITY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Medicine_AMT_APPLICATION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Medicine_AMT_CREDIT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Medicine_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Medicine_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Medicine_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Medicine_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Medicine_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Medicine_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Medicine_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Medicine_DAYS_DECISION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Medicine_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Medicine_CNT_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Medicine_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Medicine_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Medicine_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Medicine_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Medicine_DAYS_TERMINATION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Medicine_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday expenses' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Everyday_expenses_AMT_ANNUITY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday expenses' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Everyday_expenses_AMT_APPLICATION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday expenses' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Everyday_expenses_AMT_CREDIT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday expenses' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Everyday_expenses_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday expenses' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Everyday_expenses_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday expenses' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Everyday_expenses_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday expenses' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Everyday_expenses_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday expenses' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Everyday_expenses_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday expenses' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Everyday_expenses_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday expenses' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Everyday_expenses_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday expenses' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Everyday_expenses_DAYS_DECISION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday expenses' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Everyday_expenses_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday expenses' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Everyday_expenses_CNT_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday expenses' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Everyday_expenses_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday expenses' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Everyday_expenses_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday expenses' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Everyday_expenses_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday expenses' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Everyday_expenses_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday expenses' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Everyday_expenses_DAYS_TERMINATION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday expenses' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Everyday_expenses_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building a house or an annex' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex_AMT_ANNUITY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building a house or an annex' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex_AMT_APPLICATION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building a house or an annex' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex_AMT_CREDIT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building a house or an annex' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building a house or an annex' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building a house or an annex' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building a house or an annex' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building a house or an annex' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building a house or an annex' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building a house or an annex' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building a house or an annex' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex_DAYS_DECISION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building a house or an annex' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building a house or an annex' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex_CNT_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building a house or an annex' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building a house or an annex' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building a house or an annex' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building a house or an annex' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building a house or an annex' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex_DAYS_TERMINATION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building a house or an annex' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a used car' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car_AMT_ANNUITY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a used car' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car_AMT_APPLICATION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a used car' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car_AMT_CREDIT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a used car' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a used car' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a used car' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a used car' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a used car' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a used car' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a used car' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a used car' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car_DAYS_DECISION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a used car' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a used car' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car_CNT_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a used car' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a used car' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a used car' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a used car' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a used car' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car_DAYS_TERMINATION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a used car' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_AMT_ANNUITY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_AMT_APPLICATION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_AMT_CREDIT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_DAYS_DECISION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_CNT_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_DAYS_TERMINATION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal to name the goal' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person_AMT_ANNUITY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person_AMT_APPLICATION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person_AMT_CREDIT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person_DAYS_DECISION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person_CNT_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person_DAYS_TERMINATION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money for a third person' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Hobby_AMT_ANNUITY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Hobby_AMT_APPLICATION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Hobby_AMT_CREDIT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Hobby_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Hobby_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Hobby_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Hobby_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Hobby_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Hobby_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Hobby_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Hobby_DAYS_DECISION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Hobby_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Hobby_CNT_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Hobby_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Hobby_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Hobby_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Hobby_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Hobby_DAYS_TERMINATION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Hobby_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_garage_AMT_ANNUITY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_garage_AMT_APPLICATION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_garage_AMT_CREDIT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_garage_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_garage_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_garage_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_garage_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_garage_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_garage_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_garage_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_garage_DAYS_DECISION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_garage_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_garage_CNT_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_garage_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_garage_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_garage_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_garage_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_garage_DAYS_TERMINATION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a garage' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_garage_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply_AMT_ANNUITY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply_AMT_APPLICATION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply_AMT_CREDIT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply_DAYS_DECISION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply_CNT_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply_DAYS_TERMINATION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification / water supply' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Gasificationwater_supply_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Business_development_AMT_ANNUITY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Business_development_AMT_APPLICATION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Business_development_AMT_CREDIT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Business_development_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Business_development_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Business_development_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Business_development_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Business_development_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Business_development_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Business_development_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Business_development_DAYS_DECISION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Business_development_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Business_development_CNT_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Business_development_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Business_development_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Business_development_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Business_development_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Business_development_DAYS_TERMINATION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business development' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Business_development_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland_AMT_ANNUITY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland_AMT_APPLICATION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland_AMT_CREDIT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland_DAYS_DECISION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland_CNT_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland_DAYS_TERMINATION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a holiday home / land' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_homeland_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Furniture_AMT_ANNUITY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Furniture_AMT_APPLICATION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Furniture_AMT_CREDIT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Furniture_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Furniture_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Furniture_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Furniture_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Furniture_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Furniture_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Furniture_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Furniture_DAYS_DECISION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Furniture_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Furniture_CNT_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Furniture_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Furniture_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Furniture_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Furniture_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Furniture_DAYS_TERMINATION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Furniture_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car repairs' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Car_repairs_AMT_ANNUITY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car repairs' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Car_repairs_AMT_APPLICATION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car repairs' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Car_repairs_AMT_CREDIT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car repairs' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Car_repairs_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car repairs' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Car_repairs_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car repairs' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Car_repairs_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car repairs' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Car_repairs_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car repairs' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Car_repairs_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car repairs' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Car_repairs_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car repairs' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Car_repairs_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car repairs' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Car_repairs_DAYS_DECISION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car repairs' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Car_repairs_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car repairs' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Car_repairs_CNT_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car repairs' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Car_repairs_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car repairs' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Car_repairs_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car repairs' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Car_repairs_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car repairs' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Car_repairs_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car repairs' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Car_repairs_DAYS_TERMINATION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car repairs' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Car_repairs_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a home' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_home_AMT_ANNUITY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a home' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_home_AMT_APPLICATION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a home' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_home_AMT_CREDIT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a home' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_home_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a home' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_home_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a home' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_home_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a home' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_home_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a home' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_home_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a home' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_home_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a home' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_home_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a home' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_home_DAYS_DECISION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a home' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_home_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a home' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_home_CNT_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a home' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_home_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a home' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_home_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a home' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_home_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a home' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_home_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a home' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_home_DAYS_TERMINATION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a home' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_home_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding / gift / holiday' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Weddinggiftholiday_AMT_ANNUITY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding / gift / holiday' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Weddinggiftholiday_AMT_APPLICATION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding / gift / holiday' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Weddinggiftholiday_AMT_CREDIT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding / gift / holiday' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Weddinggiftholiday_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding / gift / holiday' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Weddinggiftholiday_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding / gift / holiday' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Weddinggiftholiday_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding / gift / holiday' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Weddinggiftholiday_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding / gift / holiday' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Weddinggiftholiday_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding / gift / holiday' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Weddinggiftholiday_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding / gift / holiday' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Weddinggiftholiday_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding / gift / holiday' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Weddinggiftholiday_DAYS_DECISION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding / gift / holiday' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Weddinggiftholiday_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding / gift / holiday' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Weddinggiftholiday_CNT_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding / gift / holiday' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Weddinggiftholiday_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding / gift / holiday' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Weddinggiftholiday_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding / gift / holiday' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Weddinggiftholiday_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding / gift / holiday' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Weddinggiftholiday_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding / gift / holiday' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Weddinggiftholiday_DAYS_TERMINATION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding / gift / holiday' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Weddinggiftholiday_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a new car' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_AMT_ANNUITY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a new car' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_AMT_APPLICATION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a new car' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_AMT_CREDIT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a new car' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a new car' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a new car' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a new car' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a new car' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a new car' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a new car' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a new car' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_DAYS_DECISION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a new car' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a new car' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_CNT_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a new car' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a new car' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a new car' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a new car' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a new car' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_DAYS_TERMINATION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a new car' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase of electronic equipment' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment_AMT_ANNUITY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase of electronic equipment' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment_AMT_APPLICATION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase of electronic equipment' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment_AMT_CREDIT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase of electronic equipment' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase of electronic equipment' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase of electronic equipment' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase of electronic equipment' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase of electronic equipment' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase of electronic equipment' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase of electronic equipment' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase of electronic equipment' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment_DAYS_DECISION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase of electronic equipment' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase of electronic equipment' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment_CNT_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase of electronic equipment' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase of electronic equipment' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase of electronic equipment' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase of electronic equipment' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase of electronic equipment' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment_DAYS_TERMINATION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase of electronic equipment' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Journey_AMT_ANNUITY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Journey_AMT_APPLICATION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Journey_AMT_CREDIT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Journey_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Journey_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Journey_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Journey_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Journey_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Journey_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Journey_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Journey_DAYS_DECISION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Journey_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Journey_CNT_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Journey_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Journey_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Journey_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Journey_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Journey_DAYS_TERMINATION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Journey_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Education_AMT_ANNUITY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Education_AMT_APPLICATION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Education_AMT_CREDIT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Education_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Education_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Education_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Education_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Education_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Education_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Education_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Education_DAYS_DECISION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Education_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Education_CNT_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Education_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Education_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Education_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Education_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Education_DAYS_TERMINATION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Education_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments on other loans' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans_AMT_ANNUITY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments on other loans' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans_AMT_APPLICATION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments on other loans' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans_AMT_CREDIT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments on other loans' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments on other loans' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments on other loans' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments on other loans' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments on other loans' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments on other loans' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments on other loans' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments on other loans' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans_DAYS_DECISION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments on other loans' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments on other loans' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans_CNT_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments on other loans' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments on other loans' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments on other loans' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments on other loans' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments on other loans' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans_DAYS_TERMINATION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments on other loans' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Medicine_AMT_ANNUITY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Medicine_AMT_APPLICATION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Medicine_AMT_CREDIT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Medicine_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Medicine_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Medicine_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Medicine_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Medicine_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Medicine_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Medicine_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Medicine_DAYS_DECISION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Medicine_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Medicine_CNT_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Medicine_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Medicine_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Medicine_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Medicine_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Medicine_DAYS_TERMINATION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Medicine_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday expenses' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Everyday_expenses_AMT_ANNUITY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday expenses' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Everyday_expenses_AMT_APPLICATION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday expenses' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Everyday_expenses_AMT_CREDIT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday expenses' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Everyday_expenses_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday expenses' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Everyday_expenses_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday expenses' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Everyday_expenses_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday expenses' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Everyday_expenses_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday expenses' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Everyday_expenses_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday expenses' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Everyday_expenses_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday expenses' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Everyday_expenses_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday expenses' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Everyday_expenses_DAYS_DECISION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday expenses' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Everyday_expenses_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday expenses' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Everyday_expenses_CNT_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday expenses' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Everyday_expenses_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday expenses' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Everyday_expenses_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday expenses' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Everyday_expenses_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday expenses' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Everyday_expenses_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday expenses' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Everyday_expenses_DAYS_TERMINATION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday expenses' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Everyday_expenses_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building a house or an annex' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex_AMT_ANNUITY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building a house or an annex' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex_AMT_APPLICATION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building a house or an annex' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex_AMT_CREDIT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building a house or an annex' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building a house or an annex' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building a house or an annex' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building a house or an annex' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building a house or an annex' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building a house or an annex' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building a house or an annex' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building a house or an annex' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex_DAYS_DECISION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building a house or an annex' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building a house or an annex' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex_CNT_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building a house or an annex' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building a house or an annex' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building a house or an annex' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building a house or an annex' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building a house or an annex' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex_DAYS_TERMINATION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building a house or an annex' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a used car' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car_AMT_ANNUITY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a used car' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car_AMT_APPLICATION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a used car' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car_AMT_CREDIT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a used car' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a used car' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a used car' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a used car' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a used car' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a used car' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a used car' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a used car' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car_DAYS_DECISION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a used car' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a used car' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car_CNT_PAYMENT,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a used car' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a used car' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a used car' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a used car' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a used car' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car_DAYS_TERMINATION,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying a used car' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car_NFLAG_INSURED_ON_APPROVAL
FROM
  df_previous_application
GROUP BY
  SK_ID_CURR
""")

###5 - Considerando a variável categórica NAME_CONTRACT_STATUS

In [136]:
# Criando as agregações

agg = ['SUM', 'AVG']

# Criando a variável categórica

cat = ['NAME_CONTRACT_STATUS']

# Criando os domínios

dominios_cat = ['Unused offer', 'Refused', 'Canceled', 'Approved']

# Criando variável numérica

var_num = ['AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE','HOUR_APPR_PROCESS_START', 'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'DAYS_DECISION', 'SELLERPLACE_AREA', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL']

In [137]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN " + var_n + " ELSE 0 END) AS PA_" + ag + "_" + cate + "_" + dominio.replace(" ", "_").replace('/', '').replace('+', '').replace('(', '').replace(')', '').replace(':', '').replace('-', '').replace('__', '') + "_" + var_n + ',')

SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Unused_offer_AMT_ANNUITY,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Unused_offer_AMT_APPLICATION,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Unused_offer_AMT_CREDIT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Unused_offer_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Unused_offer_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Unused_offer_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Unused_o

In [138]:
df_previous_application_contract_NAME_CONTRACT_STATUS = spark.sql("""
SELECT
SK_ID_CURR,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Unused_offer_AMT_ANNUITY,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Unused_offer_AMT_APPLICATION,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Unused_offer_AMT_CREDIT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Unused_offer_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Unused_offer_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Unused_offer_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Unused_offer_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Unused_offer_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Unused_offer_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Unused_offer_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Unused_offer_DAYS_DECISION,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Unused_offer_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Unused_offer_CNT_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Unused_offer_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Unused_offer_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Unused_offer_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Unused_offer_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Unused_offer_DAYS_TERMINATION,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Unused_offer_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Refused_AMT_ANNUITY,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Refused_AMT_APPLICATION,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Refused_AMT_CREDIT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Refused_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Refused_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Refused_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Refused_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Refused_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Refused_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Refused_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Refused_DAYS_DECISION,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Refused_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Refused_CNT_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Refused_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Refused_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Refused_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Refused_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Refused_DAYS_TERMINATION,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Refused_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Canceled_AMT_ANNUITY,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Canceled_AMT_APPLICATION,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Canceled_AMT_CREDIT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Canceled_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Canceled_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Canceled_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Canceled_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Canceled_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Canceled_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Canceled_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Canceled_DAYS_DECISION,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Canceled_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Canceled_CNT_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Canceled_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Canceled_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Canceled_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Canceled_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Canceled_DAYS_TERMINATION,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Canceled_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Approved_AMT_ANNUITY,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Approved_AMT_APPLICATION,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Approved_AMT_CREDIT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Approved_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Approved_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Approved_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Approved_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Approved_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Approved_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Approved_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Approved_DAYS_DECISION,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Approved_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Approved_CNT_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Approved_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Approved_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Approved_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Approved_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Approved_DAYS_TERMINATION,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_CONTRACT_STATUS_Approved_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Unused_offer_AMT_ANNUITY,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Unused_offer_AMT_APPLICATION,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Unused_offer_AMT_CREDIT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Unused_offer_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Unused_offer_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Unused_offer_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Unused_offer_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Unused_offer_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Unused_offer_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Unused_offer_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Unused_offer_DAYS_DECISION,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Unused_offer_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Unused_offer_CNT_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Unused_offer_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Unused_offer_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Unused_offer_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Unused_offer_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Unused_offer_DAYS_TERMINATION,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Unused offer' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Unused_offer_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Refused_AMT_ANNUITY,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Refused_AMT_APPLICATION,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Refused_AMT_CREDIT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Refused_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Refused_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Refused_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Refused_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Refused_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Refused_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Refused_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Refused_DAYS_DECISION,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Refused_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Refused_CNT_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Refused_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Refused_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Refused_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Refused_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Refused_DAYS_TERMINATION,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Refused_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Canceled_AMT_ANNUITY,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Canceled_AMT_APPLICATION,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Canceled_AMT_CREDIT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Canceled_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Canceled_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Canceled_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Canceled_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Canceled_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Canceled_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Canceled_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Canceled_DAYS_DECISION,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Canceled_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Canceled_CNT_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Canceled_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Canceled_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Canceled_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Canceled_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Canceled_DAYS_TERMINATION,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Canceled_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Approved_AMT_ANNUITY,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Approved_AMT_APPLICATION,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Approved_AMT_CREDIT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Approved_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Approved_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Approved_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Approved_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Approved_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Approved_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Approved_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Approved_DAYS_DECISION,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Approved_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Approved_CNT_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Approved_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Approved_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Approved_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Approved_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Approved_DAYS_TERMINATION,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_CONTRACT_STATUS_Approved_NFLAG_INSURED_ON_APPROVAL
FROM
  df_previous_application
GROUP BY
  SK_ID_CURR
""")

###6 - Considerando a variável categórica NAME_PAYMENT_TYPE

In [139]:
# Criando as agregações

agg = ['SUM', 'AVG']

# Criando a variável categórica

cat = ['NAME_PAYMENT_TYPE']

# Criando os domínios

dominios_cat = ['Cashless from the account of the employer', 'Non cash from your account', 'XNA', 'Cash through the bank']

# Criando variável numérica

var_num = ['AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE','HOUR_APPR_PROCESS_START', 'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'DAYS_DECISION', 'SELLERPLACE_AREA', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL']

In [140]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN " + var_n + " ELSE 0 END) AS PA_" + ag + "_" + cate + "_" + dominio.replace(" ", "_").replace('/', '').replace('+', '').replace('(', '').replace(')', '').replace(':', '').replace('-', '').replace('__', '') + "_" + var_n + ',')

SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_AMT_ANNUITY,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_AMT_APPLICATION,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_AMT_CREDIT,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_PAYMENT_TYPE

In [141]:
df_previous_application_contract_NAME_PAYMENT_TYPE = spark.sql("""
SELECT
SK_ID_CURR,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_AMT_ANNUITY,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_AMT_APPLICATION,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_AMT_CREDIT,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_DAYS_DECISION,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_CNT_PAYMENT,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_DAYS_TERMINATION,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Non_cash_from_your_account_AMT_ANNUITY,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Non_cash_from_your_account_AMT_APPLICATION,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Non_cash_from_your_account_AMT_CREDIT,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Non_cash_from_your_account_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Non_cash_from_your_account_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Non_cash_from_your_account_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Non_cash_from_your_account_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Non_cash_from_your_account_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Non_cash_from_your_account_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Non_cash_from_your_account_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Non_cash_from_your_account_DAYS_DECISION,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Non_cash_from_your_account_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Non_cash_from_your_account_CNT_PAYMENT,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Non_cash_from_your_account_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Non_cash_from_your_account_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Non_cash_from_your_account_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Non_cash_from_your_account_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Non_cash_from_your_account_DAYS_TERMINATION,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Non_cash_from_your_account_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_XNA_AMT_ANNUITY,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_XNA_AMT_APPLICATION,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_XNA_AMT_CREDIT,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_XNA_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_XNA_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_XNA_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_XNA_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_XNA_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_XNA_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_XNA_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_XNA_DAYS_DECISION,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_XNA_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_XNA_CNT_PAYMENT,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_XNA_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_XNA_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_XNA_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_XNA_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_XNA_DAYS_TERMINATION,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_XNA_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cash_through_the_bank_AMT_ANNUITY,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cash_through_the_bank_AMT_APPLICATION,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cash_through_the_bank_AMT_CREDIT,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cash_through_the_bank_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cash_through_the_bank_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cash_through_the_bank_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cash_through_the_bank_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cash_through_the_bank_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cash_through_the_bank_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cash_through_the_bank_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cash_through_the_bank_DAYS_DECISION,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cash_through_the_bank_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cash_through_the_bank_CNT_PAYMENT,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cash_through_the_bank_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cash_through_the_bank_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cash_through_the_bank_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cash_through_the_bank_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cash_through_the_bank_DAYS_TERMINATION,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_PAYMENT_TYPE_Cash_through_the_bank_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_AMT_ANNUITY,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_AMT_APPLICATION,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_AMT_CREDIT,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_DAYS_DECISION,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_CNT_PAYMENT,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_DAYS_TERMINATION,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless from the account of the employer' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Non_cash_from_your_account_AMT_ANNUITY,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Non_cash_from_your_account_AMT_APPLICATION,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Non_cash_from_your_account_AMT_CREDIT,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Non_cash_from_your_account_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Non_cash_from_your_account_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Non_cash_from_your_account_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Non_cash_from_your_account_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Non_cash_from_your_account_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Non_cash_from_your_account_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Non_cash_from_your_account_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Non_cash_from_your_account_DAYS_DECISION,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Non_cash_from_your_account_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Non_cash_from_your_account_CNT_PAYMENT,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Non_cash_from_your_account_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Non_cash_from_your_account_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Non_cash_from_your_account_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Non_cash_from_your_account_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Non_cash_from_your_account_DAYS_TERMINATION,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Non cash from your account' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Non_cash_from_your_account_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_XNA_AMT_ANNUITY,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_XNA_AMT_APPLICATION,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_XNA_AMT_CREDIT,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_XNA_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_XNA_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_XNA_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_XNA_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_XNA_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_XNA_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_XNA_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_XNA_DAYS_DECISION,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_XNA_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_XNA_CNT_PAYMENT,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_XNA_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_XNA_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_XNA_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_XNA_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_XNA_DAYS_TERMINATION,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_XNA_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Cash_through_the_bank_AMT_ANNUITY,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Cash_through_the_bank_AMT_APPLICATION,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Cash_through_the_bank_AMT_CREDIT,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Cash_through_the_bank_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Cash_through_the_bank_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Cash_through_the_bank_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Cash_through_the_bank_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Cash_through_the_bank_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Cash_through_the_bank_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Cash_through_the_bank_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Cash_through_the_bank_DAYS_DECISION,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Cash_through_the_bank_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Cash_through_the_bank_CNT_PAYMENT,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Cash_through_the_bank_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Cash_through_the_bank_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Cash_through_the_bank_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Cash_through_the_bank_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Cash_through_the_bank_DAYS_TERMINATION,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cash through the bank' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_PAYMENT_TYPE_Cash_through_the_bank_NFLAG_INSURED_ON_APPROVAL
FROM
  df_previous_application
GROUP BY
  SK_ID_CURR
""")

###7 - Considerando a variável categórica CODE_REJECT_REASON

In [142]:
# Criando as agregações

agg = ['SUM', 'AVG']

# Criando a variável categórica

cat = ['CODE_REJECT_REASON']

# Criando os domínios

dominios_cat = ['SYSTEM', 'VERIF', 'XNA', 'SCOFR','CLIENT','SCO','LIMIT', 'HC','XAP']

# Criando variável numérica

var_num = ['AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE','HOUR_APPR_PROCESS_START', 'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'DAYS_DECISION', 'SELLERPLACE_AREA', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL']

In [143]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN " + var_n + " ELSE 0 END) AS PA_" + ag + "_" + cate + "_" + dominio.replace(" ", "_").replace('/', '').replace('+', '').replace('(', '').replace(')', '').replace(':', '').replace('-', '').replace('__', '') + "_" + var_n + ',')

SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SYSTEM_AMT_ANNUITY,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SYSTEM_AMT_APPLICATION,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SYSTEM_AMT_CREDIT,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SYSTEM_AMT_DOWN_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SYSTEM_AMT_GOODS_PRICE,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SYSTEM_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SYSTEM_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA

In [144]:
df_previous_application_contract_CODE_REJECT_REASON = spark.sql("""
SELECT
SK_ID_CURR,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SYSTEM_AMT_ANNUITY,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SYSTEM_AMT_APPLICATION,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SYSTEM_AMT_CREDIT,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SYSTEM_AMT_DOWN_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SYSTEM_AMT_GOODS_PRICE,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SYSTEM_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SYSTEM_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SYSTEM_RATE_DOWN_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SYSTEM_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SYSTEM_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SYSTEM_DAYS_DECISION,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SYSTEM_SELLERPLACE_AREA,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SYSTEM_CNT_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SYSTEM_DAYS_FIRST_DRAWING,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SYSTEM_DAYS_FIRST_DUE,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SYSTEM_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SYSTEM_DAYS_LAST_DUE,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SYSTEM_DAYS_TERMINATION,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SYSTEM_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_VERIF_AMT_ANNUITY,
SUM(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_VERIF_AMT_APPLICATION,
SUM(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_VERIF_AMT_CREDIT,
SUM(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_VERIF_AMT_DOWN_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_VERIF_AMT_GOODS_PRICE,
SUM(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_VERIF_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_VERIF_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_VERIF_RATE_DOWN_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_VERIF_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_VERIF_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_VERIF_DAYS_DECISION,
SUM(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_VERIF_SELLERPLACE_AREA,
SUM(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_VERIF_CNT_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_VERIF_DAYS_FIRST_DRAWING,
SUM(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_VERIF_DAYS_FIRST_DUE,
SUM(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_VERIF_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_VERIF_DAYS_LAST_DUE,
SUM(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_VERIF_DAYS_TERMINATION,
SUM(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_VERIF_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_XNA_AMT_ANNUITY,
SUM(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_XNA_AMT_APPLICATION,
SUM(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_XNA_AMT_CREDIT,
SUM(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_XNA_AMT_DOWN_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_XNA_AMT_GOODS_PRICE,
SUM(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_XNA_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_XNA_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_XNA_RATE_DOWN_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_XNA_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_XNA_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_XNA_DAYS_DECISION,
SUM(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_XNA_SELLERPLACE_AREA,
SUM(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_XNA_CNT_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_XNA_DAYS_FIRST_DRAWING,
SUM(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_XNA_DAYS_FIRST_DUE,
SUM(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_XNA_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_XNA_DAYS_LAST_DUE,
SUM(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_XNA_DAYS_TERMINATION,
SUM(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_XNA_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SCOFR_AMT_ANNUITY,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SCOFR_AMT_APPLICATION,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SCOFR_AMT_CREDIT,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SCOFR_AMT_DOWN_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SCOFR_AMT_GOODS_PRICE,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SCOFR_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SCOFR_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SCOFR_RATE_DOWN_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SCOFR_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SCOFR_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SCOFR_DAYS_DECISION,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SCOFR_SELLERPLACE_AREA,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SCOFR_CNT_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SCOFR_DAYS_FIRST_DRAWING,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SCOFR_DAYS_FIRST_DUE,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SCOFR_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SCOFR_DAYS_LAST_DUE,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SCOFR_DAYS_TERMINATION,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SCOFR_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_CLIENT_AMT_ANNUITY,
SUM(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_CLIENT_AMT_APPLICATION,
SUM(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_CLIENT_AMT_CREDIT,
SUM(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_CLIENT_AMT_DOWN_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_CLIENT_AMT_GOODS_PRICE,
SUM(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_CLIENT_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_CLIENT_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_CLIENT_RATE_DOWN_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_CLIENT_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_CLIENT_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_CLIENT_DAYS_DECISION,
SUM(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_CLIENT_SELLERPLACE_AREA,
SUM(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_CLIENT_CNT_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_CLIENT_DAYS_FIRST_DRAWING,
SUM(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_CLIENT_DAYS_FIRST_DUE,
SUM(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_CLIENT_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_CLIENT_DAYS_LAST_DUE,
SUM(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_CLIENT_DAYS_TERMINATION,
SUM(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_CLIENT_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SCO_AMT_ANNUITY,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SCO_AMT_APPLICATION,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SCO_AMT_CREDIT,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SCO_AMT_DOWN_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SCO_AMT_GOODS_PRICE,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SCO_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SCO_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SCO_RATE_DOWN_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SCO_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SCO_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SCO_DAYS_DECISION,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SCO_SELLERPLACE_AREA,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SCO_CNT_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SCO_DAYS_FIRST_DRAWING,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SCO_DAYS_FIRST_DUE,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SCO_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SCO_DAYS_LAST_DUE,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SCO_DAYS_TERMINATION,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_SCO_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_LIMIT_AMT_ANNUITY,
SUM(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_LIMIT_AMT_APPLICATION,
SUM(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_LIMIT_AMT_CREDIT,
SUM(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_LIMIT_AMT_DOWN_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_LIMIT_AMT_GOODS_PRICE,
SUM(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_LIMIT_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_LIMIT_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_LIMIT_RATE_DOWN_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_LIMIT_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_LIMIT_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_LIMIT_DAYS_DECISION,
SUM(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_LIMIT_SELLERPLACE_AREA,
SUM(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_LIMIT_CNT_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_LIMIT_DAYS_FIRST_DRAWING,
SUM(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_LIMIT_DAYS_FIRST_DUE,
SUM(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_LIMIT_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_LIMIT_DAYS_LAST_DUE,
SUM(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_LIMIT_DAYS_TERMINATION,
SUM(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_LIMIT_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN CODE_REJECT_REASON= 'HC' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_HC_AMT_ANNUITY,
SUM(CASE WHEN CODE_REJECT_REASON= 'HC' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_HC_AMT_APPLICATION,
SUM(CASE WHEN CODE_REJECT_REASON= 'HC' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_HC_AMT_CREDIT,
SUM(CASE WHEN CODE_REJECT_REASON= 'HC' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_HC_AMT_DOWN_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'HC' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_HC_AMT_GOODS_PRICE,
SUM(CASE WHEN CODE_REJECT_REASON= 'HC' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_HC_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN CODE_REJECT_REASON= 'HC' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_HC_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN CODE_REJECT_REASON= 'HC' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_HC_RATE_DOWN_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'HC' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_HC_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN CODE_REJECT_REASON= 'HC' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_HC_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN CODE_REJECT_REASON= 'HC' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_HC_DAYS_DECISION,
SUM(CASE WHEN CODE_REJECT_REASON= 'HC' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_HC_SELLERPLACE_AREA,
SUM(CASE WHEN CODE_REJECT_REASON= 'HC' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_HC_CNT_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'HC' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_HC_DAYS_FIRST_DRAWING,
SUM(CASE WHEN CODE_REJECT_REASON= 'HC' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_HC_DAYS_FIRST_DUE,
SUM(CASE WHEN CODE_REJECT_REASON= 'HC' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_HC_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN CODE_REJECT_REASON= 'HC' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_HC_DAYS_LAST_DUE,
SUM(CASE WHEN CODE_REJECT_REASON= 'HC' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_HC_DAYS_TERMINATION,
SUM(CASE WHEN CODE_REJECT_REASON= 'HC' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_HC_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_XAP_AMT_ANNUITY,
SUM(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_XAP_AMT_APPLICATION,
SUM(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_XAP_AMT_CREDIT,
SUM(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_XAP_AMT_DOWN_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_XAP_AMT_GOODS_PRICE,
SUM(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_XAP_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_XAP_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_XAP_RATE_DOWN_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_XAP_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_XAP_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_XAP_DAYS_DECISION,
SUM(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_XAP_SELLERPLACE_AREA,
SUM(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_XAP_CNT_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_XAP_DAYS_FIRST_DRAWING,
SUM(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_XAP_DAYS_FIRST_DUE,
SUM(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_XAP_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_XAP_DAYS_LAST_DUE,
SUM(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_XAP_DAYS_TERMINATION,
SUM(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_CODE_REJECT_REASON_XAP_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SYSTEM_AMT_ANNUITY,
AVG(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SYSTEM_AMT_APPLICATION,
AVG(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SYSTEM_AMT_CREDIT,
AVG(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SYSTEM_AMT_DOWN_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SYSTEM_AMT_GOODS_PRICE,
AVG(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SYSTEM_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SYSTEM_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SYSTEM_RATE_DOWN_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SYSTEM_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SYSTEM_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SYSTEM_DAYS_DECISION,
AVG(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SYSTEM_SELLERPLACE_AREA,
AVG(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SYSTEM_CNT_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SYSTEM_DAYS_FIRST_DRAWING,
AVG(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SYSTEM_DAYS_FIRST_DUE,
AVG(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SYSTEM_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SYSTEM_DAYS_LAST_DUE,
AVG(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SYSTEM_DAYS_TERMINATION,
AVG(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SYSTEM_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_VERIF_AMT_ANNUITY,
AVG(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_VERIF_AMT_APPLICATION,
AVG(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_VERIF_AMT_CREDIT,
AVG(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_VERIF_AMT_DOWN_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_VERIF_AMT_GOODS_PRICE,
AVG(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_VERIF_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_VERIF_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_VERIF_RATE_DOWN_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_VERIF_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_VERIF_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_VERIF_DAYS_DECISION,
AVG(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_VERIF_SELLERPLACE_AREA,
AVG(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_VERIF_CNT_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_VERIF_DAYS_FIRST_DRAWING,
AVG(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_VERIF_DAYS_FIRST_DUE,
AVG(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_VERIF_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_VERIF_DAYS_LAST_DUE,
AVG(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_VERIF_DAYS_TERMINATION,
AVG(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_VERIF_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_XNA_AMT_ANNUITY,
AVG(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_XNA_AMT_APPLICATION,
AVG(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_XNA_AMT_CREDIT,
AVG(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_XNA_AMT_DOWN_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_XNA_AMT_GOODS_PRICE,
AVG(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_XNA_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_XNA_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_XNA_RATE_DOWN_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_XNA_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_XNA_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_XNA_DAYS_DECISION,
AVG(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_XNA_SELLERPLACE_AREA,
AVG(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_XNA_CNT_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_XNA_DAYS_FIRST_DRAWING,
AVG(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_XNA_DAYS_FIRST_DUE,
AVG(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_XNA_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_XNA_DAYS_LAST_DUE,
AVG(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_XNA_DAYS_TERMINATION,
AVG(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_XNA_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SCOFR_AMT_ANNUITY,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SCOFR_AMT_APPLICATION,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SCOFR_AMT_CREDIT,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SCOFR_AMT_DOWN_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SCOFR_AMT_GOODS_PRICE,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SCOFR_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SCOFR_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SCOFR_RATE_DOWN_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SCOFR_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SCOFR_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SCOFR_DAYS_DECISION,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SCOFR_SELLERPLACE_AREA,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SCOFR_CNT_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SCOFR_DAYS_FIRST_DRAWING,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SCOFR_DAYS_FIRST_DUE,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SCOFR_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SCOFR_DAYS_LAST_DUE,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SCOFR_DAYS_TERMINATION,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SCOFR_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_CLIENT_AMT_ANNUITY,
AVG(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_CLIENT_AMT_APPLICATION,
AVG(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_CLIENT_AMT_CREDIT,
AVG(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_CLIENT_AMT_DOWN_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_CLIENT_AMT_GOODS_PRICE,
AVG(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_CLIENT_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_CLIENT_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_CLIENT_RATE_DOWN_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_CLIENT_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_CLIENT_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_CLIENT_DAYS_DECISION,
AVG(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_CLIENT_SELLERPLACE_AREA,
AVG(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_CLIENT_CNT_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_CLIENT_DAYS_FIRST_DRAWING,
AVG(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_CLIENT_DAYS_FIRST_DUE,
AVG(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_CLIENT_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_CLIENT_DAYS_LAST_DUE,
AVG(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_CLIENT_DAYS_TERMINATION,
AVG(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_CLIENT_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SCO_AMT_ANNUITY,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SCO_AMT_APPLICATION,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SCO_AMT_CREDIT,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SCO_AMT_DOWN_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SCO_AMT_GOODS_PRICE,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SCO_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SCO_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SCO_RATE_DOWN_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SCO_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SCO_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SCO_DAYS_DECISION,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SCO_SELLERPLACE_AREA,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SCO_CNT_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SCO_DAYS_FIRST_DRAWING,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SCO_DAYS_FIRST_DUE,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SCO_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SCO_DAYS_LAST_DUE,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SCO_DAYS_TERMINATION,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_SCO_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_LIMIT_AMT_ANNUITY,
AVG(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_LIMIT_AMT_APPLICATION,
AVG(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_LIMIT_AMT_CREDIT,
AVG(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_LIMIT_AMT_DOWN_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_LIMIT_AMT_GOODS_PRICE,
AVG(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_LIMIT_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_LIMIT_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_LIMIT_RATE_DOWN_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_LIMIT_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_LIMIT_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_LIMIT_DAYS_DECISION,
AVG(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_LIMIT_SELLERPLACE_AREA,
AVG(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_LIMIT_CNT_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_LIMIT_DAYS_FIRST_DRAWING,
AVG(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_LIMIT_DAYS_FIRST_DUE,
AVG(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_LIMIT_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_LIMIT_DAYS_LAST_DUE,
AVG(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_LIMIT_DAYS_TERMINATION,
AVG(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_LIMIT_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN CODE_REJECT_REASON= 'HC' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_HC_AMT_ANNUITY,
AVG(CASE WHEN CODE_REJECT_REASON= 'HC' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_HC_AMT_APPLICATION,
AVG(CASE WHEN CODE_REJECT_REASON= 'HC' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_HC_AMT_CREDIT,
AVG(CASE WHEN CODE_REJECT_REASON= 'HC' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_HC_AMT_DOWN_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'HC' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_HC_AMT_GOODS_PRICE,
AVG(CASE WHEN CODE_REJECT_REASON= 'HC' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_HC_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN CODE_REJECT_REASON= 'HC' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_HC_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN CODE_REJECT_REASON= 'HC' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_HC_RATE_DOWN_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'HC' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_HC_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN CODE_REJECT_REASON= 'HC' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_HC_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN CODE_REJECT_REASON= 'HC' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_HC_DAYS_DECISION,
AVG(CASE WHEN CODE_REJECT_REASON= 'HC' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_HC_SELLERPLACE_AREA,
AVG(CASE WHEN CODE_REJECT_REASON= 'HC' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_HC_CNT_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'HC' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_HC_DAYS_FIRST_DRAWING,
AVG(CASE WHEN CODE_REJECT_REASON= 'HC' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_HC_DAYS_FIRST_DUE,
AVG(CASE WHEN CODE_REJECT_REASON= 'HC' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_HC_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN CODE_REJECT_REASON= 'HC' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_HC_DAYS_LAST_DUE,
AVG(CASE WHEN CODE_REJECT_REASON= 'HC' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_HC_DAYS_TERMINATION,
AVG(CASE WHEN CODE_REJECT_REASON= 'HC' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_HC_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_XAP_AMT_ANNUITY,
AVG(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_XAP_AMT_APPLICATION,
AVG(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_XAP_AMT_CREDIT,
AVG(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_XAP_AMT_DOWN_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_XAP_AMT_GOODS_PRICE,
AVG(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_XAP_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_XAP_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_XAP_RATE_DOWN_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_XAP_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_XAP_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_XAP_DAYS_DECISION,
AVG(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_XAP_SELLERPLACE_AREA,
AVG(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_XAP_CNT_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_XAP_DAYS_FIRST_DRAWING,
AVG(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_XAP_DAYS_FIRST_DUE,
AVG(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_XAP_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_XAP_DAYS_LAST_DUE,
AVG(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_XAP_DAYS_TERMINATION,
AVG(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_CODE_REJECT_REASON_XAP_NFLAG_INSURED_ON_APPROVAL
FROM
  df_previous_application
GROUP BY
  SK_ID_CURR
""")

###8 - Considerando a variável categórica NAME_TYPE_SUITE

In [145]:
# Criando as agregações

agg = ['SUM', 'AVG']

# Criando a variável categórica

cat = ['NAME_TYPE_SUITE']

# Criando os domínios

dominios_cat = ['Group of people', 'Other_A', 'Other_B', 'Children','Spouse, partner','Family','Unaccompanied', 'null']

# Criando variável numérica

var_num = ['AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE','HOUR_APPR_PROCESS_START', 'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'DAYS_DECISION', 'SELLERPLACE_AREA', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL']

In [146]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN " + var_n + " ELSE 0 END) AS PA_" + ag + "_" + cate + "_" + dominio.replace(" ", "_").replace('/', '').replace('+', '').replace('(', '').replace(')', '').replace(':', '').replace('-', '').replace(',', '').replace('__', '') + "_" + var_n + ',')

SUM(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Group_of_people_AMT_ANNUITY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Group_of_people_AMT_APPLICATION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Group_of_people_AMT_CREDIT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Group_of_people_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Group_of_people_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Group_of_people_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Group_of_people_NFLAG_LAST_APPL_IN_DAY,

In [147]:
df_previous_application_contract_NAME_TYPE_SUITE = spark.sql("""
SELECT
SK_ID_CURR,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Group_of_people_AMT_ANNUITY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Group_of_people_AMT_APPLICATION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Group_of_people_AMT_CREDIT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Group_of_people_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Group_of_people_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Group_of_people_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Group_of_people_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Group_of_people_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Group_of_people_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Group_of_people_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Group_of_people_DAYS_DECISION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Group_of_people_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Group_of_people_CNT_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Group_of_people_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Group_of_people_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Group_of_people_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Group_of_people_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Group_of_people_DAYS_TERMINATION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Group_of_people_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Other_A_AMT_ANNUITY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Other_A_AMT_APPLICATION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Other_A_AMT_CREDIT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Other_A_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Other_A_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Other_A_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Other_A_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Other_A_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Other_A_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Other_A_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Other_A_DAYS_DECISION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Other_A_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Other_A_CNT_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Other_A_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Other_A_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Other_A_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Other_A_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Other_A_DAYS_TERMINATION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Other_A_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Other_B_AMT_ANNUITY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Other_B_AMT_APPLICATION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Other_B_AMT_CREDIT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Other_B_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Other_B_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Other_B_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Other_B_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Other_B_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Other_B_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Other_B_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Other_B_DAYS_DECISION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Other_B_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Other_B_CNT_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Other_B_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Other_B_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Other_B_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Other_B_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Other_B_DAYS_TERMINATION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Other_B_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Children_AMT_ANNUITY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Children_AMT_APPLICATION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Children_AMT_CREDIT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Children_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Children_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Children_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Children_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Children_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Children_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Children_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Children_DAYS_DECISION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Children_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Children_CNT_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Children_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Children_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Children_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Children_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Children_DAYS_TERMINATION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Children_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Spouse_partner_AMT_ANNUITY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Spouse_partner_AMT_APPLICATION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Spouse_partner_AMT_CREDIT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Spouse_partner_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Spouse_partner_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Spouse_partner_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Spouse_partner_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Spouse_partner_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Spouse_partner_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Spouse_partner_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Spouse_partner_DAYS_DECISION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Spouse_partner_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Spouse_partner_CNT_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Spouse_partner_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Spouse_partner_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Spouse_partner_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Spouse_partner_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Spouse_partner_DAYS_TERMINATION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Spouse_partner_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Family_AMT_ANNUITY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Family_AMT_APPLICATION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Family_AMT_CREDIT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Family_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Family_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Family_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Family_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Family_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Family_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Family_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Family_DAYS_DECISION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Family_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Family_CNT_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Family_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Family_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Family_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Family_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Family_DAYS_TERMINATION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Family_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Unaccompanied_AMT_ANNUITY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Unaccompanied_AMT_APPLICATION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Unaccompanied_AMT_CREDIT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Unaccompanied_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Unaccompanied_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Unaccompanied_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Unaccompanied_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Unaccompanied_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Unaccompanied_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Unaccompanied_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Unaccompanied_DAYS_DECISION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Unaccompanied_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Unaccompanied_CNT_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Unaccompanied_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Unaccompanied_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Unaccompanied_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Unaccompanied_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Unaccompanied_DAYS_TERMINATION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_Unaccompanied_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_TYPE_SUITE= 'null' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_null_AMT_ANNUITY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'null' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_null_AMT_APPLICATION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'null' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_null_AMT_CREDIT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'null' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_null_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'null' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_null_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'null' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_null_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_TYPE_SUITE= 'null' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_null_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'null' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_null_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'null' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_null_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'null' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_null_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_TYPE_SUITE= 'null' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_null_DAYS_DECISION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'null' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_null_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_TYPE_SUITE= 'null' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_null_CNT_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'null' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_null_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_TYPE_SUITE= 'null' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_null_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'null' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_null_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'null' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_null_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'null' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_null_DAYS_TERMINATION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'null' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_TYPE_SUITE_null_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Group_of_people_AMT_ANNUITY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Group_of_people_AMT_APPLICATION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Group_of_people_AMT_CREDIT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Group_of_people_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Group_of_people_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Group_of_people_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Group_of_people_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Group_of_people_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Group_of_people_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Group_of_people_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Group_of_people_DAYS_DECISION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Group_of_people_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Group_of_people_CNT_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Group_of_people_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Group_of_people_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Group_of_people_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Group_of_people_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Group_of_people_DAYS_TERMINATION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Group of people' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Group_of_people_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Other_A_AMT_ANNUITY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Other_A_AMT_APPLICATION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Other_A_AMT_CREDIT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Other_A_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Other_A_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Other_A_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Other_A_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Other_A_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Other_A_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Other_A_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Other_A_DAYS_DECISION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Other_A_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Other_A_CNT_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Other_A_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Other_A_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Other_A_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Other_A_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Other_A_DAYS_TERMINATION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Other_A_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Other_B_AMT_ANNUITY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Other_B_AMT_APPLICATION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Other_B_AMT_CREDIT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Other_B_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Other_B_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Other_B_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Other_B_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Other_B_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Other_B_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Other_B_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Other_B_DAYS_DECISION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Other_B_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Other_B_CNT_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Other_B_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Other_B_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Other_B_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Other_B_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Other_B_DAYS_TERMINATION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Other_B_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Children_AMT_ANNUITY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Children_AMT_APPLICATION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Children_AMT_CREDIT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Children_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Children_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Children_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Children_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Children_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Children_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Children_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Children_DAYS_DECISION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Children_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Children_CNT_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Children_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Children_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Children_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Children_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Children_DAYS_TERMINATION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Children_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Spouse_partner_AMT_ANNUITY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Spouse_partner_AMT_APPLICATION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Spouse_partner_AMT_CREDIT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Spouse_partner_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Spouse_partner_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Spouse_partner_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Spouse_partner_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Spouse_partner_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Spouse_partner_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Spouse_partner_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Spouse_partner_DAYS_DECISION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Spouse_partner_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Spouse_partner_CNT_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Spouse_partner_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Spouse_partner_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Spouse_partner_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Spouse_partner_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Spouse_partner_DAYS_TERMINATION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Spouse, partner' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Spouse_partner_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Family_AMT_ANNUITY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Family_AMT_APPLICATION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Family_AMT_CREDIT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Family_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Family_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Family_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Family_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Family_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Family_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Family_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Family_DAYS_DECISION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Family_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Family_CNT_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Family_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Family_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Family_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Family_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Family_DAYS_TERMINATION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Family_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Unaccompanied_AMT_ANNUITY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Unaccompanied_AMT_APPLICATION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Unaccompanied_AMT_CREDIT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Unaccompanied_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Unaccompanied_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Unaccompanied_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Unaccompanied_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Unaccompanied_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Unaccompanied_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Unaccompanied_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Unaccompanied_DAYS_DECISION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Unaccompanied_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Unaccompanied_CNT_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Unaccompanied_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Unaccompanied_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Unaccompanied_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Unaccompanied_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Unaccompanied_DAYS_TERMINATION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_Unaccompanied_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_TYPE_SUITE= 'null' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_null_AMT_ANNUITY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'null' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_null_AMT_APPLICATION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'null' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_null_AMT_CREDIT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'null' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_null_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'null' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_null_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'null' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_null_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_TYPE_SUITE= 'null' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_null_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'null' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_null_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'null' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_null_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'null' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_null_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_TYPE_SUITE= 'null' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_null_DAYS_DECISION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'null' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_null_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_TYPE_SUITE= 'null' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_null_CNT_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'null' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_null_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_TYPE_SUITE= 'null' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_null_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'null' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_null_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'null' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_null_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'null' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_null_DAYS_TERMINATION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'null' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_TYPE_SUITE_null_NFLAG_INSURED_ON_APPROVAL
FROM
  df_previous_application
GROUP BY
  SK_ID_CURR
""")

###9 - Considerando a variável categórica NAME_CLIENT_TYPE

In [148]:
# Criando as agregações

agg = ['SUM', 'AVG']

# Criando a variável categórica

cat = ['NAME_CLIENT_TYPE']

# Criando os domínios

dominios_cat = ['XNA', 'Refreshed', 'New', 'Repeater']

# Criando variável numérica

var_num = ['AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE','HOUR_APPR_PROCESS_START', 'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'DAYS_DECISION', 'SELLERPLACE_AREA', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL']

In [149]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN " + var_n + " ELSE 0 END) AS PA_" + ag + "_" + cate + "_" + dominio.replace(" ", "_").replace('/', '').replace('+', '').replace('(', '').replace(')', '').replace(':', '').replace('-', '').replace('__', '') + "_" + var_n + ',')

SUM(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_XNA_AMT_ANNUITY,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_XNA_AMT_APPLICATION,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_XNA_AMT_CREDIT,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_XNA_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_XNA_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_XNA_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_XNA_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_XNA_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CLIENT_TYPE

In [150]:
df_previous_application_contract_NAME_CLIENT_TYPE = spark.sql("""
SELECT
SK_ID_CURR,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_XNA_AMT_ANNUITY,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_XNA_AMT_APPLICATION,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_XNA_AMT_CREDIT,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_XNA_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_XNA_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_XNA_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_XNA_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_XNA_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_XNA_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_XNA_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_XNA_DAYS_DECISION,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_XNA_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_XNA_CNT_PAYMENT,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_XNA_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_XNA_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_XNA_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_XNA_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_XNA_DAYS_TERMINATION,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_XNA_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_Refreshed_AMT_ANNUITY,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_Refreshed_AMT_APPLICATION,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_Refreshed_AMT_CREDIT,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_Refreshed_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_Refreshed_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_Refreshed_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_Refreshed_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_Refreshed_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_Refreshed_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_Refreshed_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_Refreshed_DAYS_DECISION,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_Refreshed_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_Refreshed_CNT_PAYMENT,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_Refreshed_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_Refreshed_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_Refreshed_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_Refreshed_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_Refreshed_DAYS_TERMINATION,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_Refreshed_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_New_AMT_ANNUITY,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_New_AMT_APPLICATION,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_New_AMT_CREDIT,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_New_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_New_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_New_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_New_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_New_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_New_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_New_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_New_DAYS_DECISION,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_New_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_New_CNT_PAYMENT,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_New_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_New_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_New_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_New_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_New_DAYS_TERMINATION,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_New_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_Repeater_AMT_ANNUITY,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_Repeater_AMT_APPLICATION,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_Repeater_AMT_CREDIT,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_Repeater_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_Repeater_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_Repeater_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_Repeater_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_Repeater_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_Repeater_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_Repeater_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_Repeater_DAYS_DECISION,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_Repeater_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_Repeater_CNT_PAYMENT,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_Repeater_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_Repeater_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_Repeater_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_Repeater_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_Repeater_DAYS_TERMINATION,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_CLIENT_TYPE_Repeater_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_XNA_AMT_ANNUITY,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_XNA_AMT_APPLICATION,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_XNA_AMT_CREDIT,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_XNA_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_XNA_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_XNA_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_XNA_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_XNA_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_XNA_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_XNA_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_XNA_DAYS_DECISION,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_XNA_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_XNA_CNT_PAYMENT,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_XNA_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_XNA_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_XNA_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_XNA_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_XNA_DAYS_TERMINATION,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_XNA_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_Refreshed_AMT_ANNUITY,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_Refreshed_AMT_APPLICATION,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_Refreshed_AMT_CREDIT,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_Refreshed_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_Refreshed_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_Refreshed_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_Refreshed_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_Refreshed_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_Refreshed_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_Refreshed_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_Refreshed_DAYS_DECISION,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_Refreshed_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_Refreshed_CNT_PAYMENT,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_Refreshed_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_Refreshed_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_Refreshed_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_Refreshed_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_Refreshed_DAYS_TERMINATION,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_Refreshed_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_New_AMT_ANNUITY,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_New_AMT_APPLICATION,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_New_AMT_CREDIT,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_New_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_New_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_New_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_New_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_New_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_New_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_New_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_New_DAYS_DECISION,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_New_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_New_CNT_PAYMENT,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_New_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_New_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_New_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_New_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_New_DAYS_TERMINATION,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_New_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_Repeater_AMT_ANNUITY,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_Repeater_AMT_APPLICATION,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_Repeater_AMT_CREDIT,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_Repeater_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_Repeater_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_Repeater_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_Repeater_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_Repeater_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_Repeater_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_Repeater_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_Repeater_DAYS_DECISION,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_Repeater_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_Repeater_CNT_PAYMENT,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_Repeater_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_Repeater_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_Repeater_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_Repeater_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_Repeater_DAYS_TERMINATION,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_CLIENT_TYPE_Repeater_NFLAG_INSURED_ON_APPROVAL
FROM
  df_previous_application
GROUP BY
  SK_ID_CURR
""")

###10 - Considerando a variável categórica NAME_GOODS_CATEGORY

In [151]:
# Criando as agregações

agg = ['SUM', 'AVG']

# Criando a variável categórica

cat = ['NAME_GOODS_CATEGORY']

# Criando os domínios

dominios_cat = ['House Construction', 'Animals', 'Insurance', 'Education','Additional Service','Fitness','Direct Sales', 'Medicine', 'Tourism','Office Appliances','Other','Gardening','Sport and Leisure','Vehicles','Medical Supplies','Homewares','Jewelry','Auto Accessories','Clothing and Accessories']

# Criando variável numérica

var_num = ['AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE','HOUR_APPR_PROCESS_START', 'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'DAYS_DECISION', 'SELLERPLACE_AREA', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL']

In [152]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN " + var_n + " ELSE 0 END) AS PA_" + ag + "_" + cate + "_" + dominio.replace(" ", "_").replace('/', '').replace('+', '').replace('(', '').replace(')', '').replace(':', '').replace('-', '').replace('__', '') + "_" + var_n + ',')

SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_House_Construction_AMT_ANNUITY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_House_Construction_AMT_APPLICATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_House_Construction_AMT_CREDIT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_House_Construction_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_House_Construction_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_House_Construction_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN NFLAG_LAS

In [153]:
df_previous_application_contract_NAME_GOODS_CATEGORY = spark.sql("""
SELECT
SK_ID_CURR,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_House_Construction_AMT_ANNUITY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_House_Construction_AMT_APPLICATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_House_Construction_AMT_CREDIT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_House_Construction_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_House_Construction_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_House_Construction_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_House_Construction_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_House_Construction_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_House_Construction_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_House_Construction_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_House_Construction_DAYS_DECISION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_House_Construction_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_House_Construction_CNT_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_House_Construction_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_House_Construction_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_House_Construction_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_House_Construction_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_House_Construction_DAYS_TERMINATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_House_Construction_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Animals_AMT_ANNUITY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Animals_AMT_APPLICATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Animals_AMT_CREDIT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Animals_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Animals_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Animals_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Animals_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Animals_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Animals_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Animals_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Animals_DAYS_DECISION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Animals_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Animals_CNT_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Animals_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Animals_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Animals_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Animals_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Animals_DAYS_TERMINATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Animals_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Insurance_AMT_ANNUITY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Insurance_AMT_APPLICATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Insurance_AMT_CREDIT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Insurance_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Insurance_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Insurance_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Insurance_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Insurance_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Insurance_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Insurance_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Insurance_DAYS_DECISION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Insurance_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Insurance_CNT_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Insurance_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Insurance_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Insurance_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Insurance_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Insurance_DAYS_TERMINATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Insurance_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Education_AMT_ANNUITY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Education_AMT_APPLICATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Education_AMT_CREDIT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Education_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Education_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Education_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Education_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Education_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Education_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Education_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Education_DAYS_DECISION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Education_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Education_CNT_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Education_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Education_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Education_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Education_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Education_DAYS_TERMINATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Education_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Additional_Service_AMT_ANNUITY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Additional_Service_AMT_APPLICATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Additional_Service_AMT_CREDIT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Additional_Service_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Additional_Service_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Additional_Service_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Additional_Service_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Additional_Service_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Additional_Service_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Additional_Service_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Additional_Service_DAYS_DECISION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Additional_Service_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Additional_Service_CNT_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Additional_Service_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Additional_Service_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Additional_Service_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Additional_Service_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Additional_Service_DAYS_TERMINATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Additional_Service_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Fitness_AMT_ANNUITY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Fitness_AMT_APPLICATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Fitness_AMT_CREDIT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Fitness_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Fitness_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Fitness_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Fitness_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Fitness_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Fitness_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Fitness_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Fitness_DAYS_DECISION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Fitness_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Fitness_CNT_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Fitness_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Fitness_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Fitness_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Fitness_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Fitness_DAYS_TERMINATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Fitness_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Direct_Sales_AMT_ANNUITY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Direct_Sales_AMT_APPLICATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Direct_Sales_AMT_CREDIT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Direct_Sales_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Direct_Sales_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Direct_Sales_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Direct_Sales_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Direct_Sales_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Direct_Sales_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Direct_Sales_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Direct_Sales_DAYS_DECISION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Direct_Sales_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Direct_Sales_CNT_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Direct_Sales_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Direct_Sales_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Direct_Sales_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Direct_Sales_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Direct_Sales_DAYS_TERMINATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Direct_Sales_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Medicine_AMT_ANNUITY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Medicine_AMT_APPLICATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Medicine_AMT_CREDIT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Medicine_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Medicine_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Medicine_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Medicine_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Medicine_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Medicine_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Medicine_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Medicine_DAYS_DECISION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Medicine_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Medicine_CNT_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Medicine_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Medicine_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Medicine_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Medicine_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Medicine_DAYS_TERMINATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Medicine_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Tourism_AMT_ANNUITY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Tourism_AMT_APPLICATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Tourism_AMT_CREDIT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Tourism_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Tourism_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Tourism_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Tourism_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Tourism_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Tourism_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Tourism_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Tourism_DAYS_DECISION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Tourism_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Tourism_CNT_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Tourism_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Tourism_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Tourism_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Tourism_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Tourism_DAYS_TERMINATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Tourism_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Office Appliances' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Office_Appliances_AMT_ANNUITY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Office Appliances' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Office_Appliances_AMT_APPLICATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Office Appliances' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Office_Appliances_AMT_CREDIT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Office Appliances' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Office_Appliances_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Office Appliances' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Office_Appliances_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Office Appliances' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Office_Appliances_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Office Appliances' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Office_Appliances_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Office Appliances' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Office_Appliances_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Office Appliances' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Office_Appliances_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Office Appliances' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Office_Appliances_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Office Appliances' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Office_Appliances_DAYS_DECISION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Office Appliances' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Office_Appliances_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Office Appliances' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Office_Appliances_CNT_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Office Appliances' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Office_Appliances_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Office Appliances' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Office_Appliances_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Office Appliances' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Office_Appliances_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Office Appliances' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Office_Appliances_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Office Appliances' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Office_Appliances_DAYS_TERMINATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Office Appliances' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Office_Appliances_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Other_AMT_ANNUITY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Other_AMT_APPLICATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Other_AMT_CREDIT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Other_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Other_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Other_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Other_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Other_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Other_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Other_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Other_DAYS_DECISION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Other_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Other_CNT_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Other_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Other_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Other_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Other_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Other_DAYS_TERMINATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Other_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Gardening_AMT_ANNUITY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Gardening_AMT_APPLICATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Gardening_AMT_CREDIT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Gardening_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Gardening_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Gardening_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Gardening_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Gardening_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Gardening_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Gardening_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Gardening_DAYS_DECISION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Gardening_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Gardening_CNT_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Gardening_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Gardening_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Gardening_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Gardening_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Gardening_DAYS_TERMINATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Gardening_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Sport and Leisure' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Sport_and_Leisure_AMT_ANNUITY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Sport and Leisure' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Sport_and_Leisure_AMT_APPLICATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Sport and Leisure' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Sport_and_Leisure_AMT_CREDIT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Sport and Leisure' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Sport_and_Leisure_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Sport and Leisure' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Sport_and_Leisure_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Sport and Leisure' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Sport_and_Leisure_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Sport and Leisure' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Sport_and_Leisure_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Sport and Leisure' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Sport_and_Leisure_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Sport and Leisure' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Sport_and_Leisure_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Sport and Leisure' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Sport_and_Leisure_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Sport and Leisure' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Sport_and_Leisure_DAYS_DECISION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Sport and Leisure' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Sport_and_Leisure_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Sport and Leisure' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Sport_and_Leisure_CNT_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Sport and Leisure' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Sport_and_Leisure_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Sport and Leisure' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Sport_and_Leisure_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Sport and Leisure' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Sport_and_Leisure_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Sport and Leisure' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Sport_and_Leisure_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Sport and Leisure' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Sport_and_Leisure_DAYS_TERMINATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Sport and Leisure' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Sport_and_Leisure_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Vehicles_AMT_ANNUITY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Vehicles_AMT_APPLICATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Vehicles_AMT_CREDIT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Vehicles_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Vehicles_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Vehicles_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Vehicles_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Vehicles_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Vehicles_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Vehicles_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Vehicles_DAYS_DECISION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Vehicles_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Vehicles_CNT_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Vehicles_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Vehicles_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Vehicles_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Vehicles_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Vehicles_DAYS_TERMINATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Vehicles_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medical Supplies' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Medical_Supplies_AMT_ANNUITY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medical Supplies' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Medical_Supplies_AMT_APPLICATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medical Supplies' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Medical_Supplies_AMT_CREDIT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medical Supplies' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Medical_Supplies_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medical Supplies' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Medical_Supplies_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medical Supplies' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Medical_Supplies_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medical Supplies' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Medical_Supplies_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medical Supplies' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Medical_Supplies_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medical Supplies' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Medical_Supplies_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medical Supplies' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Medical_Supplies_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medical Supplies' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Medical_Supplies_DAYS_DECISION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medical Supplies' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Medical_Supplies_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medical Supplies' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Medical_Supplies_CNT_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medical Supplies' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Medical_Supplies_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medical Supplies' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Medical_Supplies_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medical Supplies' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Medical_Supplies_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medical Supplies' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Medical_Supplies_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medical Supplies' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Medical_Supplies_DAYS_TERMINATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medical Supplies' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Medical_Supplies_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Homewares_AMT_ANNUITY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Homewares_AMT_APPLICATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Homewares_AMT_CREDIT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Homewares_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Homewares_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Homewares_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Homewares_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Homewares_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Homewares_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Homewares_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Homewares_DAYS_DECISION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Homewares_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Homewares_CNT_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Homewares_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Homewares_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Homewares_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Homewares_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Homewares_DAYS_TERMINATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Homewares_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Jewelry_AMT_ANNUITY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Jewelry_AMT_APPLICATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Jewelry_AMT_CREDIT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Jewelry_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Jewelry_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Jewelry_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Jewelry_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Jewelry_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Jewelry_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Jewelry_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Jewelry_DAYS_DECISION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Jewelry_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Jewelry_CNT_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Jewelry_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Jewelry_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Jewelry_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Jewelry_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Jewelry_DAYS_TERMINATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Jewelry_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Auto Accessories' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Auto_Accessories_AMT_ANNUITY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Auto Accessories' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Auto_Accessories_AMT_APPLICATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Auto Accessories' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Auto_Accessories_AMT_CREDIT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Auto Accessories' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Auto_Accessories_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Auto Accessories' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Auto_Accessories_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Auto Accessories' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Auto_Accessories_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Auto Accessories' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Auto_Accessories_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Auto Accessories' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Auto_Accessories_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Auto Accessories' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Auto_Accessories_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Auto Accessories' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Auto_Accessories_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Auto Accessories' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Auto_Accessories_DAYS_DECISION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Auto Accessories' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Auto_Accessories_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Auto Accessories' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Auto_Accessories_CNT_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Auto Accessories' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Auto_Accessories_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Auto Accessories' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Auto_Accessories_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Auto Accessories' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Auto_Accessories_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Auto Accessories' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Auto_Accessories_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Auto Accessories' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Auto_Accessories_DAYS_TERMINATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Auto Accessories' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Auto_Accessories_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing and Accessories' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Clothing_and_Accessories_AMT_ANNUITY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing and Accessories' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Clothing_and_Accessories_AMT_APPLICATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing and Accessories' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Clothing_and_Accessories_AMT_CREDIT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing and Accessories' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Clothing_and_Accessories_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing and Accessories' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Clothing_and_Accessories_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing and Accessories' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Clothing_and_Accessories_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing and Accessories' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Clothing_and_Accessories_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing and Accessories' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Clothing_and_Accessories_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing and Accessories' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Clothing_and_Accessories_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing and Accessories' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Clothing_and_Accessories_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing and Accessories' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Clothing_and_Accessories_DAYS_DECISION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing and Accessories' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Clothing_and_Accessories_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing and Accessories' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Clothing_and_Accessories_CNT_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing and Accessories' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Clothing_and_Accessories_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing and Accessories' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Clothing_and_Accessories_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing and Accessories' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Clothing_and_Accessories_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing and Accessories' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Clothing_and_Accessories_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing and Accessories' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Clothing_and_Accessories_DAYS_TERMINATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing and Accessories' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_GOODS_CATEGORY_Clothing_and_Accessories_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_House_Construction_AMT_ANNUITY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_House_Construction_AMT_APPLICATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_House_Construction_AMT_CREDIT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_House_Construction_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_House_Construction_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_House_Construction_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_House_Construction_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_House_Construction_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_House_Construction_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_House_Construction_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_House_Construction_DAYS_DECISION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_House_Construction_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_House_Construction_CNT_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_House_Construction_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_House_Construction_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_House_Construction_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_House_Construction_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_House_Construction_DAYS_TERMINATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'House Construction' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_House_Construction_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Animals_AMT_ANNUITY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Animals_AMT_APPLICATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Animals_AMT_CREDIT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Animals_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Animals_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Animals_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Animals_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Animals_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Animals_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Animals_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Animals_DAYS_DECISION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Animals_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Animals_CNT_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Animals_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Animals_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Animals_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Animals_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Animals_DAYS_TERMINATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Animals_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Insurance_AMT_ANNUITY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Insurance_AMT_APPLICATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Insurance_AMT_CREDIT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Insurance_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Insurance_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Insurance_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Insurance_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Insurance_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Insurance_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Insurance_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Insurance_DAYS_DECISION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Insurance_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Insurance_CNT_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Insurance_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Insurance_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Insurance_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Insurance_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Insurance_DAYS_TERMINATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Insurance_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Education_AMT_ANNUITY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Education_AMT_APPLICATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Education_AMT_CREDIT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Education_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Education_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Education_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Education_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Education_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Education_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Education_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Education_DAYS_DECISION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Education_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Education_CNT_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Education_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Education_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Education_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Education_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Education_DAYS_TERMINATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Education_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Additional_Service_AMT_ANNUITY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Additional_Service_AMT_APPLICATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Additional_Service_AMT_CREDIT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Additional_Service_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Additional_Service_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Additional_Service_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Additional_Service_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Additional_Service_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Additional_Service_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Additional_Service_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Additional_Service_DAYS_DECISION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Additional_Service_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Additional_Service_CNT_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Additional_Service_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Additional_Service_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Additional_Service_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Additional_Service_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Additional_Service_DAYS_TERMINATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Additional Service' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Additional_Service_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Fitness_AMT_ANNUITY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Fitness_AMT_APPLICATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Fitness_AMT_CREDIT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Fitness_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Fitness_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Fitness_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Fitness_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Fitness_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Fitness_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Fitness_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Fitness_DAYS_DECISION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Fitness_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Fitness_CNT_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Fitness_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Fitness_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Fitness_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Fitness_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Fitness_DAYS_TERMINATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Fitness_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Direct_Sales_AMT_ANNUITY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Direct_Sales_AMT_APPLICATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Direct_Sales_AMT_CREDIT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Direct_Sales_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Direct_Sales_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Direct_Sales_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Direct_Sales_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Direct_Sales_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Direct_Sales_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Direct_Sales_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Direct_Sales_DAYS_DECISION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Direct_Sales_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Direct_Sales_CNT_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Direct_Sales_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Direct_Sales_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Direct_Sales_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Direct_Sales_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Direct_Sales_DAYS_TERMINATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Direct Sales' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Direct_Sales_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Medicine_AMT_ANNUITY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Medicine_AMT_APPLICATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Medicine_AMT_CREDIT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Medicine_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Medicine_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Medicine_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Medicine_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Medicine_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Medicine_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Medicine_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Medicine_DAYS_DECISION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Medicine_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Medicine_CNT_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Medicine_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Medicine_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Medicine_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Medicine_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Medicine_DAYS_TERMINATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Medicine_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Tourism_AMT_ANNUITY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Tourism_AMT_APPLICATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Tourism_AMT_CREDIT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Tourism_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Tourism_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Tourism_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Tourism_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Tourism_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Tourism_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Tourism_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Tourism_DAYS_DECISION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Tourism_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Tourism_CNT_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Tourism_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Tourism_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Tourism_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Tourism_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Tourism_DAYS_TERMINATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Tourism_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Office Appliances' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Office_Appliances_AMT_ANNUITY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Office Appliances' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Office_Appliances_AMT_APPLICATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Office Appliances' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Office_Appliances_AMT_CREDIT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Office Appliances' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Office_Appliances_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Office Appliances' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Office_Appliances_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Office Appliances' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Office_Appliances_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Office Appliances' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Office_Appliances_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Office Appliances' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Office_Appliances_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Office Appliances' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Office_Appliances_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Office Appliances' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Office_Appliances_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Office Appliances' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Office_Appliances_DAYS_DECISION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Office Appliances' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Office_Appliances_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Office Appliances' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Office_Appliances_CNT_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Office Appliances' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Office_Appliances_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Office Appliances' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Office_Appliances_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Office Appliances' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Office_Appliances_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Office Appliances' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Office_Appliances_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Office Appliances' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Office_Appliances_DAYS_TERMINATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Office Appliances' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Office_Appliances_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Other_AMT_ANNUITY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Other_AMT_APPLICATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Other_AMT_CREDIT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Other_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Other_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Other_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Other_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Other_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Other_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Other_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Other_DAYS_DECISION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Other_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Other_CNT_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Other_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Other_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Other_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Other_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Other_DAYS_TERMINATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Other_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Gardening_AMT_ANNUITY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Gardening_AMT_APPLICATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Gardening_AMT_CREDIT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Gardening_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Gardening_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Gardening_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Gardening_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Gardening_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Gardening_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Gardening_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Gardening_DAYS_DECISION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Gardening_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Gardening_CNT_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Gardening_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Gardening_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Gardening_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Gardening_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Gardening_DAYS_TERMINATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Gardening_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Sport and Leisure' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Sport_and_Leisure_AMT_ANNUITY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Sport and Leisure' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Sport_and_Leisure_AMT_APPLICATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Sport and Leisure' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Sport_and_Leisure_AMT_CREDIT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Sport and Leisure' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Sport_and_Leisure_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Sport and Leisure' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Sport_and_Leisure_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Sport and Leisure' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Sport_and_Leisure_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Sport and Leisure' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Sport_and_Leisure_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Sport and Leisure' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Sport_and_Leisure_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Sport and Leisure' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Sport_and_Leisure_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Sport and Leisure' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Sport_and_Leisure_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Sport and Leisure' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Sport_and_Leisure_DAYS_DECISION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Sport and Leisure' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Sport_and_Leisure_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Sport and Leisure' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Sport_and_Leisure_CNT_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Sport and Leisure' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Sport_and_Leisure_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Sport and Leisure' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Sport_and_Leisure_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Sport and Leisure' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Sport_and_Leisure_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Sport and Leisure' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Sport_and_Leisure_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Sport and Leisure' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Sport_and_Leisure_DAYS_TERMINATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Sport and Leisure' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Sport_and_Leisure_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Vehicles_AMT_ANNUITY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Vehicles_AMT_APPLICATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Vehicles_AMT_CREDIT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Vehicles_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Vehicles_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Vehicles_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Vehicles_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Vehicles_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Vehicles_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Vehicles_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Vehicles_DAYS_DECISION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Vehicles_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Vehicles_CNT_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Vehicles_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Vehicles_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Vehicles_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Vehicles_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Vehicles_DAYS_TERMINATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Vehicles_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medical Supplies' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Medical_Supplies_AMT_ANNUITY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medical Supplies' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Medical_Supplies_AMT_APPLICATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medical Supplies' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Medical_Supplies_AMT_CREDIT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medical Supplies' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Medical_Supplies_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medical Supplies' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Medical_Supplies_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medical Supplies' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Medical_Supplies_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medical Supplies' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Medical_Supplies_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medical Supplies' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Medical_Supplies_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medical Supplies' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Medical_Supplies_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medical Supplies' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Medical_Supplies_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medical Supplies' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Medical_Supplies_DAYS_DECISION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medical Supplies' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Medical_Supplies_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medical Supplies' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Medical_Supplies_CNT_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medical Supplies' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Medical_Supplies_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medical Supplies' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Medical_Supplies_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medical Supplies' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Medical_Supplies_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medical Supplies' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Medical_Supplies_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medical Supplies' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Medical_Supplies_DAYS_TERMINATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medical Supplies' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Medical_Supplies_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Homewares_AMT_ANNUITY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Homewares_AMT_APPLICATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Homewares_AMT_CREDIT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Homewares_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Homewares_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Homewares_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Homewares_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Homewares_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Homewares_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Homewares_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Homewares_DAYS_DECISION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Homewares_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Homewares_CNT_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Homewares_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Homewares_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Homewares_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Homewares_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Homewares_DAYS_TERMINATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Homewares_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Jewelry_AMT_ANNUITY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Jewelry_AMT_APPLICATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Jewelry_AMT_CREDIT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Jewelry_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Jewelry_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Jewelry_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Jewelry_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Jewelry_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Jewelry_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Jewelry_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Jewelry_DAYS_DECISION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Jewelry_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Jewelry_CNT_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Jewelry_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Jewelry_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Jewelry_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Jewelry_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Jewelry_DAYS_TERMINATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Jewelry_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Auto Accessories' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Auto_Accessories_AMT_ANNUITY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Auto Accessories' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Auto_Accessories_AMT_APPLICATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Auto Accessories' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Auto_Accessories_AMT_CREDIT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Auto Accessories' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Auto_Accessories_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Auto Accessories' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Auto_Accessories_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Auto Accessories' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Auto_Accessories_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Auto Accessories' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Auto_Accessories_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Auto Accessories' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Auto_Accessories_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Auto Accessories' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Auto_Accessories_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Auto Accessories' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Auto_Accessories_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Auto Accessories' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Auto_Accessories_DAYS_DECISION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Auto Accessories' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Auto_Accessories_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Auto Accessories' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Auto_Accessories_CNT_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Auto Accessories' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Auto_Accessories_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Auto Accessories' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Auto_Accessories_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Auto Accessories' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Auto_Accessories_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Auto Accessories' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Auto_Accessories_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Auto Accessories' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Auto_Accessories_DAYS_TERMINATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Auto Accessories' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Auto_Accessories_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing and Accessories' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Clothing_and_Accessories_AMT_ANNUITY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing and Accessories' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Clothing_and_Accessories_AMT_APPLICATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing and Accessories' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Clothing_and_Accessories_AMT_CREDIT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing and Accessories' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Clothing_and_Accessories_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing and Accessories' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Clothing_and_Accessories_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing and Accessories' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Clothing_and_Accessories_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing and Accessories' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Clothing_and_Accessories_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing and Accessories' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Clothing_and_Accessories_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing and Accessories' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Clothing_and_Accessories_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing and Accessories' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Clothing_and_Accessories_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing and Accessories' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Clothing_and_Accessories_DAYS_DECISION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing and Accessories' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Clothing_and_Accessories_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing and Accessories' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Clothing_and_Accessories_CNT_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing and Accessories' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Clothing_and_Accessories_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing and Accessories' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Clothing_and_Accessories_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing and Accessories' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Clothing_and_Accessories_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing and Accessories' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Clothing_and_Accessories_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing and Accessories' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Clothing_and_Accessories_DAYS_TERMINATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing and Accessories' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_GOODS_CATEGORY_Clothing_and_Accessories_NFLAG_INSURED_ON_APPROVAL
FROM
  df_previous_application
GROUP BY
  SK_ID_CURR
""")

###11 - Considerando a variável categórica NAME_PORTFOLIO

In [154]:
# Criando as agregações

agg = ['SUM', 'AVG']

# Criando a variável categórica

cat = ['NAME_PORTFOLIO']

# Criando os domínios

dominios_cat = ['Cars', 'Cards', 'XNA', 'Cash','POS']

# Criando variável numérica

var_num = ['AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE','HOUR_APPR_PROCESS_START', 'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'DAYS_DECISION', 'SELLERPLACE_AREA', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL']

In [155]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN " + var_n + " ELSE 0 END) AS PA_" + ag + "_" + cate + "_" + dominio.replace(" ", "_").replace('/', '').replace('+', '').replace('(', '').replace(')', '').replace(':', '').replace('-', '').replace('__', '') + "_" + var_n + ',')

SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cars_AMT_ANNUITY,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cars_AMT_APPLICATION,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cars_AMT_CREDIT,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cars_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cars_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cars_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cars_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cars_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN RATE

In [156]:
df_previous_application_contract_NAME_PORTFOLIO = spark.sql("""
SELECT
SK_ID_CURR,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cars_AMT_ANNUITY,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cars_AMT_APPLICATION,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cars_AMT_CREDIT,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cars_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cars_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cars_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cars_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cars_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cars_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cars_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cars_DAYS_DECISION,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cars_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cars_CNT_PAYMENT,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cars_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cars_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cars_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cars_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cars_DAYS_TERMINATION,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cars_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cards_AMT_ANNUITY,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cards_AMT_APPLICATION,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cards_AMT_CREDIT,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cards_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cards_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cards_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cards_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cards_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cards_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cards_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cards_DAYS_DECISION,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cards_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cards_CNT_PAYMENT,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cards_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cards_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cards_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cards_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cards_DAYS_TERMINATION,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cards_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_XNA_AMT_ANNUITY,
SUM(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_XNA_AMT_APPLICATION,
SUM(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_XNA_AMT_CREDIT,
SUM(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_XNA_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_XNA_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_XNA_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_XNA_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_XNA_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_XNA_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_XNA_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_XNA_DAYS_DECISION,
SUM(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_XNA_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_XNA_CNT_PAYMENT,
SUM(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_XNA_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_XNA_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_XNA_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_XNA_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_XNA_DAYS_TERMINATION,
SUM(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_XNA_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cash_AMT_ANNUITY,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cash_AMT_APPLICATION,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cash_AMT_CREDIT,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cash_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cash_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cash_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cash_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cash_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cash_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cash_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cash_DAYS_DECISION,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cash_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cash_CNT_PAYMENT,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cash_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cash_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cash_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cash_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cash_DAYS_TERMINATION,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_Cash_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_PORTFOLIO= 'POS' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_POS_AMT_ANNUITY,
SUM(CASE WHEN NAME_PORTFOLIO= 'POS' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_POS_AMT_APPLICATION,
SUM(CASE WHEN NAME_PORTFOLIO= 'POS' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_POS_AMT_CREDIT,
SUM(CASE WHEN NAME_PORTFOLIO= 'POS' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_POS_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PORTFOLIO= 'POS' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_POS_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_PORTFOLIO= 'POS' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_POS_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_PORTFOLIO= 'POS' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_POS_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_PORTFOLIO= 'POS' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_POS_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PORTFOLIO= 'POS' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_POS_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_PORTFOLIO= 'POS' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_POS_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_PORTFOLIO= 'POS' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_POS_DAYS_DECISION,
SUM(CASE WHEN NAME_PORTFOLIO= 'POS' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_POS_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_PORTFOLIO= 'POS' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_POS_CNT_PAYMENT,
SUM(CASE WHEN NAME_PORTFOLIO= 'POS' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_POS_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_PORTFOLIO= 'POS' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_POS_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_PORTFOLIO= 'POS' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_POS_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_PORTFOLIO= 'POS' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_POS_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_PORTFOLIO= 'POS' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_POS_DAYS_TERMINATION,
SUM(CASE WHEN NAME_PORTFOLIO= 'POS' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_PORTFOLIO_POS_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cars_AMT_ANNUITY,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cars_AMT_APPLICATION,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cars_AMT_CREDIT,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cars_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cars_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cars_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cars_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cars_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cars_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cars_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cars_DAYS_DECISION,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cars_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cars_CNT_PAYMENT,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cars_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cars_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cars_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cars_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cars_DAYS_TERMINATION,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cars_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cards_AMT_ANNUITY,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cards_AMT_APPLICATION,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cards_AMT_CREDIT,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cards_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cards_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cards_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cards_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cards_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cards_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cards_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cards_DAYS_DECISION,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cards_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cards_CNT_PAYMENT,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cards_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cards_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cards_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cards_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cards_DAYS_TERMINATION,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cards_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_XNA_AMT_ANNUITY,
AVG(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_XNA_AMT_APPLICATION,
AVG(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_XNA_AMT_CREDIT,
AVG(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_XNA_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_XNA_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_XNA_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_XNA_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_XNA_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_XNA_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_XNA_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_XNA_DAYS_DECISION,
AVG(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_XNA_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_XNA_CNT_PAYMENT,
AVG(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_XNA_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_XNA_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_XNA_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_XNA_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_XNA_DAYS_TERMINATION,
AVG(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_XNA_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cash_AMT_ANNUITY,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cash_AMT_APPLICATION,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cash_AMT_CREDIT,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cash_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cash_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cash_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cash_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cash_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cash_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cash_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cash_DAYS_DECISION,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cash_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cash_CNT_PAYMENT,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cash_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cash_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cash_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cash_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cash_DAYS_TERMINATION,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_Cash_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_PORTFOLIO= 'POS' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_POS_AMT_ANNUITY,
AVG(CASE WHEN NAME_PORTFOLIO= 'POS' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_POS_AMT_APPLICATION,
AVG(CASE WHEN NAME_PORTFOLIO= 'POS' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_POS_AMT_CREDIT,
AVG(CASE WHEN NAME_PORTFOLIO= 'POS' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_POS_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PORTFOLIO= 'POS' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_POS_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_PORTFOLIO= 'POS' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_POS_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_PORTFOLIO= 'POS' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_POS_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_PORTFOLIO= 'POS' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_POS_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PORTFOLIO= 'POS' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_POS_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_PORTFOLIO= 'POS' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_POS_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_PORTFOLIO= 'POS' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_POS_DAYS_DECISION,
AVG(CASE WHEN NAME_PORTFOLIO= 'POS' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_POS_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_PORTFOLIO= 'POS' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_POS_CNT_PAYMENT,
AVG(CASE WHEN NAME_PORTFOLIO= 'POS' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_POS_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_PORTFOLIO= 'POS' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_POS_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_PORTFOLIO= 'POS' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_POS_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_PORTFOLIO= 'POS' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_POS_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_PORTFOLIO= 'POS' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_POS_DAYS_TERMINATION,
AVG(CASE WHEN NAME_PORTFOLIO= 'POS' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_PORTFOLIO_POS_NFLAG_INSURED_ON_APPROVAL
FROM
  df_previous_application
GROUP BY
  SK_ID_CURR
""")

###12 - Considerando a variável categórica NAME_PRODUCT_TYPE

In [157]:
# Criando as agregações

agg = ['SUM', 'AVG']

# Criando a variável categórica

cat = ['NAME_PRODUCT_TYPE']

# Criando os domínios

dominios_cat = ['walk-in', 'x-sell', 'XNA']

# Criando variável numérica

var_num = ['AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE','HOUR_APPR_PROCESS_START', 'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'DAYS_DECISION', 'SELLERPLACE_AREA', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL']

In [158]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN " + var_n + " ELSE 0 END) AS PA_" + ag + "_" + cate + "_" + dominio.replace(" ", "_").replace('/', '').replace('+', '').replace('(', '').replace(')', '').replace(':', '').replace('-', '').replace('__', '') + "_" + var_n + ',')

SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_walkin_AMT_ANNUITY,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_walkin_AMT_APPLICATION,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_walkin_AMT_CREDIT,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_walkin_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_walkin_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_walkin_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_walkin_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NA

In [159]:
df_previous_application_contract_NAME_PRODUCT_TYPE = spark.sql("""
SELECT
SK_ID_CURR,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_walkin_AMT_ANNUITY,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_walkin_AMT_APPLICATION,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_walkin_AMT_CREDIT,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_walkin_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_walkin_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_walkin_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_walkin_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_walkin_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_walkin_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_walkin_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_walkin_DAYS_DECISION,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_walkin_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_walkin_CNT_PAYMENT,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_walkin_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_walkin_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_walkin_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_walkin_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_walkin_DAYS_TERMINATION,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_walkin_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_xsell_AMT_ANNUITY,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_xsell_AMT_APPLICATION,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_xsell_AMT_CREDIT,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_xsell_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_xsell_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_xsell_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_xsell_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_xsell_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_xsell_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_xsell_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_xsell_DAYS_DECISION,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_xsell_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_xsell_CNT_PAYMENT,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_xsell_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_xsell_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_xsell_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_xsell_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_xsell_DAYS_TERMINATION,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_xsell_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_XNA_AMT_ANNUITY,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_XNA_AMT_APPLICATION,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_XNA_AMT_CREDIT,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_XNA_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_XNA_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_XNA_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_XNA_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_XNA_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_XNA_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_XNA_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_XNA_DAYS_DECISION,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_XNA_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_XNA_CNT_PAYMENT,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_XNA_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_XNA_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_XNA_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_XNA_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_XNA_DAYS_TERMINATION,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_PRODUCT_TYPE_XNA_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_walkin_AMT_ANNUITY,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_walkin_AMT_APPLICATION,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_walkin_AMT_CREDIT,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_walkin_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_walkin_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_walkin_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_walkin_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_walkin_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_walkin_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_walkin_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_walkin_DAYS_DECISION,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_walkin_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_walkin_CNT_PAYMENT,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_walkin_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_walkin_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_walkin_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_walkin_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_walkin_DAYS_TERMINATION,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'walk-in' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_walkin_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_xsell_AMT_ANNUITY,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_xsell_AMT_APPLICATION,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_xsell_AMT_CREDIT,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_xsell_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_xsell_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_xsell_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_xsell_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_xsell_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_xsell_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_xsell_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_xsell_DAYS_DECISION,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_xsell_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_xsell_CNT_PAYMENT,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_xsell_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_xsell_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_xsell_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_xsell_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_xsell_DAYS_TERMINATION,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'x-sell' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_xsell_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_XNA_AMT_ANNUITY,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_XNA_AMT_APPLICATION,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_XNA_AMT_CREDIT,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_XNA_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_XNA_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_XNA_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_XNA_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_XNA_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_XNA_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_XNA_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_XNA_DAYS_DECISION,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_XNA_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_XNA_CNT_PAYMENT,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_XNA_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_XNA_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_XNA_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_XNA_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_XNA_DAYS_TERMINATION,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_PRODUCT_TYPE_XNA_NFLAG_INSURED_ON_APPROVAL
FROM
  df_previous_application
GROUP BY
  SK_ID_CURR
""")

###13 - Considerando a variável categórica CHANNEL_TYPE

In [160]:
# Criando as agregações

agg = ['SUM', 'AVG']

# Criando a variável categórica

cat = ['CHANNEL_TYPE']

# Criando os domínios

dominios_cat = ['Car dealer', 'Channel of corporate sales', 'AP+ (Cash loan)','Contact center','Regional / Local','Stone','Country-wide','Credit and cash offices']

# Criando variável numérica

var_num = ['AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE','HOUR_APPR_PROCESS_START', 'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'DAYS_DECISION', 'SELLERPLACE_AREA', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL']

In [161]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN " + var_n + " ELSE 0 END) AS PA_" + ag + "_" + cate + "_" + dominio.replace(" ", "_").replace('/', '').replace('+', '').replace('(', '').replace(')', '').replace(':', '').replace('-', '').replace('__', '') + "_" + var_n + ',')

SUM(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Car_dealer_AMT_ANNUITY,
SUM(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Car_dealer_AMT_APPLICATION,
SUM(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Car_dealer_AMT_CREDIT,
SUM(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Car_dealer_AMT_DOWN_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Car_dealer_AMT_GOODS_PRICE,
SUM(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Car_dealer_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Car_dealer_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Car_dealer_R

In [162]:
df_previous_application_contract_CHANNEL_TYPE = spark.sql("""
SELECT
SK_ID_CURR,
SUM(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Car_dealer_AMT_ANNUITY,
SUM(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Car_dealer_AMT_APPLICATION,
SUM(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Car_dealer_AMT_CREDIT,
SUM(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Car_dealer_AMT_DOWN_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Car_dealer_AMT_GOODS_PRICE,
SUM(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Car_dealer_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Car_dealer_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Car_dealer_RATE_DOWN_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Car_dealer_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Car_dealer_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Car_dealer_DAYS_DECISION,
SUM(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Car_dealer_SELLERPLACE_AREA,
SUM(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Car_dealer_CNT_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Car_dealer_DAYS_FIRST_DRAWING,
SUM(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Car_dealer_DAYS_FIRST_DUE,
SUM(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Car_dealer_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Car_dealer_DAYS_LAST_DUE,
SUM(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Car_dealer_DAYS_TERMINATION,
SUM(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Car_dealer_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Channel_of_corporate_sales_AMT_ANNUITY,
SUM(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Channel_of_corporate_sales_AMT_APPLICATION,
SUM(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Channel_of_corporate_sales_AMT_CREDIT,
SUM(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Channel_of_corporate_sales_AMT_DOWN_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Channel_of_corporate_sales_AMT_GOODS_PRICE,
SUM(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Channel_of_corporate_sales_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Channel_of_corporate_sales_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Channel_of_corporate_sales_RATE_DOWN_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Channel_of_corporate_sales_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Channel_of_corporate_sales_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Channel_of_corporate_sales_DAYS_DECISION,
SUM(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Channel_of_corporate_sales_SELLERPLACE_AREA,
SUM(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Channel_of_corporate_sales_CNT_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Channel_of_corporate_sales_DAYS_FIRST_DRAWING,
SUM(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Channel_of_corporate_sales_DAYS_FIRST_DUE,
SUM(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Channel_of_corporate_sales_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Channel_of_corporate_sales_DAYS_LAST_DUE,
SUM(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Channel_of_corporate_sales_DAYS_TERMINATION,
SUM(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Channel_of_corporate_sales_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_AP_Cash_loan_AMT_ANNUITY,
SUM(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_AP_Cash_loan_AMT_APPLICATION,
SUM(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_AP_Cash_loan_AMT_CREDIT,
SUM(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_AP_Cash_loan_AMT_DOWN_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_AP_Cash_loan_AMT_GOODS_PRICE,
SUM(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_AP_Cash_loan_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_AP_Cash_loan_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_AP_Cash_loan_RATE_DOWN_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_AP_Cash_loan_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_AP_Cash_loan_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_AP_Cash_loan_DAYS_DECISION,
SUM(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_AP_Cash_loan_SELLERPLACE_AREA,
SUM(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_AP_Cash_loan_CNT_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_AP_Cash_loan_DAYS_FIRST_DRAWING,
SUM(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_AP_Cash_loan_DAYS_FIRST_DUE,
SUM(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_AP_Cash_loan_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_AP_Cash_loan_DAYS_LAST_DUE,
SUM(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_AP_Cash_loan_DAYS_TERMINATION,
SUM(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_AP_Cash_loan_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Contact_center_AMT_ANNUITY,
SUM(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Contact_center_AMT_APPLICATION,
SUM(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Contact_center_AMT_CREDIT,
SUM(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Contact_center_AMT_DOWN_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Contact_center_AMT_GOODS_PRICE,
SUM(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Contact_center_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Contact_center_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Contact_center_RATE_DOWN_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Contact_center_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Contact_center_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Contact_center_DAYS_DECISION,
SUM(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Contact_center_SELLERPLACE_AREA,
SUM(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Contact_center_CNT_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Contact_center_DAYS_FIRST_DRAWING,
SUM(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Contact_center_DAYS_FIRST_DUE,
SUM(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Contact_center_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Contact_center_DAYS_LAST_DUE,
SUM(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Contact_center_DAYS_TERMINATION,
SUM(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Contact_center_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_RegionalLocal_AMT_ANNUITY,
SUM(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_RegionalLocal_AMT_APPLICATION,
SUM(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_RegionalLocal_AMT_CREDIT,
SUM(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_RegionalLocal_AMT_DOWN_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_RegionalLocal_AMT_GOODS_PRICE,
SUM(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_RegionalLocal_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_RegionalLocal_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_RegionalLocal_RATE_DOWN_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_RegionalLocal_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_RegionalLocal_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_RegionalLocal_DAYS_DECISION,
SUM(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_RegionalLocal_SELLERPLACE_AREA,
SUM(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_RegionalLocal_CNT_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_RegionalLocal_DAYS_FIRST_DRAWING,
SUM(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_RegionalLocal_DAYS_FIRST_DUE,
SUM(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_RegionalLocal_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_RegionalLocal_DAYS_LAST_DUE,
SUM(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_RegionalLocal_DAYS_TERMINATION,
SUM(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_RegionalLocal_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN CHANNEL_TYPE= 'Stone' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Stone_AMT_ANNUITY,
SUM(CASE WHEN CHANNEL_TYPE= 'Stone' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Stone_AMT_APPLICATION,
SUM(CASE WHEN CHANNEL_TYPE= 'Stone' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Stone_AMT_CREDIT,
SUM(CASE WHEN CHANNEL_TYPE= 'Stone' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Stone_AMT_DOWN_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Stone' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Stone_AMT_GOODS_PRICE,
SUM(CASE WHEN CHANNEL_TYPE= 'Stone' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Stone_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN CHANNEL_TYPE= 'Stone' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Stone_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN CHANNEL_TYPE= 'Stone' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Stone_RATE_DOWN_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Stone' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Stone_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN CHANNEL_TYPE= 'Stone' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Stone_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN CHANNEL_TYPE= 'Stone' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Stone_DAYS_DECISION,
SUM(CASE WHEN CHANNEL_TYPE= 'Stone' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Stone_SELLERPLACE_AREA,
SUM(CASE WHEN CHANNEL_TYPE= 'Stone' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Stone_CNT_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Stone' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Stone_DAYS_FIRST_DRAWING,
SUM(CASE WHEN CHANNEL_TYPE= 'Stone' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Stone_DAYS_FIRST_DUE,
SUM(CASE WHEN CHANNEL_TYPE= 'Stone' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Stone_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN CHANNEL_TYPE= 'Stone' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Stone_DAYS_LAST_DUE,
SUM(CASE WHEN CHANNEL_TYPE= 'Stone' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Stone_DAYS_TERMINATION,
SUM(CASE WHEN CHANNEL_TYPE= 'Stone' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Stone_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Countrywide_AMT_ANNUITY,
SUM(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Countrywide_AMT_APPLICATION,
SUM(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Countrywide_AMT_CREDIT,
SUM(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Countrywide_AMT_DOWN_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Countrywide_AMT_GOODS_PRICE,
SUM(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Countrywide_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Countrywide_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Countrywide_RATE_DOWN_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Countrywide_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Countrywide_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Countrywide_DAYS_DECISION,
SUM(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Countrywide_SELLERPLACE_AREA,
SUM(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Countrywide_CNT_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Countrywide_DAYS_FIRST_DRAWING,
SUM(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Countrywide_DAYS_FIRST_DUE,
SUM(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Countrywide_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Countrywide_DAYS_LAST_DUE,
SUM(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Countrywide_DAYS_TERMINATION,
SUM(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Countrywide_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Credit_and_cash_offices_AMT_ANNUITY,
SUM(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Credit_and_cash_offices_AMT_APPLICATION,
SUM(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Credit_and_cash_offices_AMT_CREDIT,
SUM(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Credit_and_cash_offices_AMT_DOWN_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Credit_and_cash_offices_AMT_GOODS_PRICE,
SUM(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Credit_and_cash_offices_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Credit_and_cash_offices_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Credit_and_cash_offices_RATE_DOWN_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Credit_and_cash_offices_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Credit_and_cash_offices_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Credit_and_cash_offices_DAYS_DECISION,
SUM(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Credit_and_cash_offices_SELLERPLACE_AREA,
SUM(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Credit_and_cash_offices_CNT_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Credit_and_cash_offices_DAYS_FIRST_DRAWING,
SUM(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Credit_and_cash_offices_DAYS_FIRST_DUE,
SUM(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Credit_and_cash_offices_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Credit_and_cash_offices_DAYS_LAST_DUE,
SUM(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Credit_and_cash_offices_DAYS_TERMINATION,
SUM(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_CHANNEL_TYPE_Credit_and_cash_offices_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Car_dealer_AMT_ANNUITY,
AVG(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Car_dealer_AMT_APPLICATION,
AVG(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Car_dealer_AMT_CREDIT,
AVG(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Car_dealer_AMT_DOWN_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Car_dealer_AMT_GOODS_PRICE,
AVG(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Car_dealer_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Car_dealer_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Car_dealer_RATE_DOWN_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Car_dealer_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Car_dealer_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Car_dealer_DAYS_DECISION,
AVG(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Car_dealer_SELLERPLACE_AREA,
AVG(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Car_dealer_CNT_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Car_dealer_DAYS_FIRST_DRAWING,
AVG(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Car_dealer_DAYS_FIRST_DUE,
AVG(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Car_dealer_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Car_dealer_DAYS_LAST_DUE,
AVG(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Car_dealer_DAYS_TERMINATION,
AVG(CASE WHEN CHANNEL_TYPE= 'Car dealer' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Car_dealer_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Channel_of_corporate_sales_AMT_ANNUITY,
AVG(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Channel_of_corporate_sales_AMT_APPLICATION,
AVG(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Channel_of_corporate_sales_AMT_CREDIT,
AVG(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Channel_of_corporate_sales_AMT_DOWN_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Channel_of_corporate_sales_AMT_GOODS_PRICE,
AVG(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Channel_of_corporate_sales_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Channel_of_corporate_sales_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Channel_of_corporate_sales_RATE_DOWN_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Channel_of_corporate_sales_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Channel_of_corporate_sales_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Channel_of_corporate_sales_DAYS_DECISION,
AVG(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Channel_of_corporate_sales_SELLERPLACE_AREA,
AVG(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Channel_of_corporate_sales_CNT_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Channel_of_corporate_sales_DAYS_FIRST_DRAWING,
AVG(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Channel_of_corporate_sales_DAYS_FIRST_DUE,
AVG(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Channel_of_corporate_sales_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Channel_of_corporate_sales_DAYS_LAST_DUE,
AVG(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Channel_of_corporate_sales_DAYS_TERMINATION,
AVG(CASE WHEN CHANNEL_TYPE= 'Channel of corporate sales' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Channel_of_corporate_sales_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_AP_Cash_loan_AMT_ANNUITY,
AVG(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_AP_Cash_loan_AMT_APPLICATION,
AVG(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_AP_Cash_loan_AMT_CREDIT,
AVG(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_AP_Cash_loan_AMT_DOWN_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_AP_Cash_loan_AMT_GOODS_PRICE,
AVG(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_AP_Cash_loan_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_AP_Cash_loan_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_AP_Cash_loan_RATE_DOWN_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_AP_Cash_loan_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_AP_Cash_loan_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_AP_Cash_loan_DAYS_DECISION,
AVG(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_AP_Cash_loan_SELLERPLACE_AREA,
AVG(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_AP_Cash_loan_CNT_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_AP_Cash_loan_DAYS_FIRST_DRAWING,
AVG(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_AP_Cash_loan_DAYS_FIRST_DUE,
AVG(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_AP_Cash_loan_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_AP_Cash_loan_DAYS_LAST_DUE,
AVG(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_AP_Cash_loan_DAYS_TERMINATION,
AVG(CASE WHEN CHANNEL_TYPE= 'AP+ (Cash loan)' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_AP_Cash_loan_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Contact_center_AMT_ANNUITY,
AVG(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Contact_center_AMT_APPLICATION,
AVG(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Contact_center_AMT_CREDIT,
AVG(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Contact_center_AMT_DOWN_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Contact_center_AMT_GOODS_PRICE,
AVG(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Contact_center_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Contact_center_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Contact_center_RATE_DOWN_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Contact_center_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Contact_center_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Contact_center_DAYS_DECISION,
AVG(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Contact_center_SELLERPLACE_AREA,
AVG(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Contact_center_CNT_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Contact_center_DAYS_FIRST_DRAWING,
AVG(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Contact_center_DAYS_FIRST_DUE,
AVG(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Contact_center_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Contact_center_DAYS_LAST_DUE,
AVG(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Contact_center_DAYS_TERMINATION,
AVG(CASE WHEN CHANNEL_TYPE= 'Contact center' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Contact_center_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_RegionalLocal_AMT_ANNUITY,
AVG(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_RegionalLocal_AMT_APPLICATION,
AVG(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_RegionalLocal_AMT_CREDIT,
AVG(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_RegionalLocal_AMT_DOWN_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_RegionalLocal_AMT_GOODS_PRICE,
AVG(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_RegionalLocal_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_RegionalLocal_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_RegionalLocal_RATE_DOWN_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_RegionalLocal_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_RegionalLocal_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_RegionalLocal_DAYS_DECISION,
AVG(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_RegionalLocal_SELLERPLACE_AREA,
AVG(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_RegionalLocal_CNT_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_RegionalLocal_DAYS_FIRST_DRAWING,
AVG(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_RegionalLocal_DAYS_FIRST_DUE,
AVG(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_RegionalLocal_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_RegionalLocal_DAYS_LAST_DUE,
AVG(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_RegionalLocal_DAYS_TERMINATION,
AVG(CASE WHEN CHANNEL_TYPE= 'Regional / Local' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_RegionalLocal_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN CHANNEL_TYPE= 'Stone' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Stone_AMT_ANNUITY,
AVG(CASE WHEN CHANNEL_TYPE= 'Stone' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Stone_AMT_APPLICATION,
AVG(CASE WHEN CHANNEL_TYPE= 'Stone' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Stone_AMT_CREDIT,
AVG(CASE WHEN CHANNEL_TYPE= 'Stone' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Stone_AMT_DOWN_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'Stone' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Stone_AMT_GOODS_PRICE,
AVG(CASE WHEN CHANNEL_TYPE= 'Stone' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Stone_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN CHANNEL_TYPE= 'Stone' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Stone_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN CHANNEL_TYPE= 'Stone' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Stone_RATE_DOWN_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'Stone' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Stone_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN CHANNEL_TYPE= 'Stone' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Stone_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN CHANNEL_TYPE= 'Stone' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Stone_DAYS_DECISION,
AVG(CASE WHEN CHANNEL_TYPE= 'Stone' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Stone_SELLERPLACE_AREA,
AVG(CASE WHEN CHANNEL_TYPE= 'Stone' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Stone_CNT_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'Stone' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Stone_DAYS_FIRST_DRAWING,
AVG(CASE WHEN CHANNEL_TYPE= 'Stone' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Stone_DAYS_FIRST_DUE,
AVG(CASE WHEN CHANNEL_TYPE= 'Stone' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Stone_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN CHANNEL_TYPE= 'Stone' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Stone_DAYS_LAST_DUE,
AVG(CASE WHEN CHANNEL_TYPE= 'Stone' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Stone_DAYS_TERMINATION,
AVG(CASE WHEN CHANNEL_TYPE= 'Stone' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Stone_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Countrywide_AMT_ANNUITY,
AVG(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Countrywide_AMT_APPLICATION,
AVG(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Countrywide_AMT_CREDIT,
AVG(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Countrywide_AMT_DOWN_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Countrywide_AMT_GOODS_PRICE,
AVG(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Countrywide_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Countrywide_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Countrywide_RATE_DOWN_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Countrywide_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Countrywide_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Countrywide_DAYS_DECISION,
AVG(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Countrywide_SELLERPLACE_AREA,
AVG(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Countrywide_CNT_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Countrywide_DAYS_FIRST_DRAWING,
AVG(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Countrywide_DAYS_FIRST_DUE,
AVG(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Countrywide_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Countrywide_DAYS_LAST_DUE,
AVG(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Countrywide_DAYS_TERMINATION,
AVG(CASE WHEN CHANNEL_TYPE= 'Country-wide' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Countrywide_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Credit_and_cash_offices_AMT_ANNUITY,
AVG(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Credit_and_cash_offices_AMT_APPLICATION,
AVG(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Credit_and_cash_offices_AMT_CREDIT,
AVG(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Credit_and_cash_offices_AMT_DOWN_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Credit_and_cash_offices_AMT_GOODS_PRICE,
AVG(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Credit_and_cash_offices_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Credit_and_cash_offices_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Credit_and_cash_offices_RATE_DOWN_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Credit_and_cash_offices_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Credit_and_cash_offices_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Credit_and_cash_offices_DAYS_DECISION,
AVG(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Credit_and_cash_offices_SELLERPLACE_AREA,
AVG(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Credit_and_cash_offices_CNT_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Credit_and_cash_offices_DAYS_FIRST_DRAWING,
AVG(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Credit_and_cash_offices_DAYS_FIRST_DUE,
AVG(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Credit_and_cash_offices_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Credit_and_cash_offices_DAYS_LAST_DUE,
AVG(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Credit_and_cash_offices_DAYS_TERMINATION,
AVG(CASE WHEN CHANNEL_TYPE= 'Credit and cash offices' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_CHANNEL_TYPE_Credit_and_cash_offices_NFLAG_INSURED_ON_APPROVAL
FROM
  df_previous_application
GROUP BY
  SK_ID_CURR
""")

###14 - Considerando a variável categórica NAME_SELLER_INDUSTRY

In [163]:
# Criando as agregações

agg = ['SUM', 'AVG']

# Criando a variável categórica

cat = ['NAME_SELLER_INDUSTRY']

# Criando os domínios

dominios_cat = ['MLM partners', 'Jewelry', 'Auto technology','Industry','Clothing','Construction','Furniture','Connectivity','Consumer electronics','XNA']

# Criando variável numérica

var_num = ['AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE','HOUR_APPR_PROCESS_START', 'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'DAYS_DECISION', 'SELLERPLACE_AREA', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL']

In [164]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN " + var_n + " ELSE 0 END) AS PA_" + ag + "_" + cate + "_" + dominio.replace(" ", "_").replace('/', '').replace('+', '').replace('(', '').replace(')', '').replace(':', '').replace('-', '').replace('__', '') + "_" + var_n + ',')

SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_MLM_partners_AMT_ANNUITY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_MLM_partners_AMT_APPLICATION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_MLM_partners_AMT_CREDIT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_MLM_partners_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_MLM_partners_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_MLM_partners_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_MLM_part

In [165]:
df_previous_application_contract_NAME_SELLER_INDUSTRY = spark.sql("""
SELECT
SK_ID_CURR,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_MLM_partners_AMT_ANNUITY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_MLM_partners_AMT_APPLICATION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_MLM_partners_AMT_CREDIT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_MLM_partners_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_MLM_partners_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_MLM_partners_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_MLM_partners_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_MLM_partners_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_MLM_partners_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_MLM_partners_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_MLM_partners_DAYS_DECISION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_MLM_partners_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_MLM_partners_CNT_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_MLM_partners_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_MLM_partners_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_MLM_partners_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_MLM_partners_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_MLM_partners_DAYS_TERMINATION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_MLM_partners_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Jewelry_AMT_ANNUITY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Jewelry_AMT_APPLICATION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Jewelry_AMT_CREDIT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Jewelry_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Jewelry_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Jewelry_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Jewelry_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Jewelry_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Jewelry_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Jewelry_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Jewelry_DAYS_DECISION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Jewelry_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Jewelry_CNT_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Jewelry_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Jewelry_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Jewelry_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Jewelry_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Jewelry_DAYS_TERMINATION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Jewelry_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Auto_technology_AMT_ANNUITY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Auto_technology_AMT_APPLICATION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Auto_technology_AMT_CREDIT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Auto_technology_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Auto_technology_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Auto_technology_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Auto_technology_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Auto_technology_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Auto_technology_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Auto_technology_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Auto_technology_DAYS_DECISION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Auto_technology_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Auto_technology_CNT_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Auto_technology_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Auto_technology_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Auto_technology_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Auto_technology_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Auto_technology_DAYS_TERMINATION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Auto_technology_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Industry_AMT_ANNUITY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Industry_AMT_APPLICATION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Industry_AMT_CREDIT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Industry_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Industry_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Industry_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Industry_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Industry_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Industry_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Industry_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Industry_DAYS_DECISION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Industry_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Industry_CNT_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Industry_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Industry_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Industry_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Industry_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Industry_DAYS_TERMINATION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Industry_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Clothing_AMT_ANNUITY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Clothing_AMT_APPLICATION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Clothing_AMT_CREDIT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Clothing_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Clothing_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Clothing_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Clothing_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Clothing_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Clothing_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Clothing_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Clothing_DAYS_DECISION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Clothing_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Clothing_CNT_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Clothing_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Clothing_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Clothing_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Clothing_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Clothing_DAYS_TERMINATION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Clothing_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Construction_AMT_ANNUITY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Construction_AMT_APPLICATION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Construction_AMT_CREDIT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Construction_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Construction_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Construction_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Construction_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Construction_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Construction_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Construction_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Construction_DAYS_DECISION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Construction_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Construction_CNT_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Construction_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Construction_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Construction_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Construction_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Construction_DAYS_TERMINATION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Construction_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Furniture_AMT_ANNUITY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Furniture_AMT_APPLICATION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Furniture_AMT_CREDIT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Furniture_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Furniture_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Furniture_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Furniture_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Furniture_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Furniture_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Furniture_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Furniture_DAYS_DECISION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Furniture_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Furniture_CNT_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Furniture_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Furniture_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Furniture_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Furniture_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Furniture_DAYS_TERMINATION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Furniture_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Connectivity_AMT_ANNUITY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Connectivity_AMT_APPLICATION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Connectivity_AMT_CREDIT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Connectivity_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Connectivity_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Connectivity_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Connectivity_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Connectivity_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Connectivity_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Connectivity_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Connectivity_DAYS_DECISION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Connectivity_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Connectivity_CNT_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Connectivity_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Connectivity_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Connectivity_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Connectivity_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Connectivity_DAYS_TERMINATION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Connectivity_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Consumer_electronics_AMT_ANNUITY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Consumer_electronics_AMT_APPLICATION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Consumer_electronics_AMT_CREDIT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Consumer_electronics_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Consumer_electronics_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Consumer_electronics_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Consumer_electronics_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Consumer_electronics_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Consumer_electronics_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Consumer_electronics_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Consumer_electronics_DAYS_DECISION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Consumer_electronics_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Consumer_electronics_CNT_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Consumer_electronics_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Consumer_electronics_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Consumer_electronics_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Consumer_electronics_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Consumer_electronics_DAYS_TERMINATION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_Consumer_electronics_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_XNA_AMT_ANNUITY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_XNA_AMT_APPLICATION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_XNA_AMT_CREDIT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_XNA_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_XNA_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_XNA_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_XNA_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_XNA_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_XNA_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_XNA_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_XNA_DAYS_DECISION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_XNA_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_XNA_CNT_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_XNA_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_XNA_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_XNA_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_XNA_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_XNA_DAYS_TERMINATION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_SELLER_INDUSTRY_XNA_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_MLM_partners_AMT_ANNUITY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_MLM_partners_AMT_APPLICATION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_MLM_partners_AMT_CREDIT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_MLM_partners_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_MLM_partners_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_MLM_partners_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_MLM_partners_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_MLM_partners_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_MLM_partners_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_MLM_partners_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_MLM_partners_DAYS_DECISION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_MLM_partners_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_MLM_partners_CNT_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_MLM_partners_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_MLM_partners_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_MLM_partners_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_MLM_partners_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_MLM_partners_DAYS_TERMINATION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM partners' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_MLM_partners_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Jewelry_AMT_ANNUITY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Jewelry_AMT_APPLICATION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Jewelry_AMT_CREDIT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Jewelry_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Jewelry_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Jewelry_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Jewelry_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Jewelry_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Jewelry_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Jewelry_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Jewelry_DAYS_DECISION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Jewelry_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Jewelry_CNT_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Jewelry_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Jewelry_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Jewelry_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Jewelry_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Jewelry_DAYS_TERMINATION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Jewelry_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Auto_technology_AMT_ANNUITY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Auto_technology_AMT_APPLICATION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Auto_technology_AMT_CREDIT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Auto_technology_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Auto_technology_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Auto_technology_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Auto_technology_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Auto_technology_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Auto_technology_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Auto_technology_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Auto_technology_DAYS_DECISION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Auto_technology_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Auto_technology_CNT_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Auto_technology_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Auto_technology_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Auto_technology_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Auto_technology_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Auto_technology_DAYS_TERMINATION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto technology' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Auto_technology_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Industry_AMT_ANNUITY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Industry_AMT_APPLICATION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Industry_AMT_CREDIT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Industry_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Industry_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Industry_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Industry_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Industry_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Industry_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Industry_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Industry_DAYS_DECISION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Industry_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Industry_CNT_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Industry_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Industry_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Industry_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Industry_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Industry_DAYS_TERMINATION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Industry_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Clothing_AMT_ANNUITY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Clothing_AMT_APPLICATION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Clothing_AMT_CREDIT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Clothing_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Clothing_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Clothing_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Clothing_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Clothing_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Clothing_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Clothing_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Clothing_DAYS_DECISION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Clothing_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Clothing_CNT_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Clothing_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Clothing_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Clothing_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Clothing_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Clothing_DAYS_TERMINATION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Clothing_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Construction_AMT_ANNUITY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Construction_AMT_APPLICATION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Construction_AMT_CREDIT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Construction_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Construction_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Construction_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Construction_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Construction_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Construction_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Construction_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Construction_DAYS_DECISION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Construction_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Construction_CNT_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Construction_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Construction_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Construction_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Construction_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Construction_DAYS_TERMINATION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Construction_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Furniture_AMT_ANNUITY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Furniture_AMT_APPLICATION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Furniture_AMT_CREDIT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Furniture_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Furniture_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Furniture_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Furniture_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Furniture_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Furniture_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Furniture_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Furniture_DAYS_DECISION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Furniture_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Furniture_CNT_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Furniture_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Furniture_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Furniture_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Furniture_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Furniture_DAYS_TERMINATION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Furniture_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Connectivity_AMT_ANNUITY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Connectivity_AMT_APPLICATION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Connectivity_AMT_CREDIT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Connectivity_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Connectivity_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Connectivity_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Connectivity_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Connectivity_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Connectivity_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Connectivity_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Connectivity_DAYS_DECISION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Connectivity_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Connectivity_CNT_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Connectivity_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Connectivity_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Connectivity_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Connectivity_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Connectivity_DAYS_TERMINATION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Connectivity_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Consumer_electronics_AMT_ANNUITY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Consumer_electronics_AMT_APPLICATION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Consumer_electronics_AMT_CREDIT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Consumer_electronics_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Consumer_electronics_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Consumer_electronics_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Consumer_electronics_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Consumer_electronics_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Consumer_electronics_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Consumer_electronics_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Consumer_electronics_DAYS_DECISION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Consumer_electronics_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Consumer_electronics_CNT_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Consumer_electronics_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Consumer_electronics_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Consumer_electronics_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Consumer_electronics_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Consumer_electronics_DAYS_TERMINATION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer electronics' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_Consumer_electronics_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_XNA_AMT_ANNUITY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_XNA_AMT_APPLICATION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_XNA_AMT_CREDIT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_XNA_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_XNA_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_XNA_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_XNA_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_XNA_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_XNA_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_XNA_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_XNA_DAYS_DECISION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_XNA_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_XNA_CNT_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_XNA_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_XNA_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_XNA_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_XNA_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_XNA_DAYS_TERMINATION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_SELLER_INDUSTRY_XNA_NFLAG_INSURED_ON_APPROVAL
FROM
  df_previous_application
GROUP BY
  SK_ID_CURR
""")

###15 - Considerando a variável categórica NAME_YIELD_GROUP

In [166]:
# Criando as agregações

agg = ['SUM', 'AVG']

# Criando a variável categórica

cat = ['NAME_YIELD_GROUP']

# Criando os domínios

dominios_cat = ['low_action', 'low_normal', 'high','middle','XNA']

# Criando variável numérica

var_num = ['AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE','HOUR_APPR_PROCESS_START', 'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'DAYS_DECISION', 'SELLERPLACE_AREA', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL']

In [167]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN " + var_n + " ELSE 0 END) AS PA_" + ag + "_" + cate + "_" + dominio.replace(" ", "_").replace('/', '').replace('+', '').replace('(', '').replace(')', '').replace(':', '').replace('-', '').replace('__', '') + "_" + var_n + ',')

SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_action_AMT_ANNUITY,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_action_AMT_APPLICATION,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_action_AMT_CREDIT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_action_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_action_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_action_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_action_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN RATE_

In [168]:
df_previous_application_contract_NAME_YIELD_GROUP = spark.sql("""
SELECT
SK_ID_CURR,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_action_AMT_ANNUITY,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_action_AMT_APPLICATION,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_action_AMT_CREDIT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_action_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_action_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_action_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_action_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_action_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_action_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_action_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_action_DAYS_DECISION,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_action_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_action_CNT_PAYMENT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_action_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_action_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_action_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_action_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_action_DAYS_TERMINATION,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_action_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_normal_AMT_ANNUITY,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_normal_AMT_APPLICATION,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_normal_AMT_CREDIT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_normal_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_normal_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_normal_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_normal_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_normal_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_normal_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_normal_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_normal_DAYS_DECISION,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_normal_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_normal_CNT_PAYMENT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_normal_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_normal_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_normal_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_normal_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_normal_DAYS_TERMINATION,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_low_normal_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_YIELD_GROUP= 'high' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_high_AMT_ANNUITY,
SUM(CASE WHEN NAME_YIELD_GROUP= 'high' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_high_AMT_APPLICATION,
SUM(CASE WHEN NAME_YIELD_GROUP= 'high' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_high_AMT_CREDIT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'high' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_high_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'high' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_high_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_YIELD_GROUP= 'high' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_high_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_YIELD_GROUP= 'high' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_high_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_YIELD_GROUP= 'high' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_high_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'high' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_high_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_YIELD_GROUP= 'high' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_high_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_YIELD_GROUP= 'high' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_high_DAYS_DECISION,
SUM(CASE WHEN NAME_YIELD_GROUP= 'high' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_high_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_YIELD_GROUP= 'high' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_high_CNT_PAYMENT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'high' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_high_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_YIELD_GROUP= 'high' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_high_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_YIELD_GROUP= 'high' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_high_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_YIELD_GROUP= 'high' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_high_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_YIELD_GROUP= 'high' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_high_DAYS_TERMINATION,
SUM(CASE WHEN NAME_YIELD_GROUP= 'high' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_high_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_middle_AMT_ANNUITY,
SUM(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_middle_AMT_APPLICATION,
SUM(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_middle_AMT_CREDIT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_middle_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_middle_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_middle_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_middle_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_middle_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_middle_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_middle_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_middle_DAYS_DECISION,
SUM(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_middle_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_middle_CNT_PAYMENT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_middle_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_middle_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_middle_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_middle_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_middle_DAYS_TERMINATION,
SUM(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_middle_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_XNA_AMT_ANNUITY,
SUM(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_XNA_AMT_APPLICATION,
SUM(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_XNA_AMT_CREDIT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_XNA_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_XNA_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_XNA_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_XNA_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_XNA_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_XNA_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_XNA_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_XNA_DAYS_DECISION,
SUM(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_XNA_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_XNA_CNT_PAYMENT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_XNA_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_XNA_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_XNA_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_XNA_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_XNA_DAYS_TERMINATION,
SUM(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_NAME_YIELD_GROUP_XNA_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_low_action_AMT_ANNUITY,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_low_action_AMT_APPLICATION,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_low_action_AMT_CREDIT,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_low_action_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_low_action_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_low_action_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_low_action_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_low_action_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_low_action_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_low_action_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_low_action_DAYS_DECISION,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_low_action_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_low_action_CNT_PAYMENT,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_low_action_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_low_action_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_low_action_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_low_action_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_low_action_DAYS_TERMINATION,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_low_action_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_low_normal_AMT_ANNUITY,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_low_normal_AMT_APPLICATION,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_low_normal_AMT_CREDIT,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_low_normal_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_low_normal_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_low_normal_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_low_normal_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_low_normal_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_low_normal_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_low_normal_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_low_normal_DAYS_DECISION,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_low_normal_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_low_normal_CNT_PAYMENT,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_low_normal_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_low_normal_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_low_normal_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_low_normal_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_low_normal_DAYS_TERMINATION,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_low_normal_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_YIELD_GROUP= 'high' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_high_AMT_ANNUITY,
AVG(CASE WHEN NAME_YIELD_GROUP= 'high' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_high_AMT_APPLICATION,
AVG(CASE WHEN NAME_YIELD_GROUP= 'high' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_high_AMT_CREDIT,
AVG(CASE WHEN NAME_YIELD_GROUP= 'high' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_high_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_YIELD_GROUP= 'high' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_high_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_YIELD_GROUP= 'high' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_high_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_YIELD_GROUP= 'high' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_high_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_YIELD_GROUP= 'high' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_high_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_YIELD_GROUP= 'high' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_high_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_YIELD_GROUP= 'high' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_high_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_YIELD_GROUP= 'high' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_high_DAYS_DECISION,
AVG(CASE WHEN NAME_YIELD_GROUP= 'high' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_high_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_YIELD_GROUP= 'high' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_high_CNT_PAYMENT,
AVG(CASE WHEN NAME_YIELD_GROUP= 'high' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_high_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_YIELD_GROUP= 'high' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_high_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_YIELD_GROUP= 'high' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_high_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_YIELD_GROUP= 'high' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_high_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_YIELD_GROUP= 'high' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_high_DAYS_TERMINATION,
AVG(CASE WHEN NAME_YIELD_GROUP= 'high' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_high_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_middle_AMT_ANNUITY,
AVG(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_middle_AMT_APPLICATION,
AVG(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_middle_AMT_CREDIT,
AVG(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_middle_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_middle_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_middle_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_middle_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_middle_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_middle_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_middle_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_middle_DAYS_DECISION,
AVG(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_middle_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_middle_CNT_PAYMENT,
AVG(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_middle_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_middle_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_middle_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_middle_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_middle_DAYS_TERMINATION,
AVG(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_middle_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_XNA_AMT_ANNUITY,
AVG(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_XNA_AMT_APPLICATION,
AVG(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_XNA_AMT_CREDIT,
AVG(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_XNA_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_XNA_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_XNA_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_XNA_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_XNA_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_XNA_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_XNA_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_XNA_DAYS_DECISION,
AVG(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_XNA_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_XNA_CNT_PAYMENT,
AVG(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_XNA_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_XNA_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_XNA_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_XNA_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_XNA_DAYS_TERMINATION,
AVG(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_NAME_YIELD_GROUP_XNA_NFLAG_INSURED_ON_APPROVAL
FROM
  df_previous_application
GROUP BY
  SK_ID_CURR
""")

###16 - Considerando a variável categórica PRODUCT_COMBINATION

In [169]:
# Criando as agregações

agg = ['SUM', 'AVG']

# Criando a variável categórica

cat = ['PRODUCT_COMBINATION']

# Criando os domínios

dominios_cat = ['null', 'POS others without interest', 'POS industry without interest','POS other with interest','POS mobile without interest','Cash Street: low','Cash Street: middle','Cash X-Sell: high','Cash Street: high', 'Card X-Sell','POS household without interest','POS industry with interest','Card Street','Cash X-Sell: low','Cash X-Sell: middle','POS mobile with interest','POS household with interest','Cash']

# Criando variável numérica

var_num = ['AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE','HOUR_APPR_PROCESS_START', 'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'DAYS_DECISION', 'SELLERPLACE_AREA', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL']

In [170]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN " + var_n + " ELSE 0 END) AS PA_" + ag + "_" + cate + "_" + dominio.replace(" ", "_").replace('/', '').replace('+', '').replace('(', '').replace(')', '').replace(':', '').replace('-', '').replace('__', '') + "_" + var_n + ',')

SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_null_AMT_ANNUITY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_null_AMT_APPLICATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_null_AMT_CREDIT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_null_AMT_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_null_AMT_GOODS_PRICE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_null_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_null_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_CO

In [171]:
df_previous_application_contract_PRODUCT_COMBINATION = spark.sql("""
SELECT
SK_ID_CURR,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_null_AMT_ANNUITY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_null_AMT_APPLICATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_null_AMT_CREDIT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_null_AMT_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_null_AMT_GOODS_PRICE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_null_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_null_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_null_RATE_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_null_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_null_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_null_DAYS_DECISION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_null_SELLERPLACE_AREA,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_null_CNT_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_null_DAYS_FIRST_DRAWING,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_null_DAYS_FIRST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_null_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_null_DAYS_LAST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_null_DAYS_TERMINATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_null_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_others_without_interest_AMT_ANNUITY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_others_without_interest_AMT_APPLICATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_others_without_interest_AMT_CREDIT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_others_without_interest_AMT_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_others_without_interest_AMT_GOODS_PRICE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_others_without_interest_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_others_without_interest_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_others_without_interest_RATE_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_others_without_interest_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_others_without_interest_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_others_without_interest_DAYS_DECISION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_others_without_interest_SELLERPLACE_AREA,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_others_without_interest_CNT_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_others_without_interest_DAYS_FIRST_DRAWING,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_others_without_interest_DAYS_FIRST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_others_without_interest_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_others_without_interest_DAYS_LAST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_others_without_interest_DAYS_TERMINATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_others_without_interest_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_industry_without_interest_AMT_ANNUITY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_industry_without_interest_AMT_APPLICATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_industry_without_interest_AMT_CREDIT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_industry_without_interest_AMT_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_industry_without_interest_AMT_GOODS_PRICE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_industry_without_interest_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_industry_without_interest_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_industry_without_interest_RATE_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_industry_without_interest_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_industry_without_interest_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_industry_without_interest_DAYS_DECISION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_industry_without_interest_SELLERPLACE_AREA,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_industry_without_interest_CNT_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_industry_without_interest_DAYS_FIRST_DRAWING,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_industry_without_interest_DAYS_FIRST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_industry_without_interest_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_industry_without_interest_DAYS_LAST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_industry_without_interest_DAYS_TERMINATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_industry_without_interest_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_other_with_interest_AMT_ANNUITY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_other_with_interest_AMT_APPLICATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_other_with_interest_AMT_CREDIT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_other_with_interest_AMT_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_other_with_interest_AMT_GOODS_PRICE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_other_with_interest_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_other_with_interest_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_other_with_interest_RATE_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_other_with_interest_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_other_with_interest_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_other_with_interest_DAYS_DECISION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_other_with_interest_SELLERPLACE_AREA,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_other_with_interest_CNT_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_other_with_interest_DAYS_FIRST_DRAWING,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_other_with_interest_DAYS_FIRST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_other_with_interest_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_other_with_interest_DAYS_LAST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_other_with_interest_DAYS_TERMINATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_other_with_interest_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_mobile_without_interest_AMT_ANNUITY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_mobile_without_interest_AMT_APPLICATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_mobile_without_interest_AMT_CREDIT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_mobile_without_interest_AMT_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_mobile_without_interest_AMT_GOODS_PRICE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_mobile_without_interest_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_mobile_without_interest_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_mobile_without_interest_RATE_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_mobile_without_interest_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_mobile_without_interest_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_mobile_without_interest_DAYS_DECISION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_mobile_without_interest_SELLERPLACE_AREA,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_mobile_without_interest_CNT_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_mobile_without_interest_DAYS_FIRST_DRAWING,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_mobile_without_interest_DAYS_FIRST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_mobile_without_interest_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_mobile_without_interest_DAYS_LAST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_mobile_without_interest_DAYS_TERMINATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_mobile_without_interest_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_low_AMT_ANNUITY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_low_AMT_APPLICATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_low_AMT_CREDIT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_low_AMT_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_low_AMT_GOODS_PRICE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_low_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_low_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_low_RATE_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_low_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_low_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_low_DAYS_DECISION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_low_SELLERPLACE_AREA,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_low_CNT_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_low_DAYS_FIRST_DRAWING,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_low_DAYS_FIRST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_low_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_low_DAYS_LAST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_low_DAYS_TERMINATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_low_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_middle_AMT_ANNUITY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_middle_AMT_APPLICATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_middle_AMT_CREDIT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_middle_AMT_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_middle_AMT_GOODS_PRICE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_middle_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_middle_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_middle_RATE_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_middle_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_middle_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_middle_DAYS_DECISION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_middle_SELLERPLACE_AREA,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_middle_CNT_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_middle_DAYS_FIRST_DRAWING,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_middle_DAYS_FIRST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_middle_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_middle_DAYS_LAST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_middle_DAYS_TERMINATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_middle_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_high_AMT_ANNUITY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_high_AMT_APPLICATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_high_AMT_CREDIT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_high_AMT_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_high_AMT_GOODS_PRICE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_high_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_high_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_high_RATE_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_high_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_high_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_high_DAYS_DECISION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_high_SELLERPLACE_AREA,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_high_CNT_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_high_DAYS_FIRST_DRAWING,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_high_DAYS_FIRST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_high_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_high_DAYS_LAST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_high_DAYS_TERMINATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_high_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: high' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_high_AMT_ANNUITY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: high' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_high_AMT_APPLICATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: high' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_high_AMT_CREDIT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: high' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_high_AMT_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: high' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_high_AMT_GOODS_PRICE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: high' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_high_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: high' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_high_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: high' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_high_RATE_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: high' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_high_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: high' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_high_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: high' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_high_DAYS_DECISION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: high' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_high_SELLERPLACE_AREA,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: high' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_high_CNT_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: high' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_high_DAYS_FIRST_DRAWING,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: high' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_high_DAYS_FIRST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: high' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_high_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: high' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_high_DAYS_LAST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: high' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_high_DAYS_TERMINATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: high' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_Street_high_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card X-Sell' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Card_XSell_AMT_ANNUITY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card X-Sell' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Card_XSell_AMT_APPLICATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card X-Sell' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Card_XSell_AMT_CREDIT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card X-Sell' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Card_XSell_AMT_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card X-Sell' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Card_XSell_AMT_GOODS_PRICE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card X-Sell' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Card_XSell_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card X-Sell' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Card_XSell_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card X-Sell' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Card_XSell_RATE_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card X-Sell' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Card_XSell_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card X-Sell' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Card_XSell_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card X-Sell' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Card_XSell_DAYS_DECISION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card X-Sell' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Card_XSell_SELLERPLACE_AREA,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card X-Sell' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Card_XSell_CNT_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card X-Sell' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Card_XSell_DAYS_FIRST_DRAWING,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card X-Sell' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Card_XSell_DAYS_FIRST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card X-Sell' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Card_XSell_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card X-Sell' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Card_XSell_DAYS_LAST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card X-Sell' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Card_XSell_DAYS_TERMINATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card X-Sell' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Card_XSell_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS household without interest' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_household_without_interest_AMT_ANNUITY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS household without interest' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_household_without_interest_AMT_APPLICATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS household without interest' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_household_without_interest_AMT_CREDIT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS household without interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_household_without_interest_AMT_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS household without interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_household_without_interest_AMT_GOODS_PRICE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS household without interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_household_without_interest_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS household without interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_household_without_interest_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS household without interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_household_without_interest_RATE_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS household without interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_household_without_interest_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS household without interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_household_without_interest_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS household without interest' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_household_without_interest_DAYS_DECISION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS household without interest' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_household_without_interest_SELLERPLACE_AREA,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS household without interest' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_household_without_interest_CNT_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS household without interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_household_without_interest_DAYS_FIRST_DRAWING,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS household without interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_household_without_interest_DAYS_FIRST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS household without interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_household_without_interest_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS household without interest' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_household_without_interest_DAYS_LAST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS household without interest' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_household_without_interest_DAYS_TERMINATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS household without interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_household_without_interest_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS industry with interest' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_industry_with_interest_AMT_ANNUITY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS industry with interest' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_industry_with_interest_AMT_APPLICATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS industry with interest' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_industry_with_interest_AMT_CREDIT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS industry with interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_industry_with_interest_AMT_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS industry with interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_industry_with_interest_AMT_GOODS_PRICE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS industry with interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_industry_with_interest_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS industry with interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_industry_with_interest_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS industry with interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_industry_with_interest_RATE_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS industry with interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_industry_with_interest_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS industry with interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_industry_with_interest_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS industry with interest' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_industry_with_interest_DAYS_DECISION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS industry with interest' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_industry_with_interest_SELLERPLACE_AREA,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS industry with interest' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_industry_with_interest_CNT_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS industry with interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_industry_with_interest_DAYS_FIRST_DRAWING,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS industry with interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_industry_with_interest_DAYS_FIRST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS industry with interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_industry_with_interest_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS industry with interest' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_industry_with_interest_DAYS_LAST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS industry with interest' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_industry_with_interest_DAYS_TERMINATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS industry with interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_industry_with_interest_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card Street' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Card_Street_AMT_ANNUITY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card Street' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Card_Street_AMT_APPLICATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card Street' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Card_Street_AMT_CREDIT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card Street' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Card_Street_AMT_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card Street' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Card_Street_AMT_GOODS_PRICE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card Street' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Card_Street_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card Street' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Card_Street_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card Street' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Card_Street_RATE_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card Street' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Card_Street_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card Street' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Card_Street_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card Street' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Card_Street_DAYS_DECISION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card Street' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Card_Street_SELLERPLACE_AREA,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card Street' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Card_Street_CNT_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card Street' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Card_Street_DAYS_FIRST_DRAWING,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card Street' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Card_Street_DAYS_FIRST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card Street' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Card_Street_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card Street' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Card_Street_DAYS_LAST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card Street' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Card_Street_DAYS_TERMINATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card Street' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Card_Street_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: low' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_low_AMT_ANNUITY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: low' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_low_AMT_APPLICATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: low' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_low_AMT_CREDIT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: low' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_low_AMT_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: low' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_low_AMT_GOODS_PRICE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: low' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_low_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: low' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_low_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: low' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_low_RATE_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: low' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_low_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: low' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_low_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: low' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_low_DAYS_DECISION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: low' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_low_SELLERPLACE_AREA,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: low' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_low_CNT_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: low' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_low_DAYS_FIRST_DRAWING,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: low' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_low_DAYS_FIRST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: low' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_low_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: low' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_low_DAYS_LAST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: low' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_low_DAYS_TERMINATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: low' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_low_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: middle' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_middle_AMT_ANNUITY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: middle' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_middle_AMT_APPLICATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: middle' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_middle_AMT_CREDIT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: middle' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_middle_AMT_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: middle' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_middle_AMT_GOODS_PRICE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: middle' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_middle_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: middle' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_middle_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: middle' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_middle_RATE_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: middle' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_middle_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: middle' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_middle_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: middle' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_middle_DAYS_DECISION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: middle' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_middle_SELLERPLACE_AREA,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: middle' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_middle_CNT_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: middle' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_middle_DAYS_FIRST_DRAWING,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: middle' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_middle_DAYS_FIRST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: middle' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_middle_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: middle' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_middle_DAYS_LAST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: middle' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_middle_DAYS_TERMINATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: middle' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_XSell_middle_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS mobile with interest' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_mobile_with_interest_AMT_ANNUITY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS mobile with interest' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_mobile_with_interest_AMT_APPLICATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS mobile with interest' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_mobile_with_interest_AMT_CREDIT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS mobile with interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_mobile_with_interest_AMT_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS mobile with interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_mobile_with_interest_AMT_GOODS_PRICE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS mobile with interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_mobile_with_interest_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS mobile with interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_mobile_with_interest_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS mobile with interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_mobile_with_interest_RATE_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS mobile with interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_mobile_with_interest_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS mobile with interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_mobile_with_interest_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS mobile with interest' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_mobile_with_interest_DAYS_DECISION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS mobile with interest' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_mobile_with_interest_SELLERPLACE_AREA,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS mobile with interest' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_mobile_with_interest_CNT_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS mobile with interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_mobile_with_interest_DAYS_FIRST_DRAWING,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS mobile with interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_mobile_with_interest_DAYS_FIRST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS mobile with interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_mobile_with_interest_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS mobile with interest' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_mobile_with_interest_DAYS_LAST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS mobile with interest' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_mobile_with_interest_DAYS_TERMINATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS mobile with interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_mobile_with_interest_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS household with interest' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_household_with_interest_AMT_ANNUITY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS household with interest' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_household_with_interest_AMT_APPLICATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS household with interest' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_household_with_interest_AMT_CREDIT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS household with interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_household_with_interest_AMT_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS household with interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_household_with_interest_AMT_GOODS_PRICE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS household with interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_household_with_interest_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS household with interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_household_with_interest_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS household with interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_household_with_interest_RATE_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS household with interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_household_with_interest_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS household with interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_household_with_interest_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS household with interest' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_household_with_interest_DAYS_DECISION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS household with interest' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_household_with_interest_SELLERPLACE_AREA,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS household with interest' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_household_with_interest_CNT_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS household with interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_household_with_interest_DAYS_FIRST_DRAWING,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS household with interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_household_with_interest_DAYS_FIRST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS household with interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_household_with_interest_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS household with interest' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_household_with_interest_DAYS_LAST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS household with interest' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_household_with_interest_DAYS_TERMINATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS household with interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_POS_household_with_interest_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN AMT_ANNUITY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_AMT_ANNUITY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN AMT_APPLICATION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_AMT_APPLICATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN AMT_CREDIT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_AMT_CREDIT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_AMT_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_AMT_GOODS_PRICE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_RATE_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN DAYS_DECISION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_DAYS_DECISION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_SELLERPLACE_AREA,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN CNT_PAYMENT ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_CNT_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_DAYS_FIRST_DRAWING,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_DAYS_FIRST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_DAYS_LAST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN DAYS_TERMINATION ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_DAYS_TERMINATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_SUM_PRODUCT_COMBINATION_Cash_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN PRODUCT_COMBINATION= 'null' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_null_AMT_ANNUITY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'null' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_null_AMT_APPLICATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'null' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_null_AMT_CREDIT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'null' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_null_AMT_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'null' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_null_AMT_GOODS_PRICE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'null' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_null_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN PRODUCT_COMBINATION= 'null' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_null_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'null' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_null_RATE_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'null' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_null_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'null' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_null_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN PRODUCT_COMBINATION= 'null' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_null_DAYS_DECISION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'null' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_null_SELLERPLACE_AREA,
AVG(CASE WHEN PRODUCT_COMBINATION= 'null' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_null_CNT_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'null' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_null_DAYS_FIRST_DRAWING,
AVG(CASE WHEN PRODUCT_COMBINATION= 'null' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_null_DAYS_FIRST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'null' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_null_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'null' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_null_DAYS_LAST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'null' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_null_DAYS_TERMINATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'null' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_null_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_others_without_interest_AMT_ANNUITY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_others_without_interest_AMT_APPLICATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_others_without_interest_AMT_CREDIT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_others_without_interest_AMT_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_others_without_interest_AMT_GOODS_PRICE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_others_without_interest_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_others_without_interest_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_others_without_interest_RATE_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_others_without_interest_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_others_without_interest_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_others_without_interest_DAYS_DECISION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_others_without_interest_SELLERPLACE_AREA,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_others_without_interest_CNT_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_others_without_interest_DAYS_FIRST_DRAWING,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_others_without_interest_DAYS_FIRST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_others_without_interest_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_others_without_interest_DAYS_LAST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_others_without_interest_DAYS_TERMINATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS others without interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_others_without_interest_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_industry_without_interest_AMT_ANNUITY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_industry_without_interest_AMT_APPLICATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_industry_without_interest_AMT_CREDIT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_industry_without_interest_AMT_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_industry_without_interest_AMT_GOODS_PRICE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_industry_without_interest_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_industry_without_interest_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_industry_without_interest_RATE_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_industry_without_interest_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_industry_without_interest_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_industry_without_interest_DAYS_DECISION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_industry_without_interest_SELLERPLACE_AREA,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_industry_without_interest_CNT_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_industry_without_interest_DAYS_FIRST_DRAWING,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_industry_without_interest_DAYS_FIRST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_industry_without_interest_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_industry_without_interest_DAYS_LAST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_industry_without_interest_DAYS_TERMINATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS industry without interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_industry_without_interest_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_other_with_interest_AMT_ANNUITY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_other_with_interest_AMT_APPLICATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_other_with_interest_AMT_CREDIT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_other_with_interest_AMT_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_other_with_interest_AMT_GOODS_PRICE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_other_with_interest_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_other_with_interest_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_other_with_interest_RATE_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_other_with_interest_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_other_with_interest_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_other_with_interest_DAYS_DECISION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_other_with_interest_SELLERPLACE_AREA,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_other_with_interest_CNT_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_other_with_interest_DAYS_FIRST_DRAWING,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_other_with_interest_DAYS_FIRST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_other_with_interest_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_other_with_interest_DAYS_LAST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_other_with_interest_DAYS_TERMINATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS other with interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_other_with_interest_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_mobile_without_interest_AMT_ANNUITY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_mobile_without_interest_AMT_APPLICATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_mobile_without_interest_AMT_CREDIT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_mobile_without_interest_AMT_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_mobile_without_interest_AMT_GOODS_PRICE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_mobile_without_interest_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_mobile_without_interest_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_mobile_without_interest_RATE_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_mobile_without_interest_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_mobile_without_interest_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_mobile_without_interest_DAYS_DECISION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_mobile_without_interest_SELLERPLACE_AREA,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_mobile_without_interest_CNT_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_mobile_without_interest_DAYS_FIRST_DRAWING,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_mobile_without_interest_DAYS_FIRST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_mobile_without_interest_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_mobile_without_interest_DAYS_LAST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_mobile_without_interest_DAYS_TERMINATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS mobile without interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_mobile_without_interest_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_low_AMT_ANNUITY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_low_AMT_APPLICATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_low_AMT_CREDIT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_low_AMT_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_low_AMT_GOODS_PRICE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_low_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_low_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_low_RATE_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_low_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_low_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_low_DAYS_DECISION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_low_SELLERPLACE_AREA,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_low_CNT_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_low_DAYS_FIRST_DRAWING,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_low_DAYS_FIRST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_low_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_low_DAYS_LAST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_low_DAYS_TERMINATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: low' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_low_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_middle_AMT_ANNUITY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_middle_AMT_APPLICATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_middle_AMT_CREDIT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_middle_AMT_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_middle_AMT_GOODS_PRICE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_middle_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_middle_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_middle_RATE_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_middle_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_middle_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_middle_DAYS_DECISION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_middle_SELLERPLACE_AREA,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_middle_CNT_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_middle_DAYS_FIRST_DRAWING,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_middle_DAYS_FIRST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_middle_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_middle_DAYS_LAST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_middle_DAYS_TERMINATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: middle' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_middle_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_high_AMT_ANNUITY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_high_AMT_APPLICATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_high_AMT_CREDIT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_high_AMT_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_high_AMT_GOODS_PRICE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_high_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_high_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_high_RATE_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_high_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_high_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_high_DAYS_DECISION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_high_SELLERPLACE_AREA,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_high_CNT_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_high_DAYS_FIRST_DRAWING,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_high_DAYS_FIRST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_high_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_high_DAYS_LAST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_high_DAYS_TERMINATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: high' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_high_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: high' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_high_AMT_ANNUITY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: high' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_high_AMT_APPLICATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: high' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_high_AMT_CREDIT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: high' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_high_AMT_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: high' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_high_AMT_GOODS_PRICE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: high' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_high_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: high' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_high_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: high' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_high_RATE_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: high' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_high_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: high' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_high_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: high' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_high_DAYS_DECISION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: high' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_high_SELLERPLACE_AREA,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: high' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_high_CNT_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: high' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_high_DAYS_FIRST_DRAWING,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: high' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_high_DAYS_FIRST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: high' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_high_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: high' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_high_DAYS_LAST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: high' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_high_DAYS_TERMINATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash Street: high' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_Street_high_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card X-Sell' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Card_XSell_AMT_ANNUITY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card X-Sell' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Card_XSell_AMT_APPLICATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card X-Sell' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Card_XSell_AMT_CREDIT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card X-Sell' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Card_XSell_AMT_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card X-Sell' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Card_XSell_AMT_GOODS_PRICE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card X-Sell' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Card_XSell_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card X-Sell' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Card_XSell_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card X-Sell' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Card_XSell_RATE_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card X-Sell' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Card_XSell_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card X-Sell' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Card_XSell_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card X-Sell' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Card_XSell_DAYS_DECISION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card X-Sell' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Card_XSell_SELLERPLACE_AREA,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card X-Sell' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Card_XSell_CNT_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card X-Sell' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Card_XSell_DAYS_FIRST_DRAWING,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card X-Sell' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Card_XSell_DAYS_FIRST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card X-Sell' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Card_XSell_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card X-Sell' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Card_XSell_DAYS_LAST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card X-Sell' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Card_XSell_DAYS_TERMINATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card X-Sell' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Card_XSell_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS household without interest' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_household_without_interest_AMT_ANNUITY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS household without interest' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_household_without_interest_AMT_APPLICATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS household without interest' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_household_without_interest_AMT_CREDIT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS household without interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_household_without_interest_AMT_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS household without interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_household_without_interest_AMT_GOODS_PRICE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS household without interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_household_without_interest_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS household without interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_household_without_interest_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS household without interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_household_without_interest_RATE_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS household without interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_household_without_interest_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS household without interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_household_without_interest_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS household without interest' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_household_without_interest_DAYS_DECISION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS household without interest' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_household_without_interest_SELLERPLACE_AREA,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS household without interest' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_household_without_interest_CNT_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS household without interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_household_without_interest_DAYS_FIRST_DRAWING,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS household without interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_household_without_interest_DAYS_FIRST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS household without interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_household_without_interest_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS household without interest' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_household_without_interest_DAYS_LAST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS household without interest' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_household_without_interest_DAYS_TERMINATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS household without interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_household_without_interest_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS industry with interest' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_industry_with_interest_AMT_ANNUITY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS industry with interest' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_industry_with_interest_AMT_APPLICATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS industry with interest' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_industry_with_interest_AMT_CREDIT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS industry with interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_industry_with_interest_AMT_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS industry with interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_industry_with_interest_AMT_GOODS_PRICE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS industry with interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_industry_with_interest_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS industry with interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_industry_with_interest_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS industry with interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_industry_with_interest_RATE_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS industry with interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_industry_with_interest_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS industry with interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_industry_with_interest_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS industry with interest' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_industry_with_interest_DAYS_DECISION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS industry with interest' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_industry_with_interest_SELLERPLACE_AREA,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS industry with interest' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_industry_with_interest_CNT_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS industry with interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_industry_with_interest_DAYS_FIRST_DRAWING,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS industry with interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_industry_with_interest_DAYS_FIRST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS industry with interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_industry_with_interest_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS industry with interest' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_industry_with_interest_DAYS_LAST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS industry with interest' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_industry_with_interest_DAYS_TERMINATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS industry with interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_industry_with_interest_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card Street' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Card_Street_AMT_ANNUITY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card Street' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Card_Street_AMT_APPLICATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card Street' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Card_Street_AMT_CREDIT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card Street' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Card_Street_AMT_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card Street' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Card_Street_AMT_GOODS_PRICE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card Street' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Card_Street_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card Street' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Card_Street_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card Street' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Card_Street_RATE_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card Street' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Card_Street_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card Street' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Card_Street_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card Street' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Card_Street_DAYS_DECISION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card Street' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Card_Street_SELLERPLACE_AREA,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card Street' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Card_Street_CNT_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card Street' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Card_Street_DAYS_FIRST_DRAWING,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card Street' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Card_Street_DAYS_FIRST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card Street' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Card_Street_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card Street' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Card_Street_DAYS_LAST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card Street' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Card_Street_DAYS_TERMINATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card Street' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Card_Street_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: low' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_low_AMT_ANNUITY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: low' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_low_AMT_APPLICATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: low' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_low_AMT_CREDIT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: low' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_low_AMT_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: low' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_low_AMT_GOODS_PRICE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: low' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_low_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: low' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_low_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: low' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_low_RATE_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: low' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_low_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: low' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_low_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: low' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_low_DAYS_DECISION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: low' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_low_SELLERPLACE_AREA,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: low' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_low_CNT_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: low' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_low_DAYS_FIRST_DRAWING,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: low' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_low_DAYS_FIRST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: low' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_low_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: low' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_low_DAYS_LAST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: low' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_low_DAYS_TERMINATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: low' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_low_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: middle' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_middle_AMT_ANNUITY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: middle' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_middle_AMT_APPLICATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: middle' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_middle_AMT_CREDIT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: middle' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_middle_AMT_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: middle' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_middle_AMT_GOODS_PRICE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: middle' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_middle_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: middle' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_middle_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: middle' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_middle_RATE_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: middle' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_middle_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: middle' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_middle_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: middle' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_middle_DAYS_DECISION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: middle' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_middle_SELLERPLACE_AREA,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: middle' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_middle_CNT_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: middle' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_middle_DAYS_FIRST_DRAWING,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: middle' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_middle_DAYS_FIRST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: middle' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_middle_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: middle' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_middle_DAYS_LAST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: middle' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_middle_DAYS_TERMINATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash X-Sell: middle' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_XSell_middle_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS mobile with interest' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_mobile_with_interest_AMT_ANNUITY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS mobile with interest' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_mobile_with_interest_AMT_APPLICATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS mobile with interest' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_mobile_with_interest_AMT_CREDIT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS mobile with interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_mobile_with_interest_AMT_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS mobile with interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_mobile_with_interest_AMT_GOODS_PRICE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS mobile with interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_mobile_with_interest_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS mobile with interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_mobile_with_interest_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS mobile with interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_mobile_with_interest_RATE_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS mobile with interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_mobile_with_interest_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS mobile with interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_mobile_with_interest_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS mobile with interest' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_mobile_with_interest_DAYS_DECISION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS mobile with interest' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_mobile_with_interest_SELLERPLACE_AREA,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS mobile with interest' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_mobile_with_interest_CNT_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS mobile with interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_mobile_with_interest_DAYS_FIRST_DRAWING,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS mobile with interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_mobile_with_interest_DAYS_FIRST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS mobile with interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_mobile_with_interest_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS mobile with interest' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_mobile_with_interest_DAYS_LAST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS mobile with interest' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_mobile_with_interest_DAYS_TERMINATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS mobile with interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_mobile_with_interest_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS household with interest' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_household_with_interest_AMT_ANNUITY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS household with interest' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_household_with_interest_AMT_APPLICATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS household with interest' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_household_with_interest_AMT_CREDIT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS household with interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_household_with_interest_AMT_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS household with interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_household_with_interest_AMT_GOODS_PRICE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS household with interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_household_with_interest_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS household with interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_household_with_interest_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS household with interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_household_with_interest_RATE_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS household with interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_household_with_interest_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS household with interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_household_with_interest_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS household with interest' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_household_with_interest_DAYS_DECISION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS household with interest' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_household_with_interest_SELLERPLACE_AREA,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS household with interest' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_household_with_interest_CNT_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS household with interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_household_with_interest_DAYS_FIRST_DRAWING,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS household with interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_household_with_interest_DAYS_FIRST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS household with interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_household_with_interest_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS household with interest' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_household_with_interest_DAYS_LAST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS household with interest' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_household_with_interest_DAYS_TERMINATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS household with interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_POS_household_with_interest_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN AMT_ANNUITY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_AMT_ANNUITY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN AMT_APPLICATION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_AMT_APPLICATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN AMT_CREDIT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_AMT_CREDIT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_AMT_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN AMT_GOODS_PRICE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_AMT_GOODS_PRICE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_RATE_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN DAYS_DECISION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_DAYS_DECISION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN SELLERPLACE_AREA ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_SELLERPLACE_AREA,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN CNT_PAYMENT ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_CNT_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_DAYS_FIRST_DRAWING,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN DAYS_FIRST_DUE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_DAYS_FIRST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN DAYS_LAST_DUE ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_DAYS_LAST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN DAYS_TERMINATION ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_DAYS_TERMINATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS PA_AVG_PRODUCT_COMBINATION_Cash_NFLAG_INSURED_ON_APPROVAL
FROM
  df_previous_application
GROUP BY
  SK_ID_CURR
""")

##Criação da tabela df_previous_application_contract_num_and_cat

In [172]:
df_previous_application_contract_num_and_cat = df_previous_application_contract_NAME_CONTRACT_TYPE.join(df_previous_application_contract_WEEKDAY_APPR_PROCESS_START, "SK_ID_CURR",  how = 'left')\
               .join(df_previous_application_contract_FLAG_LAST_APPL_PER_CONTRACT, "SK_ID_CURR",  how = 'left')\
               .join(df_previous_application_contract_NAME_CASH_LOAN_PURPOSE, "SK_ID_CURR",  how = 'left')\
               .join(df_previous_application_contract_NAME_CONTRACT_STATUS, "SK_ID_CURR",  how = 'left')\
               .join(df_previous_application_contract_NAME_PAYMENT_TYPE, "SK_ID_CURR",  how = 'left')\
               .join(df_previous_application_contract_CODE_REJECT_REASON, "SK_ID_CURR",  how = 'left')\
               .join(df_previous_application_contract_NAME_TYPE_SUITE, "SK_ID_CURR",  how = 'left')\
               .join(df_previous_application_contract_NAME_CLIENT_TYPE, "SK_ID_CURR",  how = 'left')\
               .join(df_previous_application_contract_NAME_GOODS_CATEGORY, "SK_ID_CURR",  how = 'left')\
               .join(df_previous_application_contract_NAME_PORTFOLIO, "SK_ID_CURR",  how = 'left')\
               .join(df_previous_application_contract_NAME_PRODUCT_TYPE, "SK_ID_CURR",  how = 'left')\
               .join(df_previous_application_contract_CHANNEL_TYPE, "SK_ID_CURR",  how = 'left')\
               .join(df_previous_application_contract_NAME_SELLER_INDUSTRY, "SK_ID_CURR",  how = 'left')\
               .join(df_previous_application_contract_NAME_YIELD_GROUP, "SK_ID_CURR",  how = 'left')\
               .join(df_previous_application_contract_PRODUCT_COMBINATION, "SK_ID_CURR",  how = 'left')

In [173]:
df_previous_application_contract_num_and_cat.createOrReplaceTempView("df_previous_application_contract_num_and_cat")

In [175]:
#df_previous_application_num.coalesce(1).write.csv('/content/gdrive/MyDrive/11 - Hackaton/dados_etl/df_previous_application_num.csv')

In [176]:
#df_previous_application_contract_num_and_cat.toPandas().to_csv('/content/gdrive/MyDrive/11 - Hackaton/dados_etl/df_previous_application_contract_num_and_cat.csv', index=None)

##Analisando quantidade de novas variáveis explicativas nós conseguimos criar após as etapas anteriores

In [177]:
# Conta o número de colunas na tabela
num_colunas_base = len(df_previous_application.columns)
num_colunas_base_cat = len(df_previous_application_cat.columns)
num_colunas_base_num = len(df_previous_application_num.columns)
num_colunas_base_cat_e_num = len(df_previous_application_contract_num_and_cat.columns)

# Imprime o resultado
print("A base original previous_application", num_colunas_base, "colunas.")
print('')
print("A base categórica da previous_application possui", num_colunas_base_cat, " novas colunas.")
print('')
print("A base numérica da previous_application possui", num_colunas_base_num, " novas colunas.")
print('')
print("A base numérica com categórica da previous_application possui", num_colunas_base_cat_e_num, " novas colunas.")
print('')

A base original previous_application 37 colunas.

A base categórica da previous_application possui 521  novas colunas.

A base numérica da previous_application possui 77  novas colunas.

A base numérica com categórica da previous_application possui 4941  novas colunas.



In [178]:
df_application_train = spark.read.csv("/content/gdrive/MyDrive/HD/CURSOS/Ciência - Análise de Dados/Pod Academy/Formação em Big Data e Analytics/Hackathon - Ciência de Dados/11 - Hackaton/dados/application_train.csv",sep=',',header=True, encoding='utf-8', inferSchema=True)
df_application_train.createOrReplaceTempView("df_application_train")

In [179]:
# Conta o número de colunas na tabela
num_colunas_base = len(df_application_train.columns)

# Imprime o resultado
print("A base original application_train", num_colunas_base, "colunas.")
print('')

A base original application_train 172 colunas.



In [180]:
df_application_train.count()

215257

Além do dataframe df_application_train que tem 172 colunas e do df_previous_application que tem 37 colunas, criamos mais 3 novos dataframes utilizando agregações e regras de negócio, onde:

- df_previous_application_cat: 521 colunas
- df_previous_application_num: 77 colunas
- df_previous_application_contract_num_and_cat: 4941 colunas

Lembrando que para todos os dataframes temos que o número de variáveis é igual ao número de colunas - 1, pois a coluna 'SK_ID_CURR' é a chave de todos os dataframes.

Inicialmente tínhamos 171 variáveis para a tabela df_application_train e 36 variáveis para o df_previous_application, que totalizam 207 variáveis.

##Juntando ao df_application_train os 3 dataframes criados usando a coluna SK_ID_CURR

In [181]:
df_previous_application_junto = df_application_train.join(df_previous_application_cat, "SK_ID_CURR",  how = 'left')\
               .join(df_previous_application_num, "SK_ID_CURR",  how = 'left')\
               .join(df_previous_application_contract_num_and_cat, "SK_ID_CURR",  how = 'left')

In [182]:
df_previous_application_junto.count()

215257

##Calculando o número de colunas do dataframe df_junto

In [183]:
# Contando o número de colunas do df_previous_application_junto
num_colunas = len(df_previous_application_junto.columns)

# Imprimindo o resultado
print(num_colunas, "colunas.")

5708 colunas.


In [184]:
df_previous_application_junto.createOrReplaceTempView("df_previous_application_junto")

#Conclusão

Então, temos o dataframe final df_junto com 5708 variáveis. De 207 variáveis iniciais passamos para 5708 variáveis, ou seja, aumentamos o número de variáveis em aproximandamente 27,5 vezes.

In [185]:
#df_previous_application_junto.toPandas().to_csv('/content/gdrive/MyDrive/11 - Hackaton/dados_etl/df_previous_application_junto.csv', index=None)